### **_Importar librerias_**

In [56]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones 
import re
from unidecode import unidecode
import unicodedata
import datetime


# visuzalizaciones
import missingno as msno
import matplotlib as plt 
import plotly.express as px 

In [57]:
pd.set_option("display.max_columns",None)

### **_Lectura de datos_**

In [58]:
datos_pisosmadrid_normal = pd.read_excel("PisosMadrid.xlsx")
datos_pisosmadrid_detalles = pd.read_excel("Detalle_pisosmadrid.xlsx")
dbcp_ordenado= pd.read_excel("distr_barrios_codpostal.xlsx")
# redpiso_normal= pd.read_excel("Redpiso.xlsx")
# redpiso_detalles= pd.read_excel("Detalle_RedPiso.xlsx")

In [59]:
datos_pisosmadrid_normal.head()

,Campo,Campo1,Precio,dormitorios,banos,m2construidos,Descripción,btn
0,https://www.pisosmadrid.com.es/propiedad/40866...,"Piso en venta en Ibiza, Madrid",520.000 €,3 Dormitorio(s),1 baño,70 m2,Precioso piso reformado de 60m2 enfrente del R...,Detalles
1,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Madrid,329.000 €,3 Dormitorio(s),2 Baño(s),95 m2,~SERVICHECK INMOBILIARIA comercializa un preci...,Detalles
2,https://www.pisosmadrid.com.es/propiedad/40865...,"Piso en venta en Arganzuela, Madrid",740.000 €,4 Dormitorio(s),3 Baño(s),158 m2,Se ofrece una magnífica oportunidad para adqui...,Detalles
3,https://www.pisosmadrid.com.es/propiedad/40864...,"Piso en venta en Los Angeles, Madrid",70.256 €,3 Dormitorio(s),1 baño,57 m2,"Vivienda situada en Calle Godella,, en Madrid ...",Detalles
4,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en Madrid,97.920 €,2 Dormitorio(s),1 baño,50 m2,"Vivienda situada en Calle Transmonte,, en Madr...",Detalles


In [60]:
# Bien aqui vamos a construir una funcion que recorra todo el dataframe y tome los caracteres especiales 
# las comas, los puntos, entre otros y los reemplaza con nada o sea basicamente eliminarlos. 

def limpiar_texto(x):
    return re.sub(r'[^\w\s]','',unidecode(str(x)))

datos_pisosmadrid_normal = datos_pisosmadrid_normal.apply(lambda col: col.apply(limpiar_texto)if col.name != "Campo"else col)

In [61]:
#  Aqui creamos una funcion que recorrera la columna que especificamos quitanto el valor que nosotoros le digamos y sin distincion 
# alguna sobre si es mayuscula o minuscula.                                                                                 
def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bdormitorios\b', '', palabra)

datos_pisosmadrid_normal["dormitorios"]= datos_pisosmadrid_normal["dormitorios"].apply(eliminar_palabra)

# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bbano\b', '', palabra)

datos_pisosmadrid_normal["banos"]= datos_pisosmadrid_normal["banos"].apply(eliminar_palabra)

# --------------------------------------------------------------------------
def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bbanos\b', '', palabra)

datos_pisosmadrid_normal["banos"]= datos_pisosmadrid_normal["banos"].apply(eliminar_palabra)
# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bm2\b', '', palabra)

datos_pisosmadrid_normal["m2construidos"]= datos_pisosmadrid_normal["m2construidos"].apply(eliminar_palabra)
# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bEUR\b', '', palabra)

datos_pisosmadrid_normal["Precio"]= datos_pisosmadrid_normal["Precio"].apply(eliminar_palabra)

In [62]:
# Aqiu se asignaron los nuesvos nombres por cada columna de forma indivual.
# El proceso tambien se puede hacer de forma mas corta.... Declaras una variable nueva con los nombres que quieres darles a las columnas
# en el orden que quieres poner los nombres y despues los asignas a las columnas.
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Campo":"url_website"})
 # --------------------------------------------------------------------------                  
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Campo1":"descripcion"})
 # --------------------------------------------------------------------------
                    
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Precio":"precio"})
  # --------------------------------------------------------------------------
                   
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"dormitorios":"cant_dormitorios"})
 # --------------------------------------------------------------------------
                    
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"banos":"cant_banos"})
# --------------------------------------------------------------------------

datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"m2construidos":"metros_cuadrados_construidos"})
  # --------------------------------------------------------------------------
                   
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Descripción":"descripcion_extendida"})
   # --------------------------------------------------------------------------
                  
datos_pisosmadrid_normal = datos_pisosmadrid_normal.drop(columns=["btn"])  

In [63]:
datos_pisosmadrid_normal.describe()

,url_website,descripcion,precio,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida
count,1200,1200,1200,1200,1200,1200,1200
unique,1200,415,631,12,12,306,1084
top,https://www.pisosmadrid.com.es/propiedad/40866...,Piso en venta en Madrid,5300,0,1,70,RAYMU vende todo tipo de inmuebles en la Comun...
freq,1,103,13,330,417,21,38


### **_Aqui se termina la parte de --pisosmadrid_normal-- y empieza la de --pisosmadrid_detalles--_**

In [64]:
datos_pisosmadrid_detalles["Campo18"].unique()

array(['Jardín:', nan], dtype=object)

In [65]:
datos_pisosmadrid_detalles["Campo12"].value_counts()

Campo12
0.0     327
3.0     319
2.0     272
1.0     109
4.0      98
5.0      34
6.0      12
7.0       7
8.0       5
9.0       2
10.0      1
20.0      1
Name: count, dtype: int64

In [66]:
datos_pisosmadrid_detalles["Campo1"].value_counts()

Campo1
0 Dormitorio(s)     327
3 Dormitorio(s)     319
2 Dormitorio(s)     272
1 dormitorio        109
4 Dormitorio(s)      98
5 Dormitorio(s)      34
6 Dormitorio(s)      12
7 Dormitorio(s)       7
8 Dormitorio(s)       5
9 Dormitorio(s)       2
10 Dormitorio(s)      1
20 Dormitorio(s)      1
Name: count, dtype: int64

In [67]:
datos_pisosmadrid_detalles["Campo14"].value_counts()

Campo14
1.0     416
2.0     350
0.0     266
3.0     108
4.0      25
5.0       9
6.0       6
7.0       3
31.0      1
10.0      1
11.0      1
8.0       1
Name: count, dtype: int64

In [68]:
datos_pisosmadrid_detalles["Campo2"].value_counts()

Campo2
1 baño        416
2 Baño(s)     350
0 Baño(s)     266
3 Baño(s)     108
4 Baño(s)      25
5 Baño(s)       9
6 Baño(s)       6
7 Baño(s)       3
31 Baño(s)      1
10 Baño(s)      1
11 Baño(s)      1
8 Baño(s)       1
Name: count, dtype: int64

In [69]:
datos_pisosmadrid_detalles["Campo3"].value_counts()

Campo3
70 m2      21
0 m2       18
22 m2      18
m2         16
80 m2      15
           ..
18 m2       1
314 m2      1
15 m2       1
208 m2      1
3995 m2     1
Name: count, Length: 305, dtype: int64

In [70]:
datos_pisosmadrid_detalles["Metros"].value_counts()

Metros
70.0      21
0.0       18
22.0      18
80.0      15
65.0      15
          ..
726.0      1
769.0      1
460.0      1
1600.0     1
3995.0     1
Name: count, Length: 304, dtype: int64

In [71]:
# Aqui se eliminaron una lista de columnas elegidas por mi. Se hizo un previo analisis del valor que proporcionaban cada una y la conclusion 
# fue que el valor que proprcionaban era igual o menor que nada.
datos_pisosmadrid_detalles = datos_pisosmadrid_detalles.drop(columns=["Campo4","Campo6","Campo7","Campo8","Campo9","Campo10","Campo15","Campo16","Campo17","Campo18","Provincia","Campo11",
                            "Campo12","Campo14", "Campo13","Metros"])

In [72]:
def limpiar_texto(x):
    return re.sub(r'[^\w\s]','',unidecode(str(x)))
datos_pisosmadrid_detalles = datos_pisosmadrid_detalles.apply(lambda col: col.apply(limpiar_texto)if col.name !="Publicado" else col)

In [73]:
# Aqui creamos una funcion que recorrera la columna que especificamos quitanto el valor que nosotoros le digamos y sin distincion 
# alguna sobre si es mayuscula o minuscula.
def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bdormitorios\b', '', palabra)

datos_pisosmadrid_detalles["Campo1"]= datos_pisosmadrid_detalles["Campo1"].apply(eliminar_palabra)

# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bbano\b', '', palabra)

datos_pisosmadrid_detalles["Campo2"]= datos_pisosmadrid_detalles["Campo2"].apply(eliminar_palabra)

# --------------------------------------------------------------------------
def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bbanos\b', '', palabra)

datos_pisosmadrid_detalles["Campo2"]= datos_pisosmadrid_detalles["Campo2"].apply(eliminar_palabra)
# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bm2\b', '', palabra)

datos_pisosmadrid_detalles["Campo3"]= datos_pisosmadrid_detalles["Campo3"].apply(eliminar_palabra)
# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bEUR\b', '', palabra)

datos_pisosmadrid_detalles["Texto"]= datos_pisosmadrid_detalles["Texto"].apply(eliminar_palabra)

In [74]:

nuevos_nombres = ["descripcion","cant_dormitorios","cant_banos","m_cuadrados","descripcion2","tipo_lugar","ciudad","barrio/localidad/distrito","piscina","garaje",
                  "jardin","descripcion3","referencia","fecha_publicacion","n.visitas","precio"]
datos_pisosmadrid_detalles.columns= nuevos_nombres

In [75]:
# Aqui vamos a convertir el tipo de dato que tiene una columna en especifico al tipo datetime. 
datos_pisosmadrid_detalles["fecha_publicacion"]=pd.to_datetime(datos_pisosmadrid_detalles["fecha_publicacion"], dayfirst=True)

In [76]:
# Aqui vamos a hacer una serie de reemplazamientos indivuales . 


datos_pisosmadrid_detalles["descripcion3"]= datos_pisosmadrid_detalles["descripcion3"].str.replace(r'Caracteristicas\n\t\t\t\t', "",regex=True)
datos_pisosmadrid_detalles["descripcion3"]= datos_pisosmadrid_detalles["descripcion3"].str.replace(r'Caracteristicas',"",regex=True).replace("",np.nan)

datos_pisosmadrid_detalles["descripcion2"]= datos_pisosmadrid_detalles["descripcion2"].str.replace(r'Descripcion\n\t\t\t\t', "",regex=True)
datos_pisosmadrid_detalles["descripcion2"]= datos_pisosmadrid_detalles["descripcion2"].str.replace(r'Descripcion\n', "",regex=True)


In [77]:
columnas_01= ["piscina","garaje","jardin"]

datos_pisosmadrid_detalles[columnas_01]=datos_pisosmadrid_detalles[columnas_01].replace({"Si":1,"No":0})

In [78]:
# Aqui vamos a crear una variabel que guarde un diccionario con las palabras que quremos tomar de la columna descripcion3 y el nombre
# de la columna a la que queremos que la palabra se vaya.
palabras_a_columnas = {
    "Calefaccion" : "calefaccion",
    "Trastero"    : "trastero",
    "Amueblado"   :"amueblado",
    "Chimenea"    :"chimenea",
    "Lavadero"    :"lavadero",
    "Videoportero":"videoportero",
    "Alarma"      :"alarma",
    "Aire acondicionado": "aire_acondicionado"
}

# Aqui creamos una variable donde guardaremos el nombre de la columna de donde se van a extraer todas esas palabras mencionadas antes.
columna_original = 'descripcion3'

# Aqui vamos a crear un bucle for que itera sobre las columnas y encuentra los valores que escribimos antes. 
for palabra, nueva_columna in palabras_a_columnas.items():
    # Usar apply para crear la columna con 1 si la palabra está presente, de lo contrario 0
    datos_pisosmadrid_detalles[nueva_columna] = datos_pisosmadrid_detalles[columna_original].apply(lambda x: 1 if isinstance(x, str) and palabra.lower() in x.lower() else 0)

# Este bucle itera sobre las claves del diccionario palabras_a_columnas. 

for palabra in palabras_a_columnas.keys():
    datos_pisosmadrid_detalles[columna_original] = datos_pisosmadrid_detalles[columna_original].str.replace(rf'\b{palabra}\b', '', regex=True)
datos_pisosmadrid_detalles[columna_original] = datos_pisosmadrid_detalles[columna_original].str.strip()
# Reemplazar valores vacíos o "nan" con NaN en la columna original
datos_pisosmadrid_detalles[columna_original] = datos_pisosmadrid_detalles[columna_original].replace(["", "nan"], np.nan)



C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\3063665124.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datos_pisosmadrid_detalles[columna_original] = datos_pisosmadrid_detalles[columna_original].replace(["", "nan"], np.nan)


In [79]:
# Aqui eliminamos la columna de la que sacamos todas las palabras anteriores, ahora solo queda con un nulos asi que hay que dejarla ir.
datos_pisosmadrid_detalles= datos_pisosmadrid_detalles.drop("descripcion3",axis=1)

In [80]:
# datos_pisosmadrid_detalles = datos_pisosmadrid_detalles.to_csv("pisosmadrid_detalles_tratado.csv")

### **_EDA/ Exploratory Data Analysis y Preprocesamiento PisosMadrid normal y detalle_**

In [81]:
datos_pisosmadrid_detalles.shape

(1188, 23)

In [82]:
datos_pisosmadrid_normal.shape

(1200, 7)

In [83]:
datos_pisosmadrid_normal.head(3)

,url_website,descripcion,precio,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida
0,https://www.pisosmadrid.com.es/propiedad/40866...,Piso en venta en Ibiza Madrid,520000,3,1,70,Precioso piso reformado de 60m2 enfrente del R...
1,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Madrid,329000,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...
2,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Arganzuela Madrid,740000,4,3,158,Se ofrece una magnifica oportunidad para adqui...


In [84]:
datos_pisosmadrid_detalles.head(3)

,descripcion,cant_dormitorios,cant_banos,m_cuadrados,descripcion2,tipo_lugar,ciudad,barrio/localidad/distrito,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado
0,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso ...,Piso,Madrid,Ibiza,0,0,0,PM4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0
1,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,Madrid,nan,0,1,1,PM4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0
2,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,Madrid,Arganzuela,0,0,0,PM4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0


In [85]:
datos_pisosmadrid_detalles= datos_pisosmadrid_detalles.rename(columns={"m_cuadrados":"metros_cuadrados_construidos"})
datos_pisosmadrid_detalles= datos_pisosmadrid_detalles.rename(columns={"descripcion2":"descripcion_extendida"})

In [86]:
datos_pisosmadrid_detalles["fecha_publicacion"]=pd.to_datetime(datos_pisosmadrid_detalles["fecha_publicacion"],format='%Y-%m-%d',errors="coerce")

In [87]:
datos_pisosmadrid_detalles["url_website"] = datos_pisosmadrid_normal["url_website"]

In [88]:
# Reemplazamiento de la palabra dormitorio 
datos_pisosmadrid_detalles['cant_dormitorios'] = datos_pisosmadrid_detalles['cant_dormitorios'] \
    .replace(r"\bdormitorio\b\s*", "", regex=True)
datos_pisosmadrid_normal['cant_dormitorios'] = datos_pisosmadrid_normal['cant_dormitorios'] \
    .replace(r"\bdormitorio\b\s*", "", regex=True)

In [89]:
datos_pisosmadrid_detalles.shape

(1188, 24)

In [90]:
col_conversion= ["piscina","garaje","jardin","n.visitas","cant_dormitorios",
                 "precio","cant_banos","metros_cuadrados_construidos" ]
# Reemplazar 'nan' y cadenas vacías '' por 0 en todas las columnas relevantes
datos_pisosmadrid_detalles[col_conversion] = datos_pisosmadrid_detalles[col_conversion].replace({'nan': '0', '': '0'})

# Convertir las columnas a tipo numérico (float primero para manejar correctamente los valores)
datos_pisosmadrid_detalles[col_conversion] = datos_pisosmadrid_detalles[col_conversion].apply(pd.to_numeric, errors='coerce')

# Convertir las columnas a enteros, llenando NaN con 0 antes de la conversión
datos_pisosmadrid_detalles[col_conversion] = datos_pisosmadrid_detalles[col_conversion].fillna(0).astype("int64")


In [91]:
datos_pisosmadrid_detalles.shape

(1188, 24)

In [92]:
datos_pisosmadrid_detalles.dtypes

descripcion                             object
cant_dormitorios                         int64
cant_banos                               int64
metros_cuadrados_construidos             int64
descripcion_extendida                   object
tipo_lugar                              object
ciudad                                  object
barrio/localidad/distrito               object
piscina                                  int64
garaje                                   int64
jardin                                   int64
referencia                              object
fecha_publicacion               datetime64[ns]
n.visitas                                int64
precio                                   int64
calefaccion                              int64
trastero                                 int64
amueblado                                int64
chimenea                                 int64
lavadero                                 int64
videoportero                             int64
alarma       

In [93]:
datos_pisosmadrid_detalles.isna().sum()

descripcion                     0
cant_dormitorios                0
cant_banos                      0
metros_cuadrados_construidos    0
descripcion_extendida           0
tipo_lugar                      0
ciudad                          0
barrio/localidad/distrito       0
piscina                         0
garaje                          0
jardin                          0
referencia                      0
fecha_publicacion               1
n.visitas                       0
precio                          0
calefaccion                     0
trastero                        0
amueblado                       0
chimenea                        0
lavadero                        0
videoportero                    0
alarma                          0
aire_acondicionado              0
url_website                     0
dtype: int64

In [94]:
datos_pisosmadrid_detalles["piscina"].unique()

array([0, 1], dtype=int64)

In [95]:
datos_pisosmadrid_detalles["garaje"].unique()

array([0, 1], dtype=int64)

In [96]:
datos_pisosmadrid_detalles["jardin"].unique()

array([0, 1], dtype=int64)

In [97]:
datos_pisosmadrid_detalles["n.visitas"].unique()

array([ 40,  20,  30,  60,  50,   0,  10,  70,  80, 430, 140,  90, 170,
       130, 110, 100, 120, 190, 150, 240, 200, 160, 180], dtype=int64)

In [98]:
datos_pisosmadrid_detalles["precio"].unique()

array([  520000,   329000,   740000,    70256,    97920,    69313,
          21999,  1400000,  1800000,   168000,   250000,    40000,
          20000,    18000,    18900,    19900,   380000,   235000,
        1354900,  1129000,   320000,  2099000,   960000,   315000,
         220000,   299000,  2350000,   515000,   870000,   625000,
         258000,   298500,   322900,   317000,   322000,  1500000,
         189900,   575000,   289000,   675000,   275000,   335000,
         270000,   370000,   285000,   475000,   100000,   347300,
         570000,  1300000,   249000,   460000,  1355000,    54000,
         539000,   600000,   889000,   135000,   843800,   970000,
         850000,  1770000,  1200000,   655000,   301000,  1150000,
         368700,   800000,   239000,   238000,   210000,  1550000,
        1450000,   500000,   240000,  2475000,   925000,  1140000,
        1330000,   650000,   190000,   173000,    43000,   266500,
         230000,   286700,   459000,   179990,   400000,   430

In [99]:
columnas_texto = datos_pisosmadrid_detalles.select_dtypes(include=['object']).columns

# Paso 2: Limpiar las columnas de texto aplicando .str.strip(), .lower(), etc.
for col in columnas_texto:
    # Reemplazar NaN por cadena vacía
    datos_pisosmadrid_detalles[col] = datos_pisosmadrid_detalles[col].fillna('')  # Reemplaza NaN por cadena vacía
    # Convertir a tipo str si es necesario
    datos_pisosmadrid_detalles[col] = datos_pisosmadrid_detalles[col].astype(str)
    # Limpiar espacios al principio y al final
    datos_pisosmadrid_detalles[col] = datos_pisosmadrid_detalles[col].str.strip()

In [100]:
columnas_comparar = ['descripcion', 'precio',
                     "cant_dormitorios","cant_banos","metros_cuadrados_construidos",
                     "descripcion_extendida"]

# Limitar ambos DataFrames a las primeras 1188 filas
datos_subset = datos_pisosmadrid_detalles[1:15]
datos2_subset = datos_pisosmadrid_normal[1:15]

# Comparar todas las columnas seleccionadas
coinciden = datos_subset[columnas_comparar] == (datos2_subset[columnas_comparar])

coinciden

,descripcion,precio,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida
1,True,False,False,False,False,False
2,True,False,False,False,False,False
3,True,False,False,False,False,False
4,True,False,False,False,False,False
5,True,False,False,False,False,False
6,True,False,False,False,False,False
7,True,False,False,False,False,False
8,True,False,False,False,False,False
9,True,False,False,False,False,False
10,True,False,False,False,False,False


In [101]:
datos_pisosmadrid_detalles["barrio/localidad/distrito"].value_counts()

barrio/localidad/distrito
nan                        153
Centro                      64
Malasana                    42
Salamanca                   39
Ensanche de Vallecas        31
                          ... 
Principe Pio                 1
Conde Orgaz                  1
Valverde                     1
Ventas                       1
Montecarmelo  El Goloso      1
Name: count, Length: 150, dtype: int64

In [102]:
datos_pisosmadrid_detalles["descripcion_extendida"].to_string()

'0       Precioso piso reformado de 60m2 enfrente del R...\n1       SERVICHECK INMOBILIARIA comercializa un precio...\n2       Se ofrece una magnifica oportunidad para adqui...\n3       Vivienda situada en Calle Godella en Madrid ca...\n4       Vivienda situada en Calle Transmonte en Madrid...\n5       Vivienda situada en Calle Callejo en Madrid ca...\n6       FAM INMOBILIARIA vende en exclusiva estupenda ...\n7       Gran Oportunidad en Malasana Vivienda amplia y...\n8       Que interesante propiedad El piso que describe...\n9       Descubre esta oportunidad unica para invertir ...\n10      inmobiliarias Encuentro pone a su disposicion ...\n11      inmobiliarias Encuentro pone a su disposicion ...\n12      inmobiliarias Encuentro pone a su disposicion ...\n13      Inmobiliarias Encuentro pone a su disposicion ...\n14      Inmobiliarias Encuentro pone a su disposicion ...\n15      Inmobiliarias Encuentro pone a su disposicion ...\n16      Inmobiliarias Encuentro VENDE este VIVIENDA CO.

In [103]:
datos_pisosmadrid_normal.isna().sum()

url_website                     0
descripcion                     0
precio                          0
cant_dormitorios                0
cant_banos                      0
metros_cuadrados_construidos    0
descripcion_extendida           0
dtype: int64

In [104]:
datos_pisosmadrid_detalles.isnull().sum()

descripcion                     0
cant_dormitorios                0
cant_banos                      0
metros_cuadrados_construidos    0
descripcion_extendida           0
tipo_lugar                      0
ciudad                          0
barrio/localidad/distrito       0
piscina                         0
garaje                          0
jardin                          0
referencia                      0
fecha_publicacion               1
n.visitas                       0
precio                          0
calefaccion                     0
trastero                        0
amueblado                       0
chimenea                        0
lavadero                        0
videoportero                    0
alarma                          0
aire_acondicionado              0
url_website                     0
dtype: int64

In [105]:
dbcp_ordenado

,cod_distrito,nombre_distrito,cod_barrio,cod_barrio_2,nombre_barrio,codigo_Postal
0,1,Centro,11,1,Palacio,28013
1,1,Centro,12,2,Embajadores,28012
2,1,Centro,13,3,Cortes,28014
3,1,Centro,14,4,Justicia,28004
4,1,Centro,15,5,Universidad,28015
...,...,...,...,...,...,...
126,20,San Blas-Canillejas,208,8,Salvador,28037
127,21,Barajas,211,1,Alameda de Osuna,28042
128,21,Barajas,212,2,Aeropuerto,28042
129,21,Barajas,213,3,Corralejos,28042


In [106]:
datos_pisosmadrid_detalles.head()

,descripcion,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida,tipo_lugar,ciudad,barrio/localidad/distrito,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,url_website
0,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso reformado de 60m2 enfrente del R...,Piso,Madrid,Ibiza,0,0,0,PM4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40866...
1,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,Madrid,nan,0,1,1,PM4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0,https://www.pisosmadrid.com.es/propiedad/40865...
2,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,Madrid,Arganzuela,0,0,0,PM4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0,https://www.pisosmadrid.com.es/propiedad/40865...
3,Piso en venta en Los Angeles Madrid,3,1,57,Vivienda situada en Calle Godella en Madrid ca...,Piso,Madrid,Los Angeles,0,0,0,PM4086495,2024-10-24,40,70256,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40864...
4,Piso en venta en Madrid,2,1,50,Vivienda situada en Calle Transmonte en Madrid...,Piso,Madrid,nan,0,0,0,PM4086493,2024-10-24,30,97920,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40864...


In [107]:
datos_pisosmadrid_detalles.shape

(1188, 24)

In [108]:
# Aqui guardamos el nombre de la columna objetivo en una variable
columna_a_clasificar= "barrio/localidad/distrito"

# Creamos la funcion que clasificara nombres en las columnas nuevas y tomara los valores de la columna target
def clasificar_barrio_distrito(datos_pisosmadrid_detalles, dbcp_ordenado,columna_a_clasificar):
    distritos= dbcp_ordenado["nombre_distrito"].tolist()
    barrios=   dbcp_ordenado["nombre_barrio"].tolist()

    # Creamos las nuevas columnas y las llenamos de nulos 
    datos_pisosmadrid_detalles["barrios"]   =np.nan 
    datos_pisosmadrid_detalles["distrito"]  =np.nan 
    datos_pisosmadrid_detalles["localidad"] =np.nan 

    for idx, valor in datos_pisosmadrid_detalles[columna_a_clasificar].items():
        if valor in distritos:
            datos_pisosmadrid_detalles.at[idx,"distrito" ]= valor
        elif valor in barrios:
            datos_pisosmadrid_detalles.at[idx,"barrios"   ]= valor 
        else:
            datos_pisosmadrid_detalles.at[idx,"localidad"]=valor
    datos_pisosmadrid_detalles[columna_a_clasificar]= np.nan

clasificar_barrio_distrito(datos_pisosmadrid_detalles = datos_pisosmadrid_detalles,dbcp_ordenado=dbcp_ordenado,columna_a_clasificar="barrio/localidad/distrito")


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\2805591847.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Ibiza' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  datos_pisosmadrid_detalles.at[idx,"barrios"   ]= valor
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\2805591847.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  datos_pisosmadrid_detalles.at[idx,"localidad"]=valor
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\2805591847.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Arganzuela' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  d

In [109]:
datos_pisosmadrid_detalles

,descripcion,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida,tipo_lugar,ciudad,barrio/localidad/distrito,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,url_website,barrios,distrito,localidad
0,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso reformado de 60m2 enfrente del R...,Piso,Madrid,NaN,0,0,0,PM4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40866...,Ibiza,NaN,NaN
1,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,Madrid,NaN,0,1,1,PM4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0,https://www.pisosmadrid.com.es/propiedad/40865...,NaN,NaN,nan
2,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,Madrid,NaN,0,0,0,PM4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0,https://www.pisosmadrid.com.es/propiedad/40865...,NaN,Arganzuela,NaN
3,Piso en venta en Los Angeles Madrid,3,1,57,Vivienda situada en Calle Godella en Madrid ca...,Piso,Madrid,NaN,0,0,0,PM4086495,2024-10-24,40,70256,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40864...,NaN,NaN,Los Angeles
4,Piso en venta en Madrid,2,1,50,Vivienda situada en Calle Transmonte en Madrid...,Piso,Madrid,NaN,0,0,0,PM4086493,2024-10-24,30,97920,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40864...,NaN,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,Garaje en venta en Madrid Madrid,0,0,9,JM Plaza de Garaje de 9 m2 situada en la plan...,Garaje,Madrid,NaN,0,0,0,PM3865049,2024-05-18,140,60000,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/38671...,NaN,NaN,Madrid
1184,Local en venta en Ensanche de Vallecas Madrid,0,2,89,SERVICHECK INMOBILIARIA Vende bonito local es ...,Local,Madrid,NaN,0,0,0,PM3863985,2024-05-17,70,280000,1,0,0,0,0,0,1,1,https://www.pisosmadrid.com.es/propiedad/38668...,Ensanche de Vallecas,NaN,NaN
1185,Piso en venta en Madrid,3,1,54,PISO EN MADRID Vivienda situada en Avenida Pal...,Piso,Madrid,NaN,0,0,0,PM3863841,2024-05-17,120,60000,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/38666...,NaN,NaN,nan
1186,Garaje en venta en Castellana Madrid,0,0,25,GARAJE EN MADRID Garaje situado en la Paseo de...,Garaje,Madrid,NaN,0,0,0,PM3863823,2024-05-17,150,34125,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/38666...,Castellana,NaN,NaN


In [110]:
datos_pisosmadrid_detalles["distrito"].isna().sum()

976

In [111]:
datos_pisosmadrid_detalles["barrios"].isna().sum()

725

In [112]:
# Esta funcion rellena los distritos conociendo los barrios


def rellenar_distritos(datos_pisosmadrid_detalles,dbcp_ordenado):

    barrio_a_distrito=dict(zip(dbcp_ordenado["nombre_barrio"],dbcp_ordenado["nombre_distrito"]))

    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        if pd.isna(fila["distrito"])and not pd.isna(fila["barrios"]):
            distrito = barrio_a_distrito.get(fila["barrios"])
            if distrito:
                datos_pisosmadrid_detalles.at[idx,"distrito"] = distrito 


rellenar_distritos(datos_pisosmadrid_detalles=datos_pisosmadrid_detalles,dbcp_ordenado=dbcp_ordenado)

In [113]:
# Aqui filtramos barrio y distritos por una lista de columnas y por el excel donde estan guardados una cierta lista de barrios y distritos.
def clasificar_distrito_y_barrio_por_columnas(datos_pisosmadrid_detalles, dbcp_ordenado):
    # Crear las listas de barrios y distritos desde el dataframe de referencia
    distritos = dbcp_ordenado["nombre_distrito"].tolist()
    barrios = dbcp_ordenado["nombre_barrio"].tolist()

    # Recorrer las filas del dataframe
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        # Recorrer las tres columnas en orden de prioridad: descripcion, descripcion_extendida, url_website
        for columna in ['descripcion', 'descripcion_extendida', 'url_website']:
            valor = fila[columna]
            
            # Verificamos que el valor no sea nulo
            if pd.notna(valor):
                # Buscamos si el nombre del distrito aparece en la columna
                for distrito in distritos:
                    if distrito in valor and pd.isna(fila["distrito"]):
                        datos_pisosmadrid_detalles.at[idx, "distrito"] = distrito  # Asignar el nombre del distrito a la columna "distrito"

                # Buscamos si el nombre del barrio aparece en la columna
                for barrio in barrios:
                    if barrio in valor and pd.isna(fila["barrios"]):
                        datos_pisosmadrid_detalles.at[idx, "barrios"] = barrio  # Asignar el nombre del barrio a la columna "barrio"

    return datos_pisosmadrid_detalles

# Llamar a la función pasándole el dataframe y el dataframe de referencia
datos_pisosmadrid_detalles = clasificar_distrito_y_barrio_por_columnas(datos_pisosmadrid_detalles, dbcp_ordenado)


In [114]:
# Esta funcion rellena los distritos conociendo los barrios

def rellenar_distritos(datos_pisosmadrid_detalles,dbcp_ordenado):

    barrio_a_distrito=dict(zip(dbcp_ordenado["nombre_barrio"],dbcp_ordenado["nombre_distrito"]))

    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        if pd.isna(fila["distrito"])and not pd.isna(fila["barrios"]):
            distrito = barrio_a_distrito.get(fila["barrios"])
            if distrito:
                datos_pisosmadrid_detalles.at[idx,"distrito"] = distrito 


rellenar_distritos(datos_pisosmadrid_detalles=datos_pisosmadrid_detalles,dbcp_ordenado=dbcp_ordenado)

In [115]:
# Aqui normalizamos las columnas y despues hacemos la filtracion para captar mas nombres de distritos y barrios. 

def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

def clasificar_distrito_y_barrio_por_columnas(datos_pisosmadrid_detalles, dbcp_ordenado):
    # Crear las listas de barrios y distritos desde el dataframe de referencia
    distritos = dbcp_ordenado["nombre_distrito"].apply(lambda x: normalize_string(x)).tolist()
    barrios = dbcp_ordenado["nombre_barrio"].apply(lambda x: normalize_string(x)).tolist()

    # Recorrer las filas del dataframe
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        # Recorrer las tres columnas en orden de prioridad: descripcion, descripcion_extendida, url_website
        for columna in ['descripcion', 'descripcion_extendida', 'url_website',"localidad"]:
            valor = fila[columna]
            
            # Verificamos que el valor no sea nulo
            if pd.notna(valor):
                valor_normalizado = normalize_string(valor)
                
                # Verificamos si el valor normalizado contiene el nombre de un distrito o barrio
                for distrito, barrio in zip(distritos, barrios):
                    if distrito in valor_normalizado and pd.isna(fila["distrito"]):
                        datos_pisosmadrid_detalles.at[idx, "distrito"] = distrito  # Asignar el nombre del distrito a la columna "distrito"
                    if barrio in valor_normalizado and pd.isna(fila["barrios"]):
                        datos_pisosmadrid_detalles.at[idx, "barrios"] = barrio  # Asignar el nombre del barrio a la columna "barrio"

    return datos_pisosmadrid_detalles

# Llamar a la función pasándole el dataframe y el dataframe de referencia
datos_pisosmadrid_detalles = clasificar_distrito_y_barrio_por_columnas(datos_pisosmadrid_detalles, dbcp_ordenado)

In [116]:
# Esta funcion rellena los distritos conociendo los barrios al igual que antes pero ahora normalizamos los valores antes de rellenar para
# encontrar mas valores

def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

def rellenar_distritos(datos_pisosmadrid_detalles, dbcp_ordenado):
    # Crear un diccionario de barrios a distritos normalizado
    barrio_a_distrito = {normalize_string(barrio): distrito for barrio, distrito in zip(dbcp_ordenado["nombre_barrio"], dbcp_ordenado["nombre_distrito"])}

    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        # Verificamos si el valor de distrito está vacío y si hay un valor en barrios
        if pd.isna(fila["distrito"]) and not pd.isna(fila["barrios"]):
            barrio_normalizado = normalize_string(fila["barrios"])  # Normalizar el valor del barrio
            distrito = barrio_a_distrito.get(barrio_normalizado)  # Buscar el distrito usando el barrio normalizado
            if distrito:
                datos_pisosmadrid_detalles.at[idx, "distrito"] = distrito  # Asignar el distrito correspondiente

# Llamar a la función pasándole el dataframe y el dataframe de referencia
rellenar_distritos(datos_pisosmadrid_detalles=datos_pisosmadrid_detalles, dbcp_ordenado=dbcp_ordenado)

In [117]:
# Aca vamos a hacer la filtracion desde la columna localidad y el dataframe de referencia  dbcp_ordenado donde hay valores de distritos y barrios

# Función para normalizar las cadenas de texto
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

def clasificar_en_localidad(datos_pisosmadrid_detalles, dbcp_ordenado):
    # Crear las listas de barrios y distritos desde el dataframe de referencia
    distritos = dbcp_ordenado["nombre_distrito"].apply(lambda x: normalize_string(x)).tolist()
    barrios = dbcp_ordenado["nombre_barrio"].apply(lambda x: normalize_string(x)).tolist()

    # Recorrer las filas del dataframe
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        # Verificar que la columna 'localidad' no sea nula
        if pd.notna(fila["localidad"]):
            valor_localidad = normalize_string(fila["localidad"])

            # Buscar si el valor de la localidad coincide con algún distrito
            for distrito in distritos:
                if distrito in valor_localidad and pd.isna(fila["distrito"]):
                    datos_pisosmadrid_detalles.at[idx, "distrito"] = distrito  # Asignar el nombre del distrito a la columna "distrito"

            # Buscar si el valor de la localidad coincide con algún barrio
            for barrio in barrios:
                if barrio in valor_localidad and pd.isna(fila["barrios"]):
                    datos_pisosmadrid_detalles.at[idx, "barrios"] = barrio  # Asignar el nombre del barrio a la columna "barrio"
            
            # Convertir la columna 'localidad' en NaN después de asignar el barrio o distrito
            datos_pisosmadrid_detalles.at[idx, "localidad"] = pd.NA

    return datos_pisosmadrid_detalles

# Llamar a la función pasándole el dataframe
datos_pisosmadrid_detalles = clasificar_en_localidad(datos_pisosmadrid_detalles, dbcp_ordenado)

In [118]:
datos_pisosmadrid_detalles["barrios"].isna().sum()

147

In [119]:
datos_pisosmadrid_detalles["distrito"].isna().sum()

31

In [120]:
# Aca seguimos escarbando mas y aruñando  para conseguir mas barrios y distritos

# Filtrar las filas donde no hay valor en barrio, pero sí en distrito y localidad
filtro_1 = datos_pisosmadrid_detalles[pd.isna(datos_pisosmadrid_detalles["barrios"]) & pd.notna(datos_pisosmadrid_detalles["distrito"]) & pd.notna(datos_pisosmadrid_detalles["localidad"])]

# Filtrar las filas donde no hay valor en distrito, pero sí en barrio y localidad
filtro_2 = datos_pisosmadrid_detalles[pd.isna(datos_pisosmadrid_detalles["distrito"]) & pd.notna(datos_pisosmadrid_detalles["barrios"]) & pd.notna(datos_pisosmadrid_detalles["localidad"])]


In [121]:
datos_pisosmadrid_detalles["distrito"].value_counts()

distrito
Centro                  186
Salamanca               158
Carabanchel              81
Villaverde               63
Arganzuela               61
centro                   61
Ciudad Lineal            48
Retiro                   47
Puente de Vallecas       42
 Villa de Vallecas       40
Chamberí                 40
Latina                   40
Chamartín                34
Tetuán                   32
 San Blas-Canillejas     31
Usera                    30
Moncloa-Aravaca          21
chamartin                20
tetuan                   20
Hortaleza                19
 Barajas                 18
chamberi                 14
Fuencarral-El Pardo      13
Moratalaz                 8
 vicalvaro                6
salamanca                 6
retiro                    3
latina                    3
 villa de vallecas        2
villaverde                2
 Vicálvaro                2
carabanchel               2
usera                     2
arganzuela                1
puente de vallecas        1
Name: count

In [122]:
filtro_1["localidad"].value_counts()

Series([], Name: count, dtype: int64)

In [123]:
datos_pisosmadrid_detalles["localidad"].value_counts()

Series([], Name: count, dtype: int64)

In [124]:
datos_pisosmadrid_detalles.head()

,descripcion,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida,tipo_lugar,ciudad,barrio/localidad/distrito,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,url_website,barrios,distrito,localidad
0,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso reformado de 60m2 enfrente del R...,Piso,Madrid,NaN,0,0,0,PM4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40866...,Ibiza,Retiro,NaN
1,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,Madrid,NaN,0,1,1,PM4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0,https://www.pisosmadrid.com.es/propiedad/40865...,Ensanche de Vallecas,Centro,<NA>
2,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,Madrid,NaN,0,0,0,PM4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0,https://www.pisosmadrid.com.es/propiedad/40865...,Delicias,Arganzuela,NaN
3,Piso en venta en Los Angeles Madrid,3,1,57,Vivienda situada en Calle Godella en Madrid ca...,Piso,Madrid,NaN,0,0,0,PM4086495,2024-10-24,40,70256,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40864...,los angeles,Villaverde,<NA>
4,Piso en venta en Madrid,2,1,50,Vivienda situada en Calle Transmonte en Madrid...,Piso,Madrid,NaN,0,0,0,PM4086493,2024-10-24,30,97920,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40864...,NaN,NaN,<NA>


In [125]:
# Ahora vamos a filtrar por un diccionario que creamos abajo y tambien la columna localidad


def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

# Función que recorre la columna localidad y asigna los barrios correspondientes
def asignar_barrios_a_localidad(datos_pisosmadrid_detalles, diccionario_barrios):
    # Normalizar los barrios en el diccionario
    diccionario_normalizado = {
        normalize_string(barrio): distrito for distrito, barrios in diccionario_barrios.items() for barrio in barrios
    }
    
    # Recorrer el dataframe
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        # Solo proceder si la columna localidad no es nula
        localidad = fila['localidad']
        if pd.notna(localidad):
            localidad_normalizada = normalize_string(localidad)

            # Comprobar si el valor normalizado de localidad contiene algún barrio del diccionario
            for barrio_normalizado, distrito in diccionario_normalizado.items():
                if barrio_normalizado in localidad_normalizada and pd.isna(fila['barrios']):
                    # Asignar el barrio en la columna "barrios"
                    datos_pisosmadrid_detalles.at[idx, 'barrios'] = barrio_normalizado

    return datos_pisosmadrid_detalles

# Diccionario de ejemplo con distritos y barrios (ya normalizados)
diccionario_barrios = {
    "Centro": ["Malasaña", "La Latina", "Chueca","malasana"],
    "Moratalaz": ["La Elipa"],
    "Moncloa-Aravaca": ["Puerta de Hierro", "Príncipe Pío"],
    "Fuencarral-El Pardo": ["Buena Vista", "Ventilla", "Fuencarral"],
    "Chamartín": ["Bernabéu"],
    "Ciudad Lineal": ["Arturo Soria"],
    "San Blas-Canillejas": ["Las Mercedes","las mercedes"],
    "Villaverde": ["Villaverde Alto", "Villaverde Bajo"],
    "Carabanchel": ["Pau Carabanchel"],
}

# Llamar a la función pasándole el dataframe y el diccionario de barrios
datos_pisosmadrid_detalles = asignar_barrios_a_localidad(datos_pisosmadrid_detalles, diccionario_barrios)

**_La funcion de arriba no genero cambio  pero ahora se que no tengo que hacerla mas adelante, asi que no la eliminare..... dejo este nota aqui_**

In [126]:
datos_pisosmadrid_detalles.head(4)

,descripcion,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida,tipo_lugar,ciudad,barrio/localidad/distrito,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,url_website,barrios,distrito,localidad
0,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso reformado de 60m2 enfrente del R...,Piso,Madrid,NaN,0,0,0,PM4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40866...,Ibiza,Retiro,NaN
1,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,Madrid,NaN,0,1,1,PM4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0,https://www.pisosmadrid.com.es/propiedad/40865...,Ensanche de Vallecas,Centro,<NA>
2,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,Madrid,NaN,0,0,0,PM4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0,https://www.pisosmadrid.com.es/propiedad/40865...,Delicias,Arganzuela,NaN
3,Piso en venta en Los Angeles Madrid,3,1,57,Vivienda situada en Calle Godella en Madrid ca...,Piso,Madrid,NaN,0,0,0,PM4086495,2024-10-24,40,70256,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40864...,los angeles,Villaverde,<NA>


In [127]:
datos_pisosmadrid_detalles["distrito"].isna().sum()

31

In [128]:
datos_pisosmadrid_detalles["barrios"].isna().sum()

147

In [129]:

# Aca vamos a hacer filtraciones como antes pero con un par de condicionantes mas... Escarbando tanto como den las ideas.

# Función para normalizar cadenas de texto
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

# Función que recorre la columna localidad y asigna los barrios correspondientes
def asignar_barrios_y_distritos(datos_pisosmadrid_detalles, diccionario_barrios):
    # Normalizar los barrios y distritos en el diccionario
    diccionario_normalizado = {
        normalize_string(barrio): distrito for distrito, barrios in diccionario_barrios.items() for barrio in barrios
    }
    
    # Recorrer el dataframe
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        localidad = fila['localidad']
        
        # Comprobar si existe un valor en localidad y no hay valores en barrios ni distrito
        if pd.notna(localidad) and pd.isna(fila['barrios']) and pd.isna(fila['distrito']):
            localidad_normalizada = normalize_string(localidad)
            
            # Comprobar si el valor normalizado de localidad contiene algún barrio del diccionario
            for barrio_normalizado, distrito in diccionario_normalizado.items():
                if barrio_normalizado in localidad_normalizada:
                    datos_pisosmadrid_detalles.at[idx, 'barrios'] = barrio_normalizado  # Asignar barrio a la columna "barrios"
                    datos_pisosmadrid_detalles.at[idx, 'distrito'] = distrito  # Asignar distrito correspondiente

            # Si no se encuentra un barrio, intentar asignar un distrito (buscando el nombre del distrito)
            if pd.isna(fila['barrios']):
                for distrito in diccionario_barrios.keys():
                    if normalize_string(distrito) in localidad_normalizada:
                        datos_pisosmadrid_detalles.at[idx, 'distrito'] = distrito  # Asignar el distrito si no se encontró barrio
                        
    return datos_pisosmadrid_detalles

# Diccionario de ejemplo con distritos y barrios (ya normalizados)
diccionario_barrios = {
    "Centro": ["Malasaña", "La Latina", "Chueca","malasana"],
    "Moratalaz": ["La Elipa"],
    "Moncloa-Aravaca": ["Puerta de Hierro", "Príncipe Pío"],
    "Fuencarral-El Pardo": ["Buena Vista", "Ventilla", "Fuencarral"],
    "Chamartín": ["Bernabéu"],
    "Ciudad Lineal": ["Arturo Soria"],
    "San Blas-Canillejas": ["Las Mercedes","las mercedes"],
    "Villaverde": ["Villaverde Alto", "Villaverde Bajo"],
    "Carabanchel": ["Pau Carabanchel"],
    
}

# Llamar a la función pasándole el dataframe y el diccionario de barrios
datos_pisosmadrid_detalles = asignar_barrios_y_distritos(datos_pisosmadrid_detalles, diccionario_barrios)

In [130]:
datos_pisosmadrid_detalles["distrito"].isna().sum()

31

In [131]:
datos_pisosmadrid_detalles["barrios"].isna().sum()

147

In [132]:
datos_pisosmadrid_detalles=datos_pisosmadrid_detalles.dropna(subset=["precio"])

In [133]:
datos_pisosmadrid_detalles["precio"].isna().sum()

0

In [134]:
datos_pisosmadrid_detalles.shape

(1188, 27)

In [135]:
barrio_distrito_info= {
    "Centro": ["Malasaña", "La Latina", "Chueca","malasana"],
    "Moratalaz": ["La Elipa"],
    "Moncloa-Aravaca": ["Puerta de Hierro", "Príncipe Pío"],
    "Fuencarral-El Pardo": ["Buena Vista", "Ventilla", "Fuencarral"],
    "Chamartín": ["Bernabéu"],
    "Ciudad Lineal": ["Arturo Soria"],
    "San Blas-Canillejas": ["Las Mercedes","las mercedes"],
    "Villaverde": ["Villaverde Alto", "Villaverde Bajo"],
    "Carabanchel": ["Pau Carabanchel"],
    
}
def normalize_string(s):
    """Normaliza las cadenas para ignorar mayúsculas, minúsculas, acentos y caracteres especiales."""
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

def corregir_distrito_por_barrio(datos_pisosmadrid_detalles, dbcp_ordenado, barrio_distrito_info):
    # Recorrer las filas del dataframe original
    filas_sin_coincidencias = []
    
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        barrio = fila["barrios"]
        distrito_actual = fila["distrito"]
        
        if pd.notna(barrio):
            barrio_normalizado = normalize_string(barrio)
            
            # Buscar la coincidencia en el dataframe de referencia
            distrito_en_ref = dbcp_ordenado[dbcp_ordenado["nombre_barrio"].apply(lambda x: normalize_string(x) == barrio_normalizado)]
            
            if not distrito_en_ref.empty:
                # Si hay coincidencia en el dataframe de referencia
                distrito_nuevo = distrito_en_ref.iloc[0]["nombre_distrito"]
                
                # Asignar el nuevo distrito si es diferente al actual
                if pd.isna(distrito_actual) or distrito_actual != distrito_nuevo:
                    datos_pisosmadrid_detalles.at[idx, "distrito"] = distrito_nuevo
            else:
                # Si no se encuentra en el dataframe de referencia, buscar en el diccionario
                for (distrito, barrio_ref), (cod_distrito, cod_barrio, cod_barrio2, codigo_postal) in barrio_distrito_info.items():
                    if normalize_string(barrio_ref) == barrio_normalizado:
                        # Asignar el distrito desde el diccionario
                        if pd.isna(distrito_actual) or distrito_actual != distrito:
                            datos_pisosmadrid_detalles.at[idx, "distrito"] = distrito
                        break
                else:
                    # Si no se encuentra en el dataframe de referencia ni en el diccionario, mostrar la fila
                    filas_sin_coincidencias.append(fila)
    
    return datos_pisosmadrid_detalles, pd.DataFrame(filas_sin_coincidencias)

# Llamar a la función
datos_actualizados, filas_sin_coincidencias = corregir_distrito_por_barrio(datos_pisosmadrid_detalles, dbcp_ordenado, barrio_distrito_info)

# Mostrar las filas que no tienen coincidencias
filas_sin_coincidencias

""


In [136]:
datos_pisosmadrid_detalles.head()

,descripcion,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida,tipo_lugar,ciudad,barrio/localidad/distrito,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,url_website,barrios,distrito,localidad
0,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso reformado de 60m2 enfrente del R...,Piso,Madrid,NaN,0,0,0,PM4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40866...,Ibiza,Retiro,NaN
1,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,Madrid,NaN,0,1,1,PM4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0,https://www.pisosmadrid.com.es/propiedad/40865...,Ensanche de Vallecas,Villa de Vallecas,<NA>
2,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,Madrid,NaN,0,0,0,PM4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0,https://www.pisosmadrid.com.es/propiedad/40865...,Delicias,Arganzuela,NaN
3,Piso en venta en Los Angeles Madrid,3,1,57,Vivienda situada en Calle Godella en Madrid ca...,Piso,Madrid,NaN,0,0,0,PM4086495,2024-10-24,40,70256,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40864...,los angeles,Villaverde,<NA>
4,Piso en venta en Madrid,2,1,50,Vivienda situada en Calle Transmonte en Madrid...,Piso,Madrid,NaN,0,0,0,PM4086493,2024-10-24,30,97920,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40864...,NaN,NaN,<NA>


In [137]:
# Aqui eliminamos dos columnas con informacion sin utilidad
columnas_baja= ["ciudad","barrio/localidad/distrito",]
datos_pisosmadrid_detalles = datos_pisosmadrid_detalles.drop(columns=columnas_baja)

In [138]:
barrio_distrito_info= {
    "Centro": ["Malasaña", "La Latina", "Chueca","malasana"],
    "Moratalaz": ["La Elipa"],
    "Moncloa-Aravaca": ["Puerta de Hierro", "Príncipe Pío"],
    "Fuencarral-El Pardo": ["Buena Vista", "Ventilla", "Fuencarral"],
    "Chamartín": ["Bernabéu"],
    "Ciudad Lineal": ["Arturo Soria"],
    "San Blas-Canillejas": ["Las Mercedes","las mercedes"],
    "Villaverde": ["Villaverde Alto", "Villaverde Bajo"],
    "Carabanchel": ["Pau Carabanchel"],
    
}

In [139]:
# Aqui agrego los datos de cod_ditrito al dataframe original.Normalizando


def normalize_string(s):
    #Normaliza las cadenas para ignorar mayúsculas, minúsculas, acentos y caracteres especiales.
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

    # Asigna el valor de cod_distrito en el dataframe original basado en el dataframe de referencia o sea el dbcp_ordenado
def asignar_cod_distrito(datos_pisosmadrid_detalles, dbcp_ordenado):

    # Asegurarse de que la columna cod_distrito exista en el dataframe original
    if "cod_distrito" not in datos_pisosmadrid_detalles.columns:
        datos_pisosmadrid_detalles["cod_distrito"] = pd.NA

    # Normalizar la columna nombre_distrito del dataframe de referencia
    dbcp_ordenado["nombre_distrito_normalizado"] = dbcp_ordenado["nombre_distrito"].apply(normalize_string)

    # Inicializar una lista para las filas sin coincidencias
    filas_sin_coincidencia = []

    # Recorrer el dataframe original
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        distrito = fila["distrito"]

        if pd.notna(distrito):
            distrito_normalizado = normalize_string(distrito)

            # Buscar coincidencia en el dataframe de referencia
            coincidencia = dbcp_ordenado[dbcp_ordenado["nombre_distrito_normalizado"] == distrito_normalizado]

            if not coincidencia.empty:
                # Si hay coincidencia, asignar el valor de cod_distrito
                datos_pisosmadrid_detalles.at[idx, "cod_distrito"] = coincidencia.iloc[0]["cod_distrito"]
            else:
                # Si no hay coincidencia, agregar la fila a las filas sin coincidencias
                filas_sin_coincidencia.append(fila)

    # Crear un DataFrame con las filas sin coincidencias
    distrito_sin_coincidencia = pd.DataFrame(filas_sin_coincidencia)

    return datos_pisosmadrid_detalles, distrito_sin_coincidencia

# Ejecución de la función
datos_actualizados, distrito_sin_coincidencia = asignar_cod_distrito(datos_pisosmadrid_detalles, dbcp_ordenado)


In [140]:
filas_sin_coincidencias

""


In [141]:
# Aca le asigno los valores a las columnas cod_barrio y cod_barrio_2 usando el dataframe de referencia 

def asignar_cod_barrios(datos_pisosmadrid_detalles, dbcp_ordenado):

    # Asegurarse de que las columnas cod_barrio y cod_barrio_2 existan en el dataframe original
    if "cod_barrio" not in datos_pisosmadrid_detalles.columns:
        datos_pisosmadrid_detalles["cod_barrio"] = pd.NA
    if "cod_barrio_2" not in datos_pisosmadrid_detalles.columns:
        datos_pisosmadrid_detalles["cod_barrio_2"] = pd.NA

    # Normalizar la columna nombre_barrio del dataframe de referencia
    dbcp_ordenado["nombre_barrio_normalizado"] = dbcp_ordenado["nombre_barrio"].apply(normalize_string)

    # Inicializar una lista para las filas sin coincidencias
    filas_sin_coincidencia = []

    # Recorrer el dataframe original
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        barrio = fila["barrios"]

        if pd.notna(barrio):
            barrio_normalizado = normalize_string(barrio)

            # Buscar coincidencia en el dataframe de referencia
            coincidencia = dbcp_ordenado[dbcp_ordenado["nombre_barrio_normalizado"] == barrio_normalizado]

            if not coincidencia.empty:
                # Si hay coincidencia, asignar los valores de cod_barrio y cod_barrio_2
                datos_pisosmadrid_detalles.at[idx, "cod_barrio"] = coincidencia.iloc[0]["cod_barrio"]
                datos_pisosmadrid_detalles.at[idx, "cod_barrio_2"] = coincidencia.iloc[0]["cod_barrio_2"]
            else:
                # Si no hay coincidencia, agregar la fila a las filas sin coincidencias
                filas_sin_coincidencia.append(fila)

    # Crear un DataFrame con las filas sin coincidencias
    barrio_sin_coincidencia = pd.DataFrame(filas_sin_coincidencia)

    return datos_pisosmadrid_detalles, barrio_sin_coincidencia

# Ejecución de la función
datos_actualizados, barrio_sin_coincidencia = asignar_cod_barrios(datos_pisosmadrid_detalles, dbcp_ordenado)

In [142]:
# Aca voy a asignar los codigos postales tomando en cuenta las columnas barrio y nombre_barrio

def asignar_cod_barrios(datos_pisosmadrid_detalles, dbcp_ordenado):
    
    # Asegurarse de que la columna codigo_Postal exista en el dataframe original
    if "codigo_Postal" not in datos_pisosmadrid_detalles.columns:
        datos_pisosmadrid_detalles["codigo_Postal"] = pd.NA

    # Normalizar la columna barrios del dataframe original
    datos_pisosmadrid_detalles["barrios_normalizado"] = datos_pisosmadrid_detalles["barrios"].apply(normalize_string)

    # Normalizar la columna nombre_barrio del dataframe de referencia
    dbcp_ordenado["nombre_barrio_normalizado"] = dbcp_ordenado["nombre_barrio"].apply(normalize_string)

    # Inicializar una lista para las filas sin coincidencias
    filas_sin_coincidencia = []

    # Recorrer el dataframe original
    for idx, fila in datos_pisosmadrid_detalles.iterrows():
        barrio_normalizado = fila["barrios_normalizado"]

        if pd.notna(barrio_normalizado):
            # Buscar coincidencia en el dataframe de referencia usando la columna normalizada
            coincidencia = dbcp_ordenado[dbcp_ordenado["nombre_barrio_normalizado"] == barrio_normalizado]

            if not coincidencia.empty:
                # Si hay coincidencia, asignar el valor de `codigo_Postal`
                datos_pisosmadrid_detalles.at[idx, "codigo_Postal"] = coincidencia.iloc[0]["codigo_Postal"]
            else:
                # Si no hay coincidencia, agregar la fila a las filas sin coincidencias
                filas_sin_coincidencia.append(fila)

    # Crear un DataFrame con las filas sin coincidencias
    barrio_sin_coincidencia = pd.DataFrame(filas_sin_coincidencia)

    return datos_pisosmadrid_detalles, barrio_sin_coincidencia

# Ejecución de la función
datos_actualizados, barrio_sin_coincidencia = asignar_cod_barrios(datos_pisosmadrid_detalles, dbcp_ordenado)

In [143]:

barrio_sin_coincidencia

""


In [144]:
datos_pisosmadrid_detalles["cod_barrio_2"].isna().sum()

147

In [145]:
datos_pisosmadrid_detalles["barrios"].isna().sum()

147

In [146]:
datos_pisosmadrid_detalles["localidad"].value_counts()

Series([], Name: count, dtype: int64)

In [147]:
datos_pisosmadrid_detalles = datos_pisosmadrid_detalles.drop(columns=["barrios_normalizado","localidad"])

In [148]:
datos_pisosmadrid_detalles

,descripcion,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida,tipo_lugar,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,url_website,barrios,distrito,cod_distrito,cod_barrio,cod_barrio_2,codigo_Postal
0,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso reformado de 60m2 enfrente del R...,Piso,0,0,0,PM4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40866...,Ibiza,Retiro,3,34,4,28009
1,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,0,1,1,PM4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0,https://www.pisosmadrid.com.es/propiedad/40865...,Ensanche de Vallecas,Villa de Vallecas,18,183,3,28031
2,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,0,0,0,PM4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0,https://www.pisosmadrid.com.es/propiedad/40865...,Delicias,Arganzuela,2,25,5,28045
3,Piso en venta en Los Angeles Madrid,3,1,57,Vivienda situada en Calle Godella en Madrid ca...,Piso,0,0,0,PM4086495,2024-10-24,40,70256,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40864...,los angeles,Villaverde,17,174,4,28021
4,Piso en venta en Madrid,2,1,50,Vivienda situada en Calle Transmonte en Madrid...,Piso,0,0,0,PM4086493,2024-10-24,30,97920,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/40864...,NaN,NaN,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,Garaje en venta en Madrid Madrid,0,0,9,JM Plaza de Garaje de 9 m2 situada en la plan...,Garaje,0,0,0,PM3865049,2024-05-18,140,60000,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/38671...,entrevias,Puente de Vallecas,13,131,1,28053
1184,Local en venta en Ensanche de Vallecas Madrid,0,2,89,SERVICHECK INMOBILIARIA Vende bonito local es ...,Local,0,0,0,PM3863985,2024-05-17,70,280000,1,0,0,0,0,0,1,1,https://www.pisosmadrid.com.es/propiedad/38668...,Ensanche de Vallecas,Villa de Vallecas,18,183,3,28031
1185,Piso en venta en Madrid,3,1,54,PISO EN MADRID Vivienda situada en Avenida Pal...,Piso,0,0,0,PM3863841,2024-05-17,120,60000,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/38666...,NaN,salamanca,4,<NA>,<NA>,<NA>
1186,Garaje en venta en Castellana Madrid,0,0,25,GARAJE EN MADRID Garaje situado en la Paseo de...,Garaje,0,0,0,PM3863823,2024-05-17,150,34125,0,0,0,0,0,0,0,0,https://www.pisosmadrid.com.es/propiedad/38666...,Castellana,Salamanca,4,46,6,28006


In [149]:
datos_pisosmadrid_detalles.shape

(1188, 28)

In [150]:
datos_pisosmadrid_detalles.duplicated().sum()

0

In [151]:
datos_pisosmadrid_normal.head()

,url_website,descripcion,precio,cant_dormitorios,cant_banos,metros_cuadrados_construidos,descripcion_extendida
0,https://www.pisosmadrid.com.es/propiedad/40866...,Piso en venta en Ibiza Madrid,520000,3,1,70,Precioso piso reformado de 60m2 enfrente del R...
1,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Madrid,329000,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...
2,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Arganzuela Madrid,740000,4,3,158,Se ofrece una magnifica oportunidad para adqui...
3,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en Los Angeles Madrid,70256,3,1,57,Vivienda situada en Calle Godella en Madrid ca...
4,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en Madrid,97920,2,1,50,Vivienda situada en Calle Transmonte en Madrid...


In [152]:
datos_pisosmadrid_detalles.isna().sum()

descripcion                       0
cant_dormitorios                  0
cant_banos                        0
metros_cuadrados_construidos      0
descripcion_extendida             0
tipo_lugar                        0
piscina                           0
garaje                            0
jardin                            0
referencia                        0
fecha_publicacion                 1
n.visitas                         0
precio                            0
calefaccion                       0
trastero                          0
amueblado                         0
chimenea                          0
lavadero                          0
videoportero                      0
alarma                            0
aire_acondicionado                0
url_website                       0
barrios                         147
distrito                         31
cod_distrito                     31
cod_barrio                      147
cod_barrio_2                    147
codigo_Postal               

In [153]:
datos_pisosmadrid_normal.isna().sum()

url_website                     0
descripcion                     0
precio                          0
cant_dormitorios                0
cant_banos                      0
metros_cuadrados_construidos    0
descripcion_extendida           0
dtype: int64

In [154]:
datos_pisosmadrid_detalles.dtypes

descripcion                             object
cant_dormitorios                         int64
cant_banos                               int64
metros_cuadrados_construidos             int64
descripcion_extendida                   object
tipo_lugar                              object
piscina                                  int64
garaje                                   int64
jardin                                   int64
referencia                              object
fecha_publicacion               datetime64[ns]
n.visitas                                int64
precio                                   int64
calefaccion                              int64
trastero                                 int64
amueblado                                int64
chimenea                                 int64
lavadero                                 int64
videoportero                             int64
alarma                                   int64
aire_acondicionado                       int64
url_website  

In [155]:
datos_pisosmadrid_normal.dtypes

url_website                     object
descripcion                     object
precio                          object
cant_dormitorios                object
cant_banos                      object
metros_cuadrados_construidos    object
descripcion_extendida           object
dtype: object

In [156]:
# Columnas clave para el merge
columnas_clave = [
    "url_website"
]

# Merge de los DataFrames
pisos_madrid_unido= pd.merge(datos_pisosmadrid_normal,datos_pisosmadrid_detalles, on=columnas_clave, how="inner")  # Cambia el tipo de unión según sea necesario

In [157]:
pisos_madrid_unido.columns.duplicated().sum()

0

In [158]:
pisos_madrid_unido.shape

(1188, 34)

In [159]:
columnas_repetidas=["descripcion_x","precio_x",	"cant_dormitorios_x","cant_banos_x","metros_cuadrados_construidos_x","descripcion_extendida_x"]

pisos_madrid_unido= pisos_madrid_unido.drop(columns=columnas_repetidas)

In [160]:
pisos_madrid_unido= pisos_madrid_unido.dropna(subset=["barrios","distrito"],how="all")

In [161]:
pisos_madrid_unido

,url_website,descripcion_y,cant_dormitorios_y,cant_banos_y,metros_cuadrados_construidos_y,descripcion_extendida_y,tipo_lugar,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio_y,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,barrios,distrito,cod_distrito,cod_barrio,cod_barrio_2,codigo_Postal
0,https://www.pisosmadrid.com.es/propiedad/40866...,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso reformado de 60m2 enfrente del R...,Piso,0,0,0,PM4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0,Ibiza,Retiro,3,34,4,28009
1,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,0,1,1,PM4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0,Ensanche de Vallecas,Villa de Vallecas,18,183,3,28031
2,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,0,0,0,PM4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0,Delicias,Arganzuela,2,25,5,28045
3,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en Los Angeles Madrid,3,1,57,Vivienda situada en Calle Godella en Madrid ca...,Piso,0,0,0,PM4086495,2024-10-24,40,70256,0,0,0,0,0,0,0,0,los angeles,Villaverde,17,174,4,28021
5,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en San Diego Madrid,1,1,21,Vivienda situada en Calle Callejo en Madrid ca...,Piso,0,0,0,PM4086488,2024-10-24,20,69313,0,0,0,0,0,0,0,0,San Diego,Puente de Vallecas,13,132,2,28053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,https://www.pisosmadrid.com.es/propiedad/38671...,Garaje en venta en Madrid Madrid,0,0,9,JM Plaza de Garaje de 9 m2 situada en la plan...,Garaje,0,0,0,PM3865049,2024-05-18,140,60000,0,0,0,0,0,0,0,0,entrevias,Puente de Vallecas,13,131,1,28053
1184,https://www.pisosmadrid.com.es/propiedad/38668...,Local en venta en Ensanche de Vallecas Madrid,0,2,89,SERVICHECK INMOBILIARIA Vende bonito local es ...,Local,0,0,0,PM3863985,2024-05-17,70,280000,1,0,0,0,0,0,1,1,Ensanche de Vallecas,Villa de Vallecas,18,183,3,28031
1185,https://www.pisosmadrid.com.es/propiedad/38666...,Piso en venta en Madrid,3,1,54,PISO EN MADRID Vivienda situada en Avenida Pal...,Piso,0,0,0,PM3863841,2024-05-17,120,60000,0,0,0,0,0,0,0,0,NaN,salamanca,4,<NA>,<NA>,<NA>
1186,https://www.pisosmadrid.com.es/propiedad/38666...,Garaje en venta en Castellana Madrid,0,0,25,GARAJE EN MADRID Garaje situado en la Paseo de...,Garaje,0,0,0,PM3863823,2024-05-17,150,34125,0,0,0,0,0,0,0,0,Castellana,Salamanca,4,46,6,28006


In [162]:
pisos_madrid_unido.shape

(1157, 28)

In [163]:
pisos_madrid_unido.isna().sum()

url_website                         0
descripcion_y                       0
cant_dormitorios_y                  0
cant_banos_y                        0
metros_cuadrados_construidos_y      0
descripcion_extendida_y             0
tipo_lugar                          0
piscina                             0
garaje                              0
jardin                              0
referencia                          0
fecha_publicacion                   1
n.visitas                           0
precio_y                            0
calefaccion                         0
trastero                            0
amueblado                           0
chimenea                            0
lavadero                            0
videoportero                        0
alarma                              0
aire_acondicionado                  0
barrios                           116
distrito                            0
cod_distrito                        0
cod_barrio                        116
cod_barrio_2

### **_Aqui se termina la union de los dos ficheros de pisos madrid y empezamos con redpiso_**

----------

In [164]:
## Aqui empieza el primer tratamiento de redpiso normal

In [165]:
redpiso_normal= pd.read_excel("Redpiso.xlsx")

In [166]:
redpiso_normal.columns

Index(['Título', 'Imagen', 'item', 'item1', 'propertylistoptionsitememblem',
       'propertylistoptionsitem', 'propertylistoptionsitem2',
       'propertylistoptionsitem3', 'itemlink_URL', 'itemlink', 'Imagen4',
       'Número', 'Número5', 'officelink_URL', 'officelink'],
      dtype='object')

In [167]:
# Conversion de letras a minusculas en las columnas
redpiso_normal.columns= redpiso_normal.columns.str.lower()

In [168]:
redpiso_normal["item"].value_counts()

item
Salon                955
Almacén              250
Fachada(exterior)    177
Cocina               102
Terraza               93
Dormitorio            82
Vistas                71
Entorno               70
Zonas Comunes         59
Baño                  38
Hall de Entrada       35
Plano                 26
Entrada               25
Detalles              25
Piscina               17
Patio                 15
Recibidor             11
Comedor               11
Pasillo               11
Estancia               7
Jardin                 6
Garaje                 1
Sótano                 1
Tendedero              1
Name: count, dtype: int64

In [169]:
redpiso_normal.head()

,título,imagen,item,item1,propertylistoptionsitememblem,propertylistoptionsitem,propertylistoptionsitem2,propertylistoptionsitem3,itemlink_url,itemlink,imagen4,número,número5,officelink_url,officelink
0,106.500 €,NaN,Salon,1/15,RESERVADO,64 m²,2 hab.,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en CALLE BENIMAMET, San Cristóba...",NaN,tel:919208536,919208536.0,https://www.redpiso.es/oficina/inmobiliaria-sa...,REDPISO SAN CRISTOBAL
1,119.000 €,NaN,Salon,1/16,NaN,30 m²,1 hab.,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en Usera, Madrid, Madrid",https://www.redpiso.es/img/e-min.jpg,tel:914282010,914282010.0,https://www.redpiso.es/oficina/inmobiliaria-us...,REDPISO USERA-ORCASITAS
2,120.000 €,NaN,Salon,1/10,RESERVADO,37 m²,1 hab.,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en Buenavista, Carabanchel, Madr...",https://www.redpiso.es/img/c-min.jpg,tel:915996359,915996359.0,https://www.redpiso.es/oficina/inmobiliaria-pa...,REDPISO PAU CARABANCHEL
3,120.000 €,NaN,Salon,1/7,RESERVADO,33 m²,1 hab.,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en Buenavista, Carabanchel, Madr...",https://www.redpiso.es/img/d-min.jpg,tel:915996359,915996359.0,https://www.redpiso.es/oficina/inmobiliaria-pa...,REDPISO PAU CARABANCHEL
4,127.500 €,https://www.redpiso.es/upload/properties/thumb...,Salon,1/19,RESERVADO,58 m²,3 hab.,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en CALLE BENIFERRI, San Cristóba...",NaN,tel:919208536,919208536.0,https://www.redpiso.es/oficina/inmobiliaria-sa...,REDPISO SAN CRISTOBAL


In [170]:
# Columnas que seran eliminadas ya previamente analizadas.
columnas_a_eliminar = ["item","item1","imagen4","propertylistoptionsitem3","número5"]

redpiso_normal= redpiso_normal.drop(columns=columnas_a_eliminar)

In [171]:
# Renombramiento de la columna.
redpiso_normal = redpiso_normal.rename(columns={"título": "precio"})
redpiso_normal = redpiso_normal.rename(columns={"imagen": "url_imagen"})
redpiso_normal = redpiso_normal.rename(columns={"propertylistoptionsitememblem": "reservado/sin_reserva"})
redpiso_normal = redpiso_normal.rename(columns={"propertylistoptionsitem": "espacio_metros_cuadrados"})
redpiso_normal = redpiso_normal.rename(columns={"propertylistoptionsitem2": "cantidad_habitaciones"})
redpiso_normal = redpiso_normal.rename(columns={"itemlink_url": "url_piso/id"})
redpiso_normal = redpiso_normal.rename(columns={"itemlink": "descripcion"})
redpiso_normal = redpiso_normal.rename(columns={"número": "contacto"})
redpiso_normal = redpiso_normal.rename(columns={"officelink": "nombre_oficina"})

In [172]:
# quitamos la palabra tel: y solo dejamos los valores numeros 
redpiso_normal["contacto"] = redpiso_normal["contacto"].str.replace(r'(?i)^tel:', "", regex=True)

In [173]:
redpiso_normal["contacto"].isna().sum()

0

In [174]:
# Cree una funcion que que toma los espacios y los caracteres especiales sin dejar algo detras.
def limpiar_valores(valor):
    if type(valor) == str:
        valor = re.sub(r'[^\w\s]','', valor)
        valor = valor.strip()
    return valor

# Cree una funcion corta que recorre cada valor del dataset y toma los valores que especificmaos en la funcion anterior.
redpiso_normal= redpiso_normal.apply(lambda col: col.apply(limpiar_valores))

In [175]:
# Aqui se drtuvo el codigo y para seguir lo comente todo pero hay que checar esta modificacion

# # Relleno de los valores nulos con numero 0
# redpiso_normal["contacto"]= redpiso_normal["contacto"].fillna(0)
# #Conversion columna a tipo int
# redpiso_normal["contacto"]= redpiso_normal["contacto"].astype("int")

In [176]:
redpiso_normal["contacto"].dtype

dtype('O')

In [177]:
# Rellena los valores nulos a sin_reserva
redpiso_normal["reservado/sin_reserva"]= redpiso_normal["reservado/sin_reserva"].fillna("sin_reserva")

In [178]:
# Expresion regular que recorre la columna extrae los numeros de la cada valor y elimina lo demas.aplicada a la columna que se muestra.

redpiso_normal["cantidad_habitaciones"] = redpiso_normal["cantidad_habitaciones"].str.extract(r'(\d+)')

redpiso_normal["espacio_metros_cuadrados"] = redpiso_normal["espacio_metros_cuadrados"].str.extract(r'(\d+)')

In [179]:
redpiso_normal

,precio,url_imagen,reservado/sin_reserva,espacio_metros_cuadrados,cantidad_habitaciones,url_piso/id,descripcion,contacto,officelink_url,nombre_oficina
0,106500,NaN,RESERVADO,64,2,httpswwwredpisoesinmueblepisoenventaencalleben...,Piso en venta en CALLE BENIMAMET San Cristóbal...,919208536,httpswwwredpisoesoficinainmobiliariasancristob...,REDPISO SAN CRISTOBAL
1,119000,NaN,sin_reserva,30,1,httpswwwredpisoesinmueblepisoenventaenuseramad...,Piso en venta en Usera Madrid Madrid,914282010,httpswwwredpisoesoficinainmobiliariauseraorcas...,REDPISO USERAORCASITAS
2,120000,NaN,RESERVADO,37,1,httpswwwredpisoesinmueblepisoenventaenbuenavis...,Piso en venta en Buenavista Carabanchel Madrid...,915996359,httpswwwredpisoesoficinainmobiliariapaucaraban...,REDPISO PAU CARABANCHEL
3,120000,NaN,RESERVADO,33,1,httpswwwredpisoesinmueblepisoenventaenbuenavis...,Piso en venta en Buenavista Carabanchel Madrid...,915996359,httpswwwredpisoesoficinainmobiliariapaucaraban...,REDPISO PAU CARABANCHEL
4,127500,httpswwwredpisoesuploadpropertiesthumbsimg_951...,RESERVADO,58,3,httpswwwredpisoesinmueblepisoenventaencalleben...,Piso en venta en CALLE BENIFERRI San Cristóbal...,919208536,httpswwwredpisoesoficinainmobiliariasancristob...,REDPISO SAN CRISTOBAL
...,...,...,...,...,...,...,...,...,...,...
2103,A consultar,NaN,sin_reserva,NaN,3,httpswwwredpisoesinmueblepisoenventaencalleori...,Piso en venta en CALLE ORIO 4 Los Ángeles Vill...,913173186,httpswwwredpisoesoficinainmobiliariaciudaddelo...,REDPISO CIUDAD DE LOS ANGELES
2104,A consultar,NaN,sin_reserva,90,3,httpswwwredpisoesinmueblepisoenventaencallecan...,Piso en venta en CALLE CANCION DEL OLVIDO 12 L...,913173186,httpswwwredpisoesoficinainmobiliariaciudaddelo...,REDPISO CIUDAD DE LOS ANGELES
2105,A consultar,NaN,sin_reserva,79,3,httpswwwredpisoesinmueblepisoenventaencallelaa...,Piso en venta en CALLE LA ALEGRIA DE LA HUERTA...,913173186,httpswwwredpisoesoficinainmobiliariaciudaddelo...,REDPISO CIUDAD DE LOS ANGELES
2106,A consultar,httpswwwredpisoesuploadpropertiesthumbsimg_957...,sin_reserva,76,3,httpswwwredpisoesinmueblepisoenventaencalleoch...,Piso en venta en CALLE Ochagavia Valdezarza Mo...,912244889,httpswwwredpisoesoficinainmobiliariavaldezarza...,REDPISO VALDEZARZAPUERTA DE HIERRO


In [180]:
# Se convierte los valores RESERVADO a 1 y todo los demas a 0, basicamente los que aun no estan reservados seran 0.
redpiso_normal["reservado/sin_reserva"] = redpiso_normal ["reservado/sin_reserva"].apply(lambda x : 1 if x.strip().upper()== "RESERVADO" else 0)

In [181]:
redpiso_normal.isna().sum()

precio                        0
url_imagen                  723
reservado/sin_reserva         0
espacio_metros_cuadrados      7
cantidad_habitaciones         5
url_piso/id                   0
descripcion                   0
contacto                      0
officelink_url                0
nombre_oficina                0
dtype: int64

In [182]:
redpiso_normal.shape

(2108, 10)

### **_Aqui se termina el tratamiento de redpiso_normal,ahora sigue el procesmiento de redpiso_detalles_**

In [183]:
redpiso_detalles= pd.read_excel("Detalle_RedPiso.xlsx")

In [184]:
redpiso_detalles.head(3)

,Título,Campo1,Campo2,Campo3,Campo4,Campo5,Campo6,Tiempo,Tiempo1,Campo7,Campo8,Campo9,Campo10,Campo11,Campo12,Campo13,Tiempo2,Campo14,Campo15,Tiempo3,Campo16,Campo17,Campo18,Tiempo4,Campo19
0,"Piso en venta en CALLE BENIMAMET, San Cristóba...",106.500 €,"Piso en venta en CALLE BENIMAMET, San Cristóba...",ESTUPENDA VIVIENDA EN CALLE BENIMAMET,RP2842024125161,"Piso en venta en CALLE BENIMAMET, San Cristóba...",REF: RP2842024125161,Nº Visitas 5857,02/04/2024,64 m²,2.0,1.0,ESTUPENDA VIVIENDA EN CALLE BENIMAMET,"Redpiso San Cristobal de los Ángeles, inscrito...",Metros: 64 m²,Metros útiles: 60 m²,Dormitorios: 2,Exterior,Estado: Perfecto Estado,Baños: 1,Aire acondicionado,Agua caliente: Individual,Tipo de suelo: Tarima normal,Año de construcción: 1962,Antigüedad: Más de 50 años
1,"Piso en venta en Usera, Madrid, Madrid",119.000 €,"Piso en venta en Usera, Madrid, Madrid\n ...",ESTUDIO JARDINES DE YBARRA II,RP782024130863,"Piso en venta en Usera, Madrid, Madrid",REF: RP782024130863,Nº Visitas 714,02/10/2024,30 m²,1.0,1.0,ESTUDIO JARDINES DE YBARRA II,Redpiso vende en exclusiva ACOGEDOR ESTUDIO en...,Metros: 30 m²,Metros útiles: 27 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Estado: Perfecto Estado,Orientación: Sur,Exterior,Agua caliente: Individual,Cocina equipada
2,"Piso en venta en Buenavista, Carabanchel, Madr...",120.000 €,"Piso en venta en Buenavista, Carabanchel, Madr...",¡Tu nuevo hogar te espera en Buena Vista!,RP442024126007,"Piso en venta en Buenavista, Carabanchel, Madr...",REF: RP442024126007,Nº Visitas 3387,26/04/2024,37 m²,1.0,1.0,¡Tu nuevo hogar te espera en Buena Vista!,Redpiso vende en exclusiva este encantador est...,Metros: 37 m²,Metros útiles: 34 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Consumo: 111 KW h/m² año,Emisiones: 18 Kg CO2/m² año,Estado: Perfecto Estado,Exterior,Orientación: Sur


In [185]:
# renombramiento de columnas 
redpiso_detalles = redpiso_detalles.rename( columns={"Campo1": "precio"})
redpiso_detalles = redpiso_detalles.rename(columns={"Campo2":"descripcion"})
redpiso_detalles = redpiso_detalles.rename(columns={"Tiempo":"cantidad_visitas"})
redpiso_detalles = redpiso_detalles.rename(columns={"Tiempo1":"fecha_publicacion"})
redpiso_detalles = redpiso_detalles.rename(columns={"Campo7":"espacio_metros_cuadrados"})
redpiso_detalles = redpiso_detalles.rename(columns={"Tiempo1":"fecha_publicacion"})
redpiso_detalles = redpiso_detalles.rename(columns={"Campo6": "referencia_web"})
redpiso_detalles = redpiso_detalles.rename( columns={"título": "titulo"})
redpiso_detalles= redpiso_detalles.rename(columns={"campo13":"metros_utiles"})
redpiso_detalles = redpiso_detalles.rename(columns={"tiempo2": "cant_dormitorios"})
redpiso_detalles = redpiso_detalles.rename(columns={"tiempo3": "banos"})
redpiso_detalles = redpiso_detalles.rename(columns={"campo15": "cant_armarios"})
redpiso_detalles= redpiso_detalles.rename(columns={"campo14":"calidad_energetica"})

In [186]:
# eliminacion de columnas
redpiso_detalles= redpiso_detalles.drop("descripcion", axis=1)
redpiso_detalles = redpiso_detalles.drop("Campo5", axis=1)
redpiso_detalles = redpiso_detalles.drop("Campo9",axis=1)
redpiso_detalles = redpiso_detalles.drop("Campo10",axis=1)
redpiso_detalles = redpiso_detalles.drop ("Campo11", axis=1)
redpiso_detalles = redpiso_detalles.drop("Campo12",axis=1)
redpiso_detalles = redpiso_detalles.drop("Campo8", axis= 1)
redpiso_detalles = redpiso_detalles.drop("Campo3", axis= 1)
redpiso_detalles = redpiso_detalles.drop("Campo4", axis= 1)

In [187]:
redpiso_detalles.head(3)

,Título,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,Campo13,Tiempo2,Campo14,Campo15,Tiempo3,Campo16,Campo17,Campo18,Tiempo4,Campo19
0,"Piso en venta en CALLE BENIMAMET, San Cristóba...",106.500 €,REF: RP2842024125161,Nº Visitas 5857,02/04/2024,64 m²,Metros útiles: 60 m²,Dormitorios: 2,Exterior,Estado: Perfecto Estado,Baños: 1,Aire acondicionado,Agua caliente: Individual,Tipo de suelo: Tarima normal,Año de construcción: 1962,Antigüedad: Más de 50 años
1,"Piso en venta en Usera, Madrid, Madrid",119.000 €,REF: RP782024130863,Nº Visitas 714,02/10/2024,30 m²,Metros útiles: 27 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Estado: Perfecto Estado,Orientación: Sur,Exterior,Agua caliente: Individual,Cocina equipada
2,"Piso en venta en Buenavista, Carabanchel, Madr...",120.000 €,REF: RP442024126007,Nº Visitas 3387,26/04/2024,37 m²,Metros útiles: 34 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Consumo: 111 KW h/m² año,Emisiones: 18 Kg CO2/m² año,Estado: Perfecto Estado,Exterior,Orientación: Sur


In [188]:
# Columnas a minusculas
redpiso_detalles.columns = redpiso_detalles.columns.str.lower()

In [189]:
redpiso_detalles.isna().sum()

título                        0
precio                        0
referencia_web                0
cantidad_visitas              0
fecha_publicacion             0
espacio_metros_cuadrados      6
campo13                       3
tiempo2                       4
campo14                      14
campo15                       8
tiempo3                       5
campo16                      20
campo17                      49
campo18                      82
tiempo4                     186
campo19                     123
dtype: int64

In [190]:
redpiso_detalles

,título,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,campo13,tiempo2,campo14,campo15,tiempo3,campo16,campo17,campo18,tiempo4,campo19
0,"Piso en venta en CALLE BENIMAMET, San Cristóba...",106.500 €,REF: RP2842024125161,Nº Visitas 5857,02/04/2024,64 m²,Metros útiles: 60 m²,Dormitorios: 2,Exterior,Estado: Perfecto Estado,Baños: 1,Aire acondicionado,Agua caliente: Individual,Tipo de suelo: Tarima normal,Año de construcción: 1962,Antigüedad: Más de 50 años
1,"Piso en venta en Usera, Madrid, Madrid",119.000 €,REF: RP782024130863,Nº Visitas 714,02/10/2024,30 m²,Metros útiles: 27 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Estado: Perfecto Estado,Orientación: Sur,Exterior,Agua caliente: Individual,Cocina equipada
2,"Piso en venta en Buenavista, Carabanchel, Madr...",120.000 €,REF: RP442024126007,Nº Visitas 3387,26/04/2024,37 m²,Metros útiles: 34 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Consumo: 111 KW h/m² año,Emisiones: 18 Kg CO2/m² año,Estado: Perfecto Estado,Exterior,Orientación: Sur
3,"Piso en venta en Buenavista, Carabanchel, Madr...",120.000 €,REF: RP442024126427,Nº Visitas 2924,10/05/2024,33 m²,Metros útiles: 30 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Consumo: 145 KW h/m² año,Emisiones: 24 Kg CO2/m² año,Estado: Perfecto Estado,Agua caliente: Individual,Orientación: Oeste
4,"Piso en venta en CALLE BENIFERRI, San Cristóba...",127.500 €,REF: RP2842024130186,Nº Visitas 485,12/09/2024,58 m²,Metros útiles: 50 m²,Dormitorios: 3,Exterior,Estado: Perfecto Estado,Baños: 1,Ascensor,Aire acondicionado,Agua caliente: Gas natural,Tipo de suelo: Terrazo,Calefacción: Gas Natural
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,"Piso en venta en CALLE ORIO, 4, Los Ángeles, V...",A consultar,REF: RP332024126817,Nº Visitas 240,22/05/2024,NaN,Baños: 1,Cal. energética:,Exterior,Orientación: Norte,Estado: A reformar,Agua caliente: Gas natural,Calefacción: Gas Natural,Tipo de suelo: Baldosa,Antigüedad: Más de 50 años,Tipo de fachada: Cemento
2104,"Piso en venta en CALLE CANCION DEL OLVIDO, 12,...",A consultar,REF: RP332024127980,Nº Visitas 218,24/06/2024,90 m²,Metros útiles: 71 m²,Dormitorios: 3,Ascensor,Cal. energética:,Baños: 1,Agua caliente: Individual,Tipo de fachada: Otros materiales,Año de construcción: 1960,NaN,"Comunidad: 65,00 €"
2105,Piso en venta en CALLE LA ALEGRIA DE LA HUERTA...,A consultar,REF: RP332024128702,Nº Visitas 172,15/07/2024,79 m²,Metros útiles: 63 m²,Dormitorios: 3,Estado: Perfecto Estado,Cal. energética:,Baños: 1,Exterior,Ascensor,Aire acondicionado,Calefacción: Eléctrica,Agua caliente: Gas natural
2106,"Piso en venta en CALLE Ochagavia, Valdezarza, ...",A consultar,REF: RP402024127484,Nº Visitas 845,11/06/2024,76 m²,Metros útiles: 69 m²,Dormitorios: 3,Cal. energética:,Armarios: 2,Baños: 1,Consumo: 172 KW h/m² año,Emisiones: 36 Kg CO2/m² año,Orientación: Oeste,Ascensor,Exterior


In [191]:
# Eliminacion de las primeras 3 letras en la referencia web - REF- dejando solo la referencia 
redpiso_detalles["referencia_web"] = redpiso_detalles["referencia_web"].str[3:].str.strip()

In [192]:
# Reemplace todas las letras ñ por n.
redpiso_detalles = redpiso_detalles.apply(lambda col : col.str.replace("ñ","n")if col.dtype == "object" else col)

# funcion que que toma los espacios y los caracteres especiales sin dejar algo detras.
def limpiar_valores(valor):
    if type(valor) == str:
        valor = re.sub(r'[^\w\s]','', valor)
        valor = valor.strip()
    return valor 

#  Cree una funcion corta que recorre cada valor del dataset y toma los valores que especificmaos en la funcion anterior.
redpiso_detalles = redpiso_detalles.apply(lambda col: col.apply(limpiar_valores))


# Cree una funcion que elimina todos los acentos  y caracteres especiales
redpiso_detalles = redpiso_detalles.apply(lambda x: x.apply(lambda y: unidecode(y) if isinstance(y,str) else y )if x.dtype == "object" else x)



In [193]:
redpiso_detalles

,título,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,campo13,tiempo2,campo14,campo15,tiempo3,campo16,campo17,campo18,tiempo4,campo19
0,Piso en venta en CALLE BENIMAMET San Cristobal...,106500,RP2842024125161,No Visitas 5857,02042024,64 m2,Metros utiles 60 m2,Dormitorios 2,Exterior,Estado Perfecto Estado,Banos 1,Aire acondicionado,Agua caliente Individual,Tipo de suelo Tarima normal,Ano de construccion 1962,Antiguedad Mas de 50 anos
1,Piso en venta en Usera Madrid Madrid,119000,RP782024130863,No Visitas 714,02102024,30 m2,Metros utiles 27 m2,Dormitorios 1,Cal energetica,Armarios 1,Banos 1,Estado Perfecto Estado,Orientacion Sur,Exterior,Agua caliente Individual,Cocina equipada
2,Piso en venta en Buenavista Carabanchel Madrid...,120000,RP442024126007,No Visitas 3387,26042024,37 m2,Metros utiles 34 m2,Dormitorios 1,Cal energetica,Armarios 1,Banos 1,Consumo 111 KW hm2 ano,Emisiones 18 Kg CO2m2 ano,Estado Perfecto Estado,Exterior,Orientacion Sur
3,Piso en venta en Buenavista Carabanchel Madrid...,120000,RP442024126427,No Visitas 2924,10052024,33 m2,Metros utiles 30 m2,Dormitorios 1,Cal energetica,Armarios 1,Banos 1,Consumo 145 KW hm2 ano,Emisiones 24 Kg CO2m2 ano,Estado Perfecto Estado,Agua caliente Individual,Orientacion Oeste
4,Piso en venta en CALLE BENIFERRI San Cristobal...,127500,RP2842024130186,No Visitas 485,12092024,58 m2,Metros utiles 50 m2,Dormitorios 3,Exterior,Estado Perfecto Estado,Banos 1,Ascensor,Aire acondicionado,Agua caliente Gas natural,Tipo de suelo Terrazo,Calefaccion Gas Natural
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,Piso en venta en CALLE ORIO 4 Los Angeles Vill...,A consultar,RP332024126817,No Visitas 240,22052024,NaN,Banos 1,Cal energetica,Exterior,Orientacion Norte,Estado A reformar,Agua caliente Gas natural,Calefaccion Gas Natural,Tipo de suelo Baldosa,Antiguedad Mas de 50 anos,Tipo de fachada Cemento
2104,Piso en venta en CALLE CANCION DEL OLVIDO 12 L...,A consultar,RP332024127980,No Visitas 218,24062024,90 m2,Metros utiles 71 m2,Dormitorios 3,Ascensor,Cal energetica,Banos 1,Agua caliente Individual,Tipo de fachada Otros materiales,Ano de construccion 1960,NaN,Comunidad 6500
2105,Piso en venta en CALLE LA ALEGRIA DE LA HUERTA...,A consultar,RP332024128702,No Visitas 172,15072024,79 m2,Metros utiles 63 m2,Dormitorios 3,Estado Perfecto Estado,Cal energetica,Banos 1,Exterior,Ascensor,Aire acondicionado,Calefaccion Electrica,Agua caliente Gas natural
2106,Piso en venta en CALLE Ochagavia Valdezarza Mo...,A consultar,RP402024127484,No Visitas 845,11062024,76 m2,Metros utiles 69 m2,Dormitorios 3,Cal energetica,Armarios 2,Banos 1,Consumo 172 KW hm2 ano,Emisiones 36 Kg CO2m2 ano,Orientacion Oeste,Ascensor,Exterior


In [194]:
valores_buscar1= ["Dormitorios 1", "Dormitorios 2", "Dormitorios 3", "Dormitorios 4","Dormitorios1", "Dormitorios2","Dormitorios3", "Dormitorios4",
                  "Dormitorios 5", "Dormitorios 6", "Dormitorios 7", "Dormitorios 8","Dormitorios  8", "Dormitorios 9", "Dormitorios 10", "Dormitorios5","Dormitorios6",
                  "Dormitorios7", "Dormitorios8", "Dormitoros9"]

valores_buscar2= ["Banos 1", "Banos 2", "Banos 3", "Banos 4","Banos 5","Banos 6","Banos 7","Banos 8",
                   "Banos1", "Banos2", "Banos3", "Banos4", "Banos5", "Banos6", "Banos7","Banos8"]

nueva_columna = "cantidad_dormitorios"
nueva_columna2 = "cantidad_banos"



In [195]:
# Creacion de columnas nuevas y rellenarlos con nulos.  Despues iterar sobre las columnas y rellenarlas con los valores declararlos en las listas de atras


redpiso_detalles[nueva_columna]= np.nan
redpiso_detalles[nueva_columna2]= np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles .columns[:-1]:
        if pd.notna(row[col]):
            for valor in valores_buscar1:
                if valor in str(row[col]):
                    redpiso_detalles.at[index, nueva_columna] =valor 
                    redpiso_detalles.at[index, col]= np.nan
                    break




for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles .columns[:-1]:
        if pd.notna(row[col]):
            for valor in valores_buscar2:
                if valor in str(row[col]):
                    redpiso_detalles.at[index, nueva_columna2] =valor 
                    redpiso_detalles.at[index, col]= np.nan
                    break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\3484431668.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Dormitorios 2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna] =valor
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\3484431668.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Banos 1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna2] =valor


In [196]:

nueva_columna3= "cantidad_armarios"
patron = r'(?i)^armario' 
redpiso_detalles[nueva_columna3]= np.nan

redpiso_detalles[nueva_columna3] = redpiso_detalles[nueva_columna3].astype("object") # Conversion de columna float64 a object.

# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron, regex=True).any():
                redpiso_detalles.at[index, nueva_columna3]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [197]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna4 = "consumo_energia"
patron_2 = r'(?i)^consumo'
redpiso_detalles[nueva_columna4]= np.nan
redpiso_detalles[nueva_columna4] = redpiso_detalles[nueva_columna4].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_2, regex=True).any():
                redpiso_detalles.at[index, nueva_columna4]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [198]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna5 = "emisiones_CO2"
patron_3 = r'(?i)^emisiones'
redpiso_detalles[nueva_columna5]= np.nan
redpiso_detalles[nueva_columna5] = redpiso_detalles[nueva_columna5].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_3, regex=True).any():
                redpiso_detalles.at[index, nueva_columna5]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [199]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna6 = "cal_energetica"
patron_4 = r'(?i)^cal'
redpiso_detalles[nueva_columna6]= np.nan
redpiso_detalles[nueva_columna6] = redpiso_detalles[nueva_columna6].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_4, regex=True).any():
                redpiso_detalles.at[index, nueva_columna6]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break



In [200]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna7="condicino-estado"
patron_5 =  r'(?i)^estado'
redpiso_detalles[nueva_columna7]= np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_5, regex=True).any():
                redpiso_detalles.at[index, nueva_columna7]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\547808622.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Estado Perfecto Estado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna7]= row[col]


In [201]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna8 = "antiguedad"
patron_6= r'(?i)^antiguedad'
redpiso_detalles[nueva_columna8]= np.nan
redpiso_detalles[nueva_columna8] = redpiso_detalles[nueva_columna8].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_6, regex=True).any():
                redpiso_detalles.at[index, nueva_columna8]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [202]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna9 = "ascensor"
patron_7= r'(?i)^ascensor'
redpiso_detalles[nueva_columna9]= np.nan
redpiso_detalles[nueva_columna9] = redpiso_detalles[nueva_columna9].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_7, regex=True).any():
                redpiso_detalles.at[index, nueva_columna9]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [203]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna10 = "aire_acondicionado"
patron_8= r'(?i)^aire'
redpiso_detalles[nueva_columna10]= np.nan
redpiso_detalles[nueva_columna10] = redpiso_detalles[nueva_columna10].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_8, regex=True).any():
                redpiso_detalles.at[index, nueva_columna10]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [204]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna11 = "orientacion"
patron_9= r'(?i)^orientacion'
redpiso_detalles[nueva_columna11]= np.nan
redpiso_detalles[nueva_columna11] = redpiso_detalles[nueva_columna11].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_9, regex=True).any():
                redpiso_detalles.at[index, nueva_columna11]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [205]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna12 = "sistema_agua"
patron_10= r'(?i)^agua'
redpiso_detalles[nueva_columna12]= np.nan
redpiso_detalles[nueva_columna12] = redpiso_detalles[nueva_columna12].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_10, regex=True).any():
                redpiso_detalles.at[index, nueva_columna12]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [206]:
patron_11_antiguedad= r'^ano|año'

# Aqui a diferencia de los otros bucles iterativos se busco integrar los valores que iniciaran en ano y año a una columna ya existente.
for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_11_antiguedad, regex=True, flags=re.IGNORECASE).any():
                redpiso_detalles.at[index,nueva_columna8]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [207]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna13 = "sistema_calefaccion"
patron_12= r'(?i)^calefaccion'
redpiso_detalles[nueva_columna13]= np.nan
redpiso_detalles[nueva_columna13] = redpiso_detalles[nueva_columna13].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_12, regex=True).any():
                redpiso_detalles.at[index, nueva_columna13]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [208]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna14 = "Tipo de suelo_fachada"
patron_13= r'(?i)^tipo'
redpiso_detalles[nueva_columna14]= np.nan
redpiso_detalles[nueva_columna14] = redpiso_detalles[nueva_columna14].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_13, regex=True).any():
                redpiso_detalles.at[index, nueva_columna14]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break


# patron_14_tipo= r'^\S*tipo' #  Asegura que no haya espacios antes de palabra. Expresion regular.
# # Aqui se ejerce la busqueda de tipo pero reconociendo espacios antes de la palabra. Despues de coloca en la columna tipo de suelo_fachada.
# for index, row in datos.iterrows():
#     for col in datos.columns[:-1]:
#         if pd.notna(row[col]):
#             valor= str(row[col])
#             if pd.Series(valor).str.contains(patron_14_tipo, regex=True, flags=re.IGNORECASE).any():
#                 datos.at[index,nueva_columna14]= row[col]
#                 datos.at[index, col]= np.nan
#                 break

In [209]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna15 = "cocina_equipada"
patron_15= r'(?i)^cocina'


redpiso_detalles[nueva_columna15]= np.nan
redpiso_detalles[nueva_columna15] = redpiso_detalles[nueva_columna15].astype("object")
for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_15, regex=True).any():
                redpiso_detalles.at[index, nueva_columna15]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [210]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna16 = "garajes"
patron_16= r'(?i)^garajes'
redpiso_detalles[nueva_columna16]= np.nan
redpiso_detalles[nueva_columna16] = redpiso_detalles[nueva_columna16].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_16, regex=True).any():
                redpiso_detalles.at[index, nueva_columna16]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break



In [211]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_17_metros= r'(?i)^metros'
nueva_columna17 = "m_cuadrados_utiles"
redpiso_detalles[nueva_columna17] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_17_metros, regex=True).any():
                redpiso_detalles.at[index, nueva_columna17]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

# patron_18_metros= r'^\S*metros'
# for index, row in redpiso_detalles.iterrows():
#     for col in redpiso_detalles.columns[:-1]:
#         if pd.notna(row[col]):
#             valor= str(row[col])
#             if pd.Series(valor).str.contains(patron_18_metros, regex=True, flags=re.IGNORECASE).any():
#                 redpiso_detalles.at[index,nueva_columna17]= row[col]
#                 redpiso_detalles.at[index, col]= np.nan
#                 break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\3142366154.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Metros utiles 60 m2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna17]= row[col]


In [212]:
redpiso_detalles["m_cuadrados_utiles"].value_counts()

m_cuadrados_utiles
Metros utiles 50 m2     50
Metros utiles 60 m2     47
Metros utiles 70 m2     43
Metros utiles 80 m2     40
Metros utiles 65 m2     40
                        ..
Metros utiles 240 m2     1
Metros utiles 318 m2     1
Metros utiles 205 m2     1
Metros utiles 278 m2     1
Metros utiles 181 m2     1
Name: count, Length: 193, dtype: int64

In [213]:
redpiso_detalles["m_cuadrados_utiles"].value_counts()

m_cuadrados_utiles
Metros utiles 50 m2     50
Metros utiles 60 m2     47
Metros utiles 70 m2     43
Metros utiles 80 m2     40
Metros utiles 65 m2     40
                        ..
Metros utiles 240 m2     1
Metros utiles 318 m2     1
Metros utiles 205 m2     1
Metros utiles 278 m2     1
Metros utiles 181 m2     1
Name: count, Length: 193, dtype: int64

In [214]:
# patron_19_cal= r'^\S*Cal'

# for index, row in datos.iterrows():
#     for col in datos.columns[:-1]:
#         if pd.notna(row[col]):
#             valor= str(row[col])
#             if pd.Series(valor).str.contains(patron_19_cal, regex=True, flags=re.IGNORECASE).any():
#                 datos.at[index,nueva_columna6]= row[col]
#                 datos.at[index, col]= np.nan
#                 break


In [215]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_20= r'(?i)^exterior'
nueva_columna18 = "vista zona"
redpiso_detalles[nueva_columna18] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_20, regex=True).any():
                redpiso_detalles.at[index, nueva_columna18]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\3286334401.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Exterior' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna18]= row[col]


In [216]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_21= r'(?i)^piscina'
nueva_columna19 = "piscina"
redpiso_detalles[nueva_columna19] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_21, regex=True).any():
                redpiso_detalles.at[index, nueva_columna19]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\995817297.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Piscina' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna19]= row[col]


In [217]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_22= r'(?i)^trastero'
nueva_columna22 = "trastero"
redpiso_detalles[nueva_columna22] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_22, regex=True).any():
                redpiso_detalles.at[index, nueva_columna22]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\4053109142.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trastero' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna22]= row[col]


In [218]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_23= r'(?i)^comunidad'
nueva_columna23 = "Comunidad/personas edificio"
redpiso_detalles[nueva_columna23] = np.nan


for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_23, regex=True).any():
                redpiso_detalles.at[index, nueva_columna23]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\3016210081.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Comunidad 7400' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna23]= row[col]


In [219]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_24= r'(?i)^zonas'
nueva_columna24 = "Zonas verdes"
redpiso_detalles[nueva_columna24] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_24, regex=True).any():
                redpiso_detalles.at[index, nueva_columna24]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\1774972238.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Zonas verdes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna24]= row[col]


In [220]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_25= r'(?i)^numero'
nueva_columna25 = "terrazas"
redpiso_detalles[nueva_columna25] = np.nan


for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_25, regex=True).any():
                redpiso_detalles.at[index, nueva_columna25]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\1566383252.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Numero de terrazas 1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna25]= row[col]


In [221]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_26= r'(?i)^acceso'
nueva_columna26 = "acceso_minusvalido"
redpiso_detalles[nueva_columna26] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_26, regex=True).any():
                redpiso_detalles.at[index, nueva_columna26]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\3499825224.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Acceso minusvalidos' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna26]= row[col]


In [222]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_27= r'(?i)^puerta'
nueva_columna27 = "puerta_seguridad"
redpiso_detalles[nueva_columna27] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_27, regex=True).any():
                redpiso_detalles.at[index, nueva_columna27]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\1113838515.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Puerta de seguridad' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna27]= row[col]


In [223]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_28= r'(?i)^conserje'
nueva_columna28 = "conserje"
redpiso_detalles[nueva_columna28] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_28, regex=True).any():
                redpiso_detalles.at[index, nueva_columna28]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\2160241313.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Conserje' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna28]= row[col]


In [224]:
# Aqui se filtra la palabra Conserje pero sin distincion de espacios antes de la palabra. 
patron_29_conserje= r'^\S*Conserje'
for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_29_conserje, regex=True, flags=re.IGNORECASE).any():
                redpiso_detalles.at[index,nueva_columna28]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [225]:
redpiso_detalles["conserje"].value_counts()

conserje
Conserje    11
Name: count, dtype: int64

In [226]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_30= r'(?i)^plantas'

nueva_columna29 = "plantas"

redpiso_detalles[nueva_columna29] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_30, regex=True).any():
                redpiso_detalles.at[index, nueva_columna29]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\5807621.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Plantas 5' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna29]= row[col]


In [227]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_31= r'(?i)^pisos'

nueva_columna30 = "N_piso"

redpiso_detalles[nueva_columna30] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_31, regex=True).any():
                redpiso_detalles.at[index, nueva_columna30]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\354679528.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Pisos 25' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna30]= row[col]


In [228]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_32= r'(?i)^alarma'

nueva_columna31 = "alarma"

redpiso_detalles[nueva_columna31] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_32, regex=True).any():
                redpiso_detalles.at[index, nueva_columna31]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\1007444481.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Alarma' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna31]= row[col]


In [229]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_33= r'(?i)^amueblado'

nueva_columna32 = "amueblado"

redpiso_detalles[nueva_columna32] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_33, regex=True).any():
                redpiso_detalles.at[index, nueva_columna32]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\1403479313.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Amueblado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna32]= row[col]


In [230]:
redpiso_detalles["amueblado"].value_counts()

amueblado
Amueblado    4
Name: count, dtype: int64

In [231]:
redpiso_detalles["campo15"].value_counts()

campo15
Cal energetica                 1
Tipo de suelo Tarima normal    1
Name: count, dtype: int64

In [232]:
redpiso_detalles["campo16"].value_counts()

campo16
Tipo de fachada Ladrillo    2
Tipo de fachada Cemento     1
Name: count, dtype: int64

In [233]:
redpiso_detalles["campo18"].value_counts()

campo18
Tipo de fachada Ladrillo     15
Tipo de fachada Cemento       7
Tipo de fachada Mono Capa     6
Name: count, dtype: int64

In [234]:
redpiso_detalles["campo19"].value_counts()

campo19
Tipo de fachada Ladrillo            44
Tipo de suelo Parquet               31
Tipo de suelo Gres                  27
Tipo de suelo Tarima normal         15
Tipo de fachada Cemento             13
Tipo de fachada Mono Capa           11
Tipo de suelo Tarima Flotante       10
Tipo de suelo Baldosa                6
Tipo de suelo Otros                  5
Tipo de suelo Terrazo                3
Tipo de suelo Porcelanico            2
Tipo de suelo Sintasol               2
Tipo de fachada Piedra               2
Tipo de suelo Mosaico Hidraulico     1
Name: count, dtype: int64

In [235]:
redpiso_detalles["tiempo2"].value_counts()

tiempo2
Metros utiles 71 m2     2
Metros utiles 73 m2     2
Metros utiles 57 m2     2
Metros utiles 69 m2     1
Metros utiles 150 m2    1
Metros utiles 120 m2    1
Metros utiles 313 m2    1
Metros utiles 123 m2    1
Metros utiles 90 m2     1
Metros utiles 72 m2     1
Metros utiles 487 m2    1
Metros utiles 94 m2     1
Metros utiles 52 m2     1
Metros utiles 166 m2    1
Metros utiles 228 m2    1
Metros utiles 162 m2    1
Metros utiles 160 m2    1
Metros utiles 74 m2     1
Metros utiles 63 m2     1
Name: count, dtype: int64

In [236]:
# aqui intento extraer los valores los valores numericos que hay cuando se encuentra la palabra metros en la columna tiempo2 y las llevo a m_cuadrados_utiles

def mover_valores_metros(df):
   
    # Expresión regular para buscar "metros" seguido de un número
    patron = r'(?i)\s*metros?.*?(\d+)\s*m2'  # "metros" o "metros" seguido de un número

    # Iterar sobre las filas del DataFrame
    for idx, row in df.iterrows():
        tiempo2_valor = str(row['tiempo2']).strip()  # Asegurarse de eliminar los espacios antes y después
        
        # Depuración: Mostrar valor de tiempo2
        print(f"Fila {idx}: valor en 'tiempo2': {tiempo2_valor}")
        
        # Buscar si el valor en la columna 'tiempo2' contiene 'metros' seguido de un número
        match = re.search(patron, tiempo2_valor)
        
        # Depuración: Mostrar si se encontró una coincidencia
        if match:
            metros_valor = match.group(1)  # Extraer el número que sigue a "metros"
            print(f"Se encontró 'metros' en la fila {idx}: {metros_valor}")  # Depuración
            # Si la columna 'm_cuadrados_utiles' está vacía, mover el valor de 'tiempo2'
            if pd.isna(df.at[idx, 'm_cuadrados_utiles']):
                df.at[idx, 'm_cuadrados_utiles'] = metros_valor  # Transferir el número a 'm_cuadrados_utiles'
                df.at[idx, 'tiempo2'] = np.nan  # Reemplazar el valor en 'tiempo2' por NaN
                print(f"Fila {idx}: Se movió el valor a 'm_cuadrados_utiles' y se vació 'tiempo2'.")  # Depuración
            else:
                print(f"Fila {idx}: 'm_cuadrados_utiles' ya tiene valor, no se hizo el cambio.")  # Depuración

    return df

# Llamar a la función sobre tu DataFrame
redpiso_detalles2 = mover_valores_metros(redpiso_detalles)

# Ver los cambios en las columnas relevantes
print(redpiso_detalles2[['tiempo2', 'm_cuadrados_utiles']].head())





# Hay datos que se filtraron correctamente a las columnas correspondientes como en m_cuadrados_utiles pero que no se retiraron correctamente 
# de otras columnas donde se tomaron los valores. Asi que habra valores en otras columnas pero ya estaran en la columna correspondiente tambien

Fila 0: valor en 'tiempo2': nan
Fila 1: valor en 'tiempo2': nan
Fila 2: valor en 'tiempo2': nan
Fila 3: valor en 'tiempo2': nan
Fila 4: valor en 'tiempo2': nan
Fila 5: valor en 'tiempo2': nan
Fila 6: valor en 'tiempo2': nan
Fila 7: valor en 'tiempo2': nan
Fila 8: valor en 'tiempo2': nan
Fila 9: valor en 'tiempo2': nan
Fila 10: valor en 'tiempo2': nan
Fila 11: valor en 'tiempo2': nan
Fila 12: valor en 'tiempo2': nan
Fila 13: valor en 'tiempo2': nan
Fila 14: valor en 'tiempo2': nan
Fila 15: valor en 'tiempo2': nan
Fila 16: valor en 'tiempo2': nan
Fila 17: valor en 'tiempo2': nan
Fila 18: valor en 'tiempo2': nan
Fila 19: valor en 'tiempo2': nan
Fila 20: valor en 'tiempo2': nan
Fila 21: valor en 'tiempo2': nan
Fila 22: valor en 'tiempo2': nan
Fila 23: valor en 'tiempo2': nan
Fila 24: valor en 'tiempo2': nan
Fila 25: valor en 'tiempo2': nan
Fila 26: valor en 'tiempo2': nan
Fila 27: valor en 'tiempo2': nan
Fila 28: valor en 'tiempo2': nan
Fila 29: valor en 'tiempo2': nan
Fila 30: valor en 't

In [237]:
# eliminacion de columnas 
redpiso_detalles= redpiso_detalles.drop(columns=["campo13","tiempo2","campo14","campo15","tiempo3","campo16","campo17","campo18","tiempo4",
"campo19"])

In [238]:
redpiso_detalles.shape

(2108, 36)

In [239]:
# Reemplazo de valores nulos por ceros y los otros por unos 
columnas_binario= ["amueblado","alarma","conserje","puerta_seguridad","acceso_minusvalido","cocina_equipada","aire_acondicionado",
                   "ascensor","piscina","trastero","Zonas verdes"]

redpiso_detalles[columnas_binario]= redpiso_detalles[columnas_binario].notnull().astype(int)


In [240]:
redpiso_detalles.head()

,título,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,cantidad_dormitorios,cantidad_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,aire_acondicionado,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,piscina,trastero,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,alarma,amueblado
0,Piso en venta en CALLE BENIMAMET San Cristobal...,106500,RP2842024125161,No Visitas 5857,02042024,64 m2,Dormitorios 2,Banos 1,NaN,NaN,NaN,NaN,Estado Perfecto Estado,Ano de construccion 1962,0,1,NaN,Agua caliente Individual,NaN,Tipo de suelo Tarima normal,0,NaN,Metros utiles 60 m2,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
1,Piso en venta en Usera Madrid Madrid,119000,RP782024130863,No Visitas 714,02102024,30 m2,Dormitorios 1,Banos 1,Armarios 1,NaN,NaN,Cal energetica,Estado Perfecto Estado,NaN,0,0,Orientacion Sur,Agua caliente Individual,NaN,NaN,1,NaN,Metros utiles 27 m2,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
2,Piso en venta en Buenavista Carabanchel Madrid...,120000,RP442024126007,No Visitas 3387,26042024,37 m2,Dormitorios 1,Banos 1,Armarios 1,Consumo 111 KW hm2 ano,Emisiones 18 Kg CO2m2 ano,Cal energetica,Estado Perfecto Estado,NaN,0,0,Orientacion Sur,NaN,NaN,NaN,0,NaN,Metros utiles 34 m2,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
3,Piso en venta en Buenavista Carabanchel Madrid...,120000,RP442024126427,No Visitas 2924,10052024,33 m2,Dormitorios 1,Banos 1,Armarios 1,Consumo 145 KW hm2 ano,Emisiones 24 Kg CO2m2 ano,Cal energetica,Estado Perfecto Estado,NaN,0,0,Orientacion Oeste,Agua caliente Individual,NaN,NaN,0,NaN,Metros utiles 30 m2,NaN,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
4,Piso en venta en CALLE BENIFERRI San Cristobal...,127500,RP2842024130186,No Visitas 485,12092024,58 m2,Dormitorios 3,Banos 1,NaN,NaN,NaN,NaN,Estado Perfecto Estado,NaN,1,1,NaN,Agua caliente Gas natural,Calefaccion Gas Natural,Tipo de suelo Terrazo,0,NaN,Metros utiles 50 m2,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0


-----------

### **_Aqui se termina el trabajo con pisosmadrid_detalles y ahora pasamos a la union de pisosmadrid_normal y pisosmadrid_detalles_** 

In [241]:
redpiso_detalles2= redpiso_detalles
redpiso_normal2= redpiso_normal

In [242]:
redpiso_detalles2.columns

Index(['título', 'precio', 'referencia_web', 'cantidad_visitas',
       'fecha_publicacion', 'espacio_metros_cuadrados', 'cantidad_dormitorios',
       'cantidad_banos', 'cantidad_armarios', 'consumo_energia',
       'emisiones_CO2', 'cal_energetica', 'condicino-estado', 'antiguedad',
       'ascensor', 'aire_acondicionado', 'orientacion', 'sistema_agua',
       'sistema_calefaccion', 'Tipo de suelo_fachada', 'cocina_equipada',
       'garajes', 'm_cuadrados_utiles', 'vista zona', 'piscina', 'trastero',
       'Comunidad/personas edificio', 'Zonas verdes', 'terrazas',
       'acceso_minusvalido', 'puerta_seguridad', 'conserje', 'plantas',
       'N_piso', 'alarma', 'amueblado'],
      dtype='object')

In [243]:
redpiso_normal2.columns      #A diferencia del notebook donde se hizo la el trabajo inicialmente aqui tengo la columna officeling_url 

Index(['precio', 'url_imagen', 'reservado/sin_reserva',
       'espacio_metros_cuadrados', 'cantidad_habitaciones', 'url_piso/id',
       'descripcion', 'contacto', 'officelink_url', 'nombre_oficina'],
      dtype='object')

In [244]:
redpiso_normal2.isna().sum()

precio                        0
url_imagen                  723
reservado/sin_reserva         0
espacio_metros_cuadrados      7
cantidad_habitaciones         5
url_piso/id                   0
descripcion                   0
contacto                      0
officelink_url                0
nombre_oficina                0
dtype: int64

In [245]:
redpiso_detalles2.shape

(2108, 36)

In [246]:
redpiso_normal2.shape

(2108, 10)

In [247]:
redpiso_detalles2["cantidad_dormitorios"] = redpiso_detalles2["cantidad_dormitorios"].fillna(
    redpiso_normal2["cantidad_habitaciones"].reindex(redpiso_normal2.index)
)

In [248]:
redpiso_detalles2["precio"] = redpiso_detalles2["precio"] .replace("A consultar",np.nan)
redpiso_normal2["precio"] = redpiso_normal2["precio"] .replace("A consultar",np.nan)

In [249]:
redpiso_detalles2.dtypes

título                         object
precio                         object
referencia_web                 object
cantidad_visitas               object
fecha_publicacion              object
espacio_metros_cuadrados       object
cantidad_dormitorios           object
cantidad_banos                 object
cantidad_armarios              object
consumo_energia                object
emisiones_CO2                  object
cal_energetica                 object
condicino-estado               object
antiguedad                     object
ascensor                        int32
aire_acondicionado              int32
orientacion                    object
sistema_agua                   object
sistema_calefaccion            object
Tipo de suelo_fachada          object
cocina_equipada                 int32
garajes                        object
m_cuadrados_utiles             object
vista zona                     object
piscina                         int32
trastero                        int32
Comunidad/pe

In [250]:
redpiso_detalles2["cantidad_dormitorios"]= redpiso_detalles2["cantidad_dormitorios"].astype(str)
redpiso_detalles2["cantidad_banos"]= redpiso_detalles2["cantidad_banos"].astype(str)
redpiso_detalles2["espacio_metros_cuadrados"]= redpiso_detalles2["espacio_metros_cuadrados"].astype(str) 
redpiso_detalles2["cantidad_armarios"]= redpiso_detalles2["cantidad_armarios"].astype(str)
redpiso_detalles2["cantidad_visitas"]= redpiso_detalles2["cantidad_visitas"].astype(str)

In [251]:
# Aqui creamos una funcion que recorrera la columna que especificamos quitanto el valor que nosotoros le digamos y sin distincion 
# alguna sobre si es mayuscula o minuscula.
def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bdormitorios\b', '', palabra)

redpiso_detalles2["cantidad_dormitorios"]= redpiso_detalles2["cantidad_dormitorios"].apply(eliminar_palabra)

# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bbano\b', '', palabra)

redpiso_detalles2["cantidad_banos"]= redpiso_detalles2["cantidad_banos"].apply(eliminar_palabra)

# --------------------------------------------------------------------------
def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bbanos\b', '', palabra)

redpiso_detalles2["cantidad_banos"]= redpiso_detalles2["cantidad_banos"].apply(eliminar_palabra)
# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bm2\b', '', palabra)

redpiso_detalles2["espacio_metros_cuadrados"]= redpiso_detalles2["espacio_metros_cuadrados"].apply(eliminar_palabra)

# -----------------------------------------------------------------------------
def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bArmarios\b', '', str(palabra))

redpiso_detalles2["cantidad_armarios"] = redpiso_detalles2["cantidad_armarios"].apply(eliminar_palabra)

# -------------------------------------------------------------------------------

def eliminar_no_visitas(texto):
    # Eliminar "No Visitas" y dejar solo el número que le sigue
    return re.sub(r'(?i)No Visitas\s*(\d+)', r'\1', str(texto))

# Aplicar la función a la columna "cantidad_visitas"
redpiso_detalles2["cantidad_visitas"] = redpiso_detalles2["cantidad_visitas"].apply(eliminar_no_visitas)

In [252]:
redpiso_detalles2["m_cuadrados_utiles"] = redpiso_detalles2["m_cuadrados_utiles"].str.replace("Metros utiles", "", case=False, regex= True)
redpiso_detalles2["m_cuadrados_utiles"] = redpiso_detalles2["m_cuadrados_utiles"].str.replace("m2", "", case=False, regex= True)

In [253]:
# datos = pd.read_csv("avanzadonuevo_datos_redpiso_detalles.csv") en estos reemplzamientos se encontraba basicmanete el redpisodetalle_tercertratamiento

# datos = pd.read_csv("avanzadonuevo_datos_redpiso_detalles.csv")

In [254]:
redpiso_detalles2.head(5)

,título,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,cantidad_dormitorios,cantidad_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,aire_acondicionado,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,piscina,trastero,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,alarma,amueblado
0,Piso en venta en CALLE BENIMAMET San Cristobal...,106500,RP2842024125161,5857,02042024,64,2,1,nan,NaN,NaN,NaN,Estado Perfecto Estado,Ano de construccion 1962,0,1,NaN,Agua caliente Individual,NaN,Tipo de suelo Tarima normal,0,NaN,60,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
1,Piso en venta en Usera Madrid Madrid,119000,RP782024130863,714,02102024,30,1,1,1,NaN,NaN,Cal energetica,Estado Perfecto Estado,NaN,0,0,Orientacion Sur,Agua caliente Individual,NaN,NaN,1,NaN,27,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
2,Piso en venta en Buenavista Carabanchel Madrid...,120000,RP442024126007,3387,26042024,37,1,1,1,Consumo 111 KW hm2 ano,Emisiones 18 Kg CO2m2 ano,Cal energetica,Estado Perfecto Estado,NaN,0,0,Orientacion Sur,NaN,NaN,NaN,0,NaN,34,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
3,Piso en venta en Buenavista Carabanchel Madrid...,120000,RP442024126427,2924,10052024,33,1,1,1,Consumo 145 KW hm2 ano,Emisiones 24 Kg CO2m2 ano,Cal energetica,Estado Perfecto Estado,NaN,0,0,Orientacion Oeste,Agua caliente Individual,NaN,NaN,0,NaN,30,NaN,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
4,Piso en venta en CALLE BENIFERRI San Cristobal...,127500,RP2842024130186,485,12092024,58,3,1,nan,NaN,NaN,NaN,Estado Perfecto Estado,NaN,1,1,NaN,Agua caliente Gas natural,Calefaccion Gas Natural,Tipo de suelo Terrazo,0,NaN,50,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0


In [255]:
# redpiso_detalles['fecha_publicacion'] = pd.to_datetime(redpiso_detalles['fecha_publicacion'].astype(str), format='%d%m%Y')

In [256]:
col_tiponuevo= ["precio","espacio_metros_cuadrados",
                "cantidad_dormitorios","cantidad_banos",
                "cantidad_armarios"]

redpiso_detalles2 = redpiso_detalles2.replace("nan",0)
redpiso_detalles2[col_tiponuevo]=redpiso_detalles2[col_tiponuevo].fillna(0)

redpiso_detalles2[col_tiponuevo] = redpiso_detalles2[col_tiponuevo].astype("int64")

redpiso_detalles2[col_tiponuevo]=redpiso_detalles2[col_tiponuevo].replace(0,np.nan)


In [257]:
redpiso_detalles2.dtypes

título                          object
precio                         float64
referencia_web                  object
cantidad_visitas                object
fecha_publicacion               object
espacio_metros_cuadrados       float64
cantidad_dormitorios           float64
cantidad_banos                 float64
cantidad_armarios              float64
consumo_energia                 object
emisiones_CO2                   object
cal_energetica                  object
condicino-estado                object
antiguedad                      object
ascensor                         int32
aire_acondicionado               int32
orientacion                     object
sistema_agua                    object
sistema_calefaccion             object
Tipo de suelo_fachada           object
cocina_equipada                  int32
garajes                         object
m_cuadrados_utiles              object
vista zona                      object
piscina                          int32
trastero                 

In [258]:
redpiso_detalles2["referencia_web"].describe()

count               2108
unique              2107
top       RP302023121517
freq                   2
Name: referencia_web, dtype: object

In [259]:
redpiso_normal2["nombre_oficina"].describe()

count                     2108
unique                     115
top       REDPISO ARTURO SORIA
freq                        48
Name: nombre_oficina, dtype: object

In [260]:
redpiso_normal2["reservado/sin_reserva"].value_counts()

reservado/sin_reserva
0    1748
1     360
Name: count, dtype: int64

In [261]:
redpiso_detalles2

,título,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,cantidad_dormitorios,cantidad_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,aire_acondicionado,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,piscina,trastero,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,alarma,amueblado
0,Piso en venta en CALLE BENIMAMET San Cristobal...,106500.0,RP2842024125161,5857,02042024,64.0,2.0,1.0,NaN,NaN,NaN,NaN,Estado Perfecto Estado,Ano de construccion 1962,0,1,NaN,Agua caliente Individual,NaN,Tipo de suelo Tarima normal,0,NaN,60,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
1,Piso en venta en Usera Madrid Madrid,119000.0,RP782024130863,714,02102024,30.0,1.0,1.0,1.0,NaN,NaN,Cal energetica,Estado Perfecto Estado,NaN,0,0,Orientacion Sur,Agua caliente Individual,NaN,NaN,1,NaN,27,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
2,Piso en venta en Buenavista Carabanchel Madrid...,120000.0,RP442024126007,3387,26042024,37.0,1.0,1.0,1.0,Consumo 111 KW hm2 ano,Emisiones 18 Kg CO2m2 ano,Cal energetica,Estado Perfecto Estado,NaN,0,0,Orientacion Sur,NaN,NaN,NaN,0,NaN,34,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
3,Piso en venta en Buenavista Carabanchel Madrid...,120000.0,RP442024126427,2924,10052024,33.0,1.0,1.0,1.0,Consumo 145 KW hm2 ano,Emisiones 24 Kg CO2m2 ano,Cal energetica,Estado Perfecto Estado,NaN,0,0,Orientacion Oeste,Agua caliente Individual,NaN,NaN,0,NaN,30,NaN,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
4,Piso en venta en CALLE BENIFERRI San Cristobal...,127500.0,RP2842024130186,485,12092024,58.0,3.0,1.0,NaN,NaN,NaN,NaN,Estado Perfecto Estado,NaN,1,1,NaN,Agua caliente Gas natural,Calefaccion Gas Natural,Tipo de suelo Terrazo,0,NaN,50,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,Piso en venta en CALLE ORIO 4 Los Angeles Vill...,NaN,RP332024126817,240,22052024,NaN,3.0,1.0,NaN,NaN,NaN,Cal energetica,Estado A reformar,Antiguedad Mas de 50 anos,0,0,Orientacion Norte,Agua caliente Gas natural,Calefaccion Gas Natural,Tipo de suelo Baldosa,0,NaN,NaN,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
2104,Piso en venta en CALLE CANCION DEL OLVIDO 12 L...,NaN,RP332024127980,218,24062024,90.0,3.0,1.0,NaN,NaN,NaN,Cal energetica,NaN,Ano de construccion 1960,1,0,NaN,Agua caliente Individual,NaN,Tipo de fachada Otros materiales,0,NaN,71,NaN,0,0,Comunidad 6500,0,NaN,0,0,0,NaN,NaN,0,0
2105,Piso en venta en CALLE LA ALEGRIA DE LA HUERTA...,NaN,RP332024128702,172,15072024,79.0,3.0,1.0,NaN,NaN,NaN,Cal energetica,Estado Perfecto Estado,NaN,1,1,NaN,Agua caliente Gas natural,Calefaccion Electrica,NaN,0,NaN,63,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
2106,Piso en venta en CALLE Ochagavia Valdezarza Mo...,NaN,RP402024127484,845,11062024,76.0,3.0,1.0,2.0,Consumo 172 KW hm2 ano,Emisiones 36 Kg CO2m2 ano,Cal energetica,NaN,NaN,1,0,Orientacion Oeste,NaN,NaN,NaN,0,NaN,69,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0


In [262]:
redpiso_detalles2["precio"].isna().sum()

640

In [263]:
redpiso_normal2["precio"].isna().sum()

640

In [264]:
redpiso_detalles2=redpiso_detalles2.dropna(subset=["precio"])
redpiso_normal2= redpiso_normal2.dropna(subset=["precio"])

In [265]:
redpiso_detalles2.shape

(1468, 36)

In [266]:
redpiso_normal2.shape

(1468, 10)

In [267]:
nuevos_nombres={
    "descripcion":"titulo",
    "cantidad_habitaciones":"cantidad_dormitorios"
}
redpiso_normal2= redpiso_normal2.rename(columns=nuevos_nombres)

In [268]:
redpiso_detalles2["espacio_metros_cuadrados"]=redpiso_detalles2["espacio_metros_cuadrados"].fillna(0) #Tenia dos nulos 
redpiso_detalles2["cantidad_dormitorios"]=redpiso_detalles2["cantidad_dormitorios"].fillna(0) #Tenia dos nulos

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\47546438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  redpiso_detalles2["espacio_metros_cuadrados"]=redpiso_detalles2["espacio_metros_cuadrados"].fillna(0) #Tenia dos nulos
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\47546438.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  redpiso_detalles2["cantidad_dormitorios"]=redpiso_detalles2["cantidad_dormitorios"].fillna(0) #Tenia dos nulos


In [269]:
redpiso_detalles2["precio"]=redpiso_detalles2["precio"].astype("int")
redpiso_detalles2["cantidad_dormitorios"]=redpiso_detalles2["cantidad_dormitorios"].astype("int")
redpiso_detalles2["espacio_metros_cuadrados"]=redpiso_detalles2["espacio_metros_cuadrados"].astype("int")

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\910797546.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  redpiso_detalles2["precio"]=redpiso_detalles2["precio"].astype("int")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\910797546.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  redpiso_detalles2["cantidad_dormitorios"]=redpiso_detalles2["cantidad_dormitorios"].astype("int")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\910797546.py:3: SettingWithCopyWarning: 
A value is trying to be se

In [270]:
redpiso_detalles2.head(2
                  )

,título,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,cantidad_dormitorios,cantidad_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,aire_acondicionado,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,piscina,trastero,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,alarma,amueblado
0,Piso en venta en CALLE BENIMAMET San Cristobal...,106500,RP2842024125161,5857,02042024,64,2,1.0,NaN,NaN,NaN,NaN,Estado Perfecto Estado,Ano de construccion 1962,0,1,NaN,Agua caliente Individual,NaN,Tipo de suelo Tarima normal,0,NaN,60,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
1,Piso en venta en Usera Madrid Madrid,119000,RP782024130863,714,02102024,30,1,1.0,1.0,NaN,NaN,Cal energetica,Estado Perfecto Estado,NaN,0,0,Orientacion Sur,Agua caliente Individual,NaN,NaN,1,NaN,27,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0


In [271]:
redpiso_normal2.head(5)

,precio,url_imagen,reservado/sin_reserva,espacio_metros_cuadrados,cantidad_dormitorios,url_piso/id,titulo,contacto,officelink_url,nombre_oficina
0,106500,NaN,1,64,2,httpswwwredpisoesinmueblepisoenventaencalleben...,Piso en venta en CALLE BENIMAMET San Cristóbal...,919208536,httpswwwredpisoesoficinainmobiliariasancristob...,REDPISO SAN CRISTOBAL
1,119000,NaN,0,30,1,httpswwwredpisoesinmueblepisoenventaenuseramad...,Piso en venta en Usera Madrid Madrid,914282010,httpswwwredpisoesoficinainmobiliariauseraorcas...,REDPISO USERAORCASITAS
2,120000,NaN,1,37,1,httpswwwredpisoesinmueblepisoenventaenbuenavis...,Piso en venta en Buenavista Carabanchel Madrid...,915996359,httpswwwredpisoesoficinainmobiliariapaucaraban...,REDPISO PAU CARABANCHEL
3,120000,NaN,1,33,1,httpswwwredpisoesinmueblepisoenventaenbuenavis...,Piso en venta en Buenavista Carabanchel Madrid...,915996359,httpswwwredpisoesoficinainmobiliariapaucaraban...,REDPISO PAU CARABANCHEL
4,127500,httpswwwredpisoesuploadpropertiesthumbsimg_951...,1,58,3,httpswwwredpisoesinmueblepisoenventaencalleben...,Piso en venta en CALLE BENIFERRI San Cristóbal...,919208536,httpswwwredpisoesoficinainmobiliariasancristob...,REDPISO SAN CRISTOBAL


In [272]:
redpiso_normal2.dtypes

precio                      object
url_imagen                  object
reservado/sin_reserva        int64
espacio_metros_cuadrados    object
cantidad_dormitorios        object
url_piso/id                 object
titulo                      object
contacto                    object
officelink_url              object
nombre_oficina              object
dtype: object

In [273]:
redpiso_normal2["espacio_metros_cuadrados"]=redpiso_normal2["espacio_metros_cuadrados"].fillna(0) #Tenia 4 nulos 
redpiso_normal2["cantidad_dormitorios"]=redpiso_normal2["cantidad_dormitorios"].fillna(0) #Tenia 3 nulos

In [274]:
redpiso_normal2["precio"]=redpiso_normal2["precio"].astype("int")
redpiso_normal2["espacio_metros_cuadrados"]=redpiso_normal2["espacio_metros_cuadrados"].astype("int")



In [275]:
redpiso_detalles2.dtypes

título                          object
precio                           int32
referencia_web                  object
cantidad_visitas                object
fecha_publicacion               object
espacio_metros_cuadrados         int32
cantidad_dormitorios             int32
cantidad_banos                 float64
cantidad_armarios              float64
consumo_energia                 object
emisiones_CO2                   object
cal_energetica                  object
condicino-estado                object
antiguedad                      object
ascensor                         int32
aire_acondicionado               int32
orientacion                     object
sistema_agua                    object
sistema_calefaccion             object
Tipo de suelo_fachada           object
cocina_equipada                  int32
garajes                         object
m_cuadrados_utiles              object
vista zona                      object
piscina                          int32
trastero                 

In [276]:
redpiso_detalles2 = redpiso_detalles2.rename(columns={"título":"titulo"})

In [277]:
df_merge= pd.merge(redpiso_detalles2,redpiso_normal2,how="outer",on=["precio","titulo","espacio_metros_cuadrados",],indicator=True)

In [278]:
df_merge.shape

(2154, 44)

In [279]:
df_merge.duplicated().sum()

0

In [280]:
df_merge.isna().sum()

titulo                            0
precio                            0
referencia_web                  680
cantidad_visitas                680
fecha_publicacion               680
espacio_metros_cuadrados          0
cantidad_dormitorios_x          680
cantidad_banos                  687
cantidad_armarios              1566
consumo_energia                1894
emisiones_CO2                  1893
cal_energetica                 1158
condicino-estado                899
antiguedad                     1823
ascensor                        680
aire_acondicionado              680
orientacion                    1395
sistema_agua                   1460
sistema_calefaccion            1573
Tipo de suelo_fachada          1640
cocina_equipada                 680
garajes                        1848
m_cuadrados_utiles             1062
vista zona                     1022
piscina                         680
trastero                        680
Comunidad/personas edificio    2123
Zonas verdes                

In [281]:
df_merge= df_merge.drop(columns=["_merge","cantidad_dormitorios_y"])

In [282]:
df_merge.isna().sum()

titulo                            0
precio                            0
referencia_web                  680
cantidad_visitas                680
fecha_publicacion               680
espacio_metros_cuadrados          0
cantidad_dormitorios_x          680
cantidad_banos                  687
cantidad_armarios              1566
consumo_energia                1894
emisiones_CO2                  1893
cal_energetica                 1158
condicino-estado                899
antiguedad                     1823
ascensor                        680
aire_acondicionado              680
orientacion                    1395
sistema_agua                   1460
sistema_calefaccion            1573
Tipo de suelo_fachada          1640
cocina_equipada                 680
garajes                        1848
m_cuadrados_utiles             1062
vista zona                     1022
piscina                         680
trastero                        680
Comunidad/personas edificio    2123
Zonas verdes                

In [283]:
redpiso_unido= df_merge

In [284]:
# Creacion de nuevas columnas donde metere los valores de de distrito, barrios, codigos postal y el codigo por distrito y barrio.

nuevas_columnas= ["barrios","cod_barrio","cod_barrio_2","distrito","cod_distrito","codigo_Postal"]

redpiso_unido[nuevas_columnas]=np.nan

In [285]:
redpiso_unido

,titulo,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,cantidad_dormitorios_x,cantidad_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,aire_acondicionado,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,piscina,trastero,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,alarma,amueblado,url_imagen,reservado/sin_reserva,url_piso/id,contacto,officelink_url,nombre_oficina,barrios,cod_barrio,cod_barrio_2,distrito,cod_distrito,codigo_Postal
0,Apartamento en venta en CALLE Picos de Europa ...,54000,RP022023115210,1492,10062023,25,1.0,1.0,NaN,Consumo 393 KW hm2 ano,Emisiones 76 Kg CO2m2 ano,Cal energetica,Estado A reformar,Antiguedad Mas de 50 anos,0.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Baldosa,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,httpswwwredpisoesuploadpropertiesthumbsimg_092...,0.0,httpswwwredpisoesinmuebleapartamentoenventaenc...,914786966,httpswwwredpisoesoficinainmobiliariavallecasal...,REDPISO VALLECAS ALBUFERA NUEVA NUMANCIA,NaN,NaN,NaN,NaN,NaN,NaN
1,Piso en venta en CALLE QUIJADA DE PANDIELLOS S...,67000,RP152024125550,722,15042024,48,1.0,1.0,1.0,NaN,NaN,NaN,Estado Depende del precio,NaN,0.0,1.0,NaN,Agua caliente Gas natural,Calefaccion Gas Natural,NaN,1.0,NaN,38,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallequi...,914349403,httpswwwredpisoesoficinainmobiliariavallecaspu...,REDPISO VALLECASPUENTE VALLECAS,NaN,NaN,NaN,NaN,NaN,NaN
2,Piso en venta en CALLE ENRIQUE VELASCO Nueva N...,69000,RP02202294300,2190,17012022,35,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Ano de construccion 1940,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencalleenr...,914786966,httpswwwredpisoesoficinainmobiliariavallecasal...,REDPISO VALLECAS ALBUFERA NUEVA NUMANCIA,NaN,NaN,NaN,NaN,NaN,NaN
3,Piso en venta en CALLE LAVAPIES Embajadores Ce...,69000,RP1222023116499,2285,11072023,28,1.0,1.0,NaN,NaN,NaN,Cal energetica,Estado A reformar,NaN,1.0,0.0,Orientacion Oeste,NaN,NaN,Tipo de fachada Mono Capa,0.0,NaN,27,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallelav...,911921007,httpswwwredpisoesoficinainmobiliarialavapiesem...,REDPISO LAVAPIESEMBAJADORES,NaN,NaN,NaN,NaN,NaN,NaN
4,Piso en venta en CALLE VIZCONDE DE ARLESSON En...,81500,RP172024123241,713,05022024,30,2.0,1.0,NaN,NaN,NaN,NaN,Estado A reformar,Ano de construccion 1960,1.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Gres,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,1.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2149,Piso en venta en PASEO EDUARDO DATO Almagro Ch...,3850000,NaN,NaN,NaN,378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,httpswwwredpisoesuploadpropertiesthumbsimg_522...,0.0,httpswwwredpisoesinmueblepisoenventaenpaseoedu...,910378491,httpswwwredpisoesoficinainmobiliariachamberial...,REDPISO CHAMBERÍ ALMAGRO,NaN,NaN,NaN,NaN,NaN,NaN
2150,Piso en venta en AVENIDA MENENDEZ PELAYO Ibiza...,4000000,RP962024130172,150,12092024,300,5.0,5.0,NaN,NaN,NaN,Cal energetica,Estado Perfecto Estado,NaN,1.0,0.0,Orientacion Sur,Agua caliente Individual,NaN,NaN,1.0,NaN,260,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2151,Piso en venta en AVENIDA MENÉNDEZ PELAYO Ibiza...,4000000,NaN,NaN,NaN,300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [286]:
redpiso_unido2=redpiso_unido

#### **_Agregacion de barrios y distritos con sus codigos respectivos/ Redpiso._**

In [287]:
# Aqui hacemos la normalizacion de las columnas para despues hacer una mejor filtracion de nombres de distritos y barrios. 

def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

def clasificar_distrito_y_barrio_por_columnas(df_sin_duplicados, dbcp_ordenado):
    # Crear las listas de barrios y distritos desde el dataframe de referencia
    distritos = dbcp_ordenado["nombre_distrito"].apply(lambda x: normalize_string(x)).tolist()
    barrios = dbcp_ordenado["nombre_barrio"].apply(lambda x: normalize_string(x)).tolist()

    # Recorrer las filas del dataframe
    for idx, fila in df_sin_duplicados.iterrows():
        # Recorrer las tres columnas en orden de prioridad: descripcion, descripcion_extendida, url_website
        for columna in ['titulo', 'nombre_oficina']:
            valor = fila[columna]
            
            # Verificamos que el valor no sea nulo
            if pd.notna(valor):
                valor_normalizado = normalize_string(valor)
                
                # Verificamos si el valor normalizado contiene el nombre de un distrito o barrio
                for distrito, barrio in zip(distritos, barrios):
                    if distrito in valor_normalizado and pd.isna(fila["distrito"]):
                        df_sin_duplicados.at[idx, "distrito"] = distrito  # Asignar el nombre del distrito a la columna "distrito"
                    if barrio in valor_normalizado and pd.isna(fila["barrios"]):
                        df_sin_duplicados.at[idx, "barrios"] = barrio  # Asignar el nombre del barrio a la columna "barrio"

    return df_sin_duplicados

# Llamar a la función pasándole el dataframe y el dataframe de referencia
df_sin_duplicados = clasificar_distrito_y_barrio_por_columnas(redpiso_unido2, dbcp_ordenado)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\921010083.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'puente de vallecas' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sin_duplicados.at[idx, "distrito"] = distrito  # Asignar el nombre del distrito a la columna "distrito"
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\921010083.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'numancia' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sin_duplicados.at[idx, "barrios"] = barrio  # Asignar el nombre del barrio a la columna "barrio"


In [288]:
redpiso_unido2.head(5)

,titulo,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,cantidad_dormitorios_x,cantidad_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,aire_acondicionado,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,piscina,trastero,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,alarma,amueblado,url_imagen,reservado/sin_reserva,url_piso/id,contacto,officelink_url,nombre_oficina,barrios,cod_barrio,cod_barrio_2,distrito,cod_distrito,codigo_Postal
0,Apartamento en venta en CALLE Picos de Europa ...,54000,RP022023115210,1492,10062023,25,1.0,1.0,NaN,Consumo 393 KW hm2 ano,Emisiones 76 Kg CO2m2 ano,Cal energetica,Estado A reformar,Antiguedad Mas de 50 anos,0.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Baldosa,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,httpswwwredpisoesuploadpropertiesthumbsimg_092...,0.0,httpswwwredpisoesinmuebleapartamentoenventaenc...,914786966,httpswwwredpisoesoficinainmobiliariavallecasal...,REDPISO VALLECAS ALBUFERA NUEVA NUMANCIA,numancia,NaN,NaN,puente de vallecas,NaN,NaN
1,Piso en venta en CALLE QUIJADA DE PANDIELLOS S...,67000,RP152024125550,722,15042024,48,1.0,1.0,1.0,NaN,NaN,NaN,Estado Depende del precio,NaN,0.0,1.0,NaN,Agua caliente Gas natural,Calefaccion Gas Natural,NaN,1.0,NaN,38,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallequi...,914349403,httpswwwredpisoesoficinainmobiliariavallecaspu...,REDPISO VALLECASPUENTE VALLECAS,san diego,NaN,NaN,puente de vallecas,NaN,NaN
2,Piso en venta en CALLE ENRIQUE VELASCO Nueva N...,69000,RP02202294300,2190,17012022,35,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Ano de construccion 1940,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencalleenr...,914786966,httpswwwredpisoesoficinainmobiliariavallecasal...,REDPISO VALLECAS ALBUFERA NUEVA NUMANCIA,numancia,NaN,NaN,puente de vallecas,NaN,NaN
3,Piso en venta en CALLE LAVAPIES Embajadores Ce...,69000,RP1222023116499,2285,11072023,28,1.0,1.0,NaN,NaN,NaN,Cal energetica,Estado A reformar,NaN,1.0,0.0,Orientacion Oeste,NaN,NaN,Tipo de fachada Mono Capa,0.0,NaN,27,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallelav...,911921007,httpswwwredpisoesoficinainmobiliarialavapiesem...,REDPISO LAVAPIESEMBAJADORES,embajadores,NaN,NaN,centro,NaN,NaN
4,Piso en venta en CALLE VIZCONDE DE ARLESSON En...,81500,RP172024123241,713,05022024,30,2.0,1.0,NaN,NaN,NaN,NaN,Estado A reformar,Ano de construccion 1960,1.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Gres,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,1.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,entrevias,NaN,NaN,puente de vallecas,NaN,NaN


In [289]:
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

def rellenar_distritos(redpiso_unido2, dbcp_ordenado):
    # Crear un diccionario de barrios a distritos normalizado
    barrio_a_distrito = {normalize_string(barrio): distrito for barrio, distrito in zip(dbcp_ordenado["nombre_barrio"], dbcp_ordenado["nombre_distrito"])}

    for idx, fila in redpiso_unido2.iterrows():
        # Verificamos si el valor de distrito está vacío y si hay un valor en barrios
        if pd.isna(fila["distrito"]) and not pd.isna(fila["barrios"]):
            barrio_normalizado = normalize_string(fila["barrios"])  # Normalizar el valor del barrio
            distrito = barrio_a_distrito.get(barrio_normalizado)  # Buscar el distrito usando el barrio normalizado
            if distrito:
                redpiso_unido2.at[idx, "distrito"] = distrito  # Asignar el distrito correspondiente

# Llamar a la función pasándole el dataframe y el dataframe de referencia
rellenar_distritos(redpiso_unido2=redpiso_unido2, dbcp_ordenado=dbcp_ordenado)


In [290]:
redpiso_unido2.head(5)

,titulo,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,cantidad_dormitorios_x,cantidad_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,aire_acondicionado,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,piscina,trastero,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,alarma,amueblado,url_imagen,reservado/sin_reserva,url_piso/id,contacto,officelink_url,nombre_oficina,barrios,cod_barrio,cod_barrio_2,distrito,cod_distrito,codigo_Postal
0,Apartamento en venta en CALLE Picos de Europa ...,54000,RP022023115210,1492,10062023,25,1.0,1.0,NaN,Consumo 393 KW hm2 ano,Emisiones 76 Kg CO2m2 ano,Cal energetica,Estado A reformar,Antiguedad Mas de 50 anos,0.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Baldosa,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,httpswwwredpisoesuploadpropertiesthumbsimg_092...,0.0,httpswwwredpisoesinmuebleapartamentoenventaenc...,914786966,httpswwwredpisoesoficinainmobiliariavallecasal...,REDPISO VALLECAS ALBUFERA NUEVA NUMANCIA,numancia,NaN,NaN,puente de vallecas,NaN,NaN
1,Piso en venta en CALLE QUIJADA DE PANDIELLOS S...,67000,RP152024125550,722,15042024,48,1.0,1.0,1.0,NaN,NaN,NaN,Estado Depende del precio,NaN,0.0,1.0,NaN,Agua caliente Gas natural,Calefaccion Gas Natural,NaN,1.0,NaN,38,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallequi...,914349403,httpswwwredpisoesoficinainmobiliariavallecaspu...,REDPISO VALLECASPUENTE VALLECAS,san diego,NaN,NaN,puente de vallecas,NaN,NaN
2,Piso en venta en CALLE ENRIQUE VELASCO Nueva N...,69000,RP02202294300,2190,17012022,35,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Ano de construccion 1940,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencalleenr...,914786966,httpswwwredpisoesoficinainmobiliariavallecasal...,REDPISO VALLECAS ALBUFERA NUEVA NUMANCIA,numancia,NaN,NaN,puente de vallecas,NaN,NaN
3,Piso en venta en CALLE LAVAPIES Embajadores Ce...,69000,RP1222023116499,2285,11072023,28,1.0,1.0,NaN,NaN,NaN,Cal energetica,Estado A reformar,NaN,1.0,0.0,Orientacion Oeste,NaN,NaN,Tipo de fachada Mono Capa,0.0,NaN,27,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallelav...,911921007,httpswwwredpisoesoficinainmobiliarialavapiesem...,REDPISO LAVAPIESEMBAJADORES,embajadores,NaN,NaN,centro,NaN,NaN
4,Piso en venta en CALLE VIZCONDE DE ARLESSON En...,81500,RP172024123241,713,05022024,30,2.0,1.0,NaN,NaN,NaN,NaN,Estado A reformar,Ano de construccion 1960,1.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Gres,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,1.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,entrevias,NaN,NaN,puente de vallecas,NaN,NaN


In [291]:

diccionario_ditrito_barrio= {
    "Centro": ["Malasaña", "La Latina", "Chueca","malasana"],
    "Moratalaz": ["La Elipa"],
    "Moncloa-Aravaca": ["Puerta de Hierro", "Príncipe Pío"],
    "Fuencarral-El Pardo": ["Buena Vista", "Ventilla", "Fuencarral"],
    "Chamartín": ["Bernabéu"],
    "Ciudad Lineal": ["Arturo Soria"],
    "San Blas-Canillejas": ["Las Mercedes","las mercedes"],
    "Villaverde": ["Villaverde Alto", "Villaverde Bajo"],
    "Carabanchel": ["Pau Carabanchel"],
}

def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar acentos
        s = s.lower()  # Convertir a minúsculas
    return s

# Función para asignar barrios y distritos recorriendo solo el diccionario
def asignar_barrios_y_distritos(df, diccionario, columnas_busqueda):
    # Normalizar los barrios en el diccionario
    diccionario_normalizado = {
        normalize_string(barrio): distrito for distrito, barrios in diccionario.items() for barrio in barrios
    }
    
    # Asegurar que las columnas "barrios" y "distritos" existan en el DataFrame
    if "barrios" not in df.columns:
        df["barrios"] = pd.NA
    if "distrito" not in df.columns:
        df["distrito"] = pd.NA

    # Iterar sobre el diccionario de barrios y distritos
    for barrio_normalizado, distrito in diccionario_normalizado.items():
        # Buscar coincidencias en las columnas especificadas
        for columna in columnas_busqueda:
            # Generar una máscara para las coincidencias
            mask = df[columna].str.contains(barrio_normalizado, na=False, case=False)

            # Asignar valores donde haya coincidencias y columnas vacías
            df.loc[mask & df['barrios'].isna(), 'barrios'] = barrio_normalizado
            df.loc[mask & df['distrito'].isna(), 'distrito'] = distrito

    return df

# Llamar a la función
columnas_busqueda = ["barrios", "distrito"]  # Ajusta estas columnas según tu DataFrame
redpiso_unido2 = asignar_barrios_y_distritos(redpiso_unido2, diccionario_ditrito_barrio, columnas_busqueda)

# Mostrar resultados
redpiso_unido2

,titulo,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,cantidad_dormitorios_x,cantidad_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,aire_acondicionado,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,piscina,trastero,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,alarma,amueblado,url_imagen,reservado/sin_reserva,url_piso/id,contacto,officelink_url,nombre_oficina,barrios,cod_barrio,cod_barrio_2,distrito,cod_distrito,codigo_Postal
0,Apartamento en venta en CALLE Picos de Europa ...,54000,RP022023115210,1492,10062023,25,1.0,1.0,NaN,Consumo 393 KW hm2 ano,Emisiones 76 Kg CO2m2 ano,Cal energetica,Estado A reformar,Antiguedad Mas de 50 anos,0.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Baldosa,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,httpswwwredpisoesuploadpropertiesthumbsimg_092...,0.0,httpswwwredpisoesinmuebleapartamentoenventaenc...,914786966,httpswwwredpisoesoficinainmobiliariavallecasal...,REDPISO VALLECAS ALBUFERA NUEVA NUMANCIA,numancia,NaN,NaN,puente de vallecas,NaN,NaN
1,Piso en venta en CALLE QUIJADA DE PANDIELLOS S...,67000,RP152024125550,722,15042024,48,1.0,1.0,1.0,NaN,NaN,NaN,Estado Depende del precio,NaN,0.0,1.0,NaN,Agua caliente Gas natural,Calefaccion Gas Natural,NaN,1.0,NaN,38,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallequi...,914349403,httpswwwredpisoesoficinainmobiliariavallecaspu...,REDPISO VALLECASPUENTE VALLECAS,san diego,NaN,NaN,puente de vallecas,NaN,NaN
2,Piso en venta en CALLE ENRIQUE VELASCO Nueva N...,69000,RP02202294300,2190,17012022,35,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Ano de construccion 1940,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencalleenr...,914786966,httpswwwredpisoesoficinainmobiliariavallecasal...,REDPISO VALLECAS ALBUFERA NUEVA NUMANCIA,numancia,NaN,NaN,puente de vallecas,NaN,NaN
3,Piso en venta en CALLE LAVAPIES Embajadores Ce...,69000,RP1222023116499,2285,11072023,28,1.0,1.0,NaN,NaN,NaN,Cal energetica,Estado A reformar,NaN,1.0,0.0,Orientacion Oeste,NaN,NaN,Tipo de fachada Mono Capa,0.0,NaN,27,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallelav...,911921007,httpswwwredpisoesoficinainmobiliarialavapiesem...,REDPISO LAVAPIESEMBAJADORES,embajadores,NaN,NaN,centro,NaN,NaN
4,Piso en venta en CALLE VIZCONDE DE ARLESSON En...,81500,RP172024123241,713,05022024,30,2.0,1.0,NaN,NaN,NaN,NaN,Estado A reformar,Ano de construccion 1960,1.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Gres,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,1.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,entrevias,NaN,NaN,puente de vallecas,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2149,Piso en venta en PASEO EDUARDO DATO Almagro Ch...,3850000,NaN,NaN,NaN,378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,httpswwwredpisoesuploadpropertiesthumbsimg_522...,0.0,httpswwwredpisoesinmueblepisoenventaenpaseoedu...,910378491,httpswwwredpisoesoficinainmobiliariachamberial...,REDPISO CHAMBERÍ ALMAGRO,almagro,NaN,NaN,chamberi,NaN,NaN
2150,Piso en venta en AVENIDA MENENDEZ PELAYO Ibiza...,4000000,RP962024130172,150,12092024,300,5.0,5.0,NaN,NaN,NaN,Cal energetica,Estado Perfecto Estado,NaN,1.0,0.0,Orientacion Sur,Agua caliente Individual,NaN,NaN,1.0,NaN,260,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,ibiza,NaN,NaN,retiro,NaN,NaN
2151,Piso en venta en AVENIDA MENÉNDEZ PELAYO Ibiza...,4000000,NaN,NaN,NaN,300,NaN,NaN

In [292]:
def normalize_string(s):
    """Normaliza una cadena eliminando acentos y convirtiendo a minúsculas."""
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer caracteres con acento
        s = ''.join(c for c in s if not unicodedata.combining(c))  # Eliminar acentos
        s = s.lower()  # Convertir a minúsculas
    return s

def asignar_codigos(df_sin_duplicados, dbcp_ordenado):
    """
    Asigna los valores de cod_distrito, cod_barrio, cod_barrio_2 y codigo_postal al DataFrame original
    basándose en la correspondencia con el DataFrame de referencia.
    """
    # Normalizar los nombres en el DataFrame de referencia
    dbcp_ordenado['nombre_distrito_normalizado'] = dbcp_ordenado['nombre_distrito'].apply(normalize_string)
    dbcp_ordenado['nombre_barrio_normalizado'] = dbcp_ordenado['nombre_barrio'].apply(normalize_string)

    # Normalizar las columnas 'distrito' y 'barrios' en el DataFrame original
    df_sin_duplicados['distrito_normalizado'] = df_sin_duplicados['distrito'].apply(normalize_string)
    df_sin_duplicados['barrios_normalizado'] = df_sin_duplicados['barrios'].apply(normalize_string)

    # Realizar asignaciones basadas en coincidencias
    for idx, fila in df_sin_duplicados.iterrows():
        # Verificar coincidencia por distrito
        if pd.notna(fila['distrito_normalizado']):
            coincidencias_distrito = dbcp_ordenado[
                dbcp_ordenado['nombre_distrito_normalizado'] == fila['distrito_normalizado']
            ]
            if not coincidencias_distrito.empty:
                df_sin_duplicados.at[idx, 'cod_distrito'] = coincidencias_distrito['cod_distrito'].iloc[0]

        # Verificar coincidencia por barrio
        if pd.notna(fila['barrios_normalizado']):
            coincidencias_barrio = dbcp_ordenado[
                dbcp_ordenado['nombre_barrio_normalizado'] == fila['barrios_normalizado']
            ]
            if not coincidencias_barrio.empty:
                df_sin_duplicados.at[idx, 'cod_barrio'] = coincidencias_barrio['cod_barrio'].iloc[0]
                df_sin_duplicados.at[idx, 'cod_barrio_2'] = coincidencias_barrio['cod_barrio_2'].iloc[0]
                df_sin_duplicados.at[idx, 'codigo_Postal'] = coincidencias_barrio['codigo_Postal'].iloc[0]

    # Eliminar columnas normalizadas después de asignaciones
    df_sin_duplicados.drop(columns=['distrito_normalizado', 'barrios_normalizado'], inplace=True)
    dbcp_ordenado.drop(columns=['nombre_distrito_normalizado', 'nombre_barrio_normalizado'], inplace=True)

    return df_sin_duplicados

# Llamar a la función con los DataFrames correspondientes
redpiso_unido2 = asignar_codigos(redpiso_unido2, dbcp_ordenado)

In [293]:
redpiso_unido2.head()

,titulo,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,cantidad_dormitorios_x,cantidad_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,aire_acondicionado,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,piscina,trastero,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,alarma,amueblado,url_imagen,reservado/sin_reserva,url_piso/id,contacto,officelink_url,nombre_oficina,barrios,cod_barrio,cod_barrio_2,distrito,cod_distrito,codigo_Postal
0,Apartamento en venta en CALLE Picos de Europa ...,54000,RP022023115210,1492,10062023,25,1.0,1.0,NaN,Consumo 393 KW hm2 ano,Emisiones 76 Kg CO2m2 ano,Cal energetica,Estado A reformar,Antiguedad Mas de 50 anos,0.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Baldosa,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,httpswwwredpisoesuploadpropertiesthumbsimg_092...,0.0,httpswwwredpisoesinmuebleapartamentoenventaenc...,914786966,httpswwwredpisoesoficinainmobiliariavallecasal...,REDPISO VALLECAS ALBUFERA NUEVA NUMANCIA,numancia,136.0,6.0,puente de vallecas,13.0,28038.0
1,Piso en venta en CALLE QUIJADA DE PANDIELLOS S...,67000,RP152024125550,722,15042024,48,1.0,1.0,1.0,NaN,NaN,NaN,Estado Depende del precio,NaN,0.0,1.0,NaN,Agua caliente Gas natural,Calefaccion Gas Natural,NaN,1.0,NaN,38,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallequi...,914349403,httpswwwredpisoesoficinainmobiliariavallecaspu...,REDPISO VALLECASPUENTE VALLECAS,san diego,132.0,2.0,puente de vallecas,13.0,28053.0
2,Piso en venta en CALLE ENRIQUE VELASCO Nueva N...,69000,RP02202294300,2190,17012022,35,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Ano de construccion 1940,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencalleenr...,914786966,httpswwwredpisoesoficinainmobiliariavallecasal...,REDPISO VALLECAS ALBUFERA NUEVA NUMANCIA,numancia,136.0,6.0,puente de vallecas,13.0,28038.0
3,Piso en venta en CALLE LAVAPIES Embajadores Ce...,69000,RP1222023116499,2285,11072023,28,1.0,1.0,NaN,NaN,NaN,Cal energetica,Estado A reformar,NaN,1.0,0.0,Orientacion Oeste,NaN,NaN,Tipo de fachada Mono Capa,0.0,NaN,27,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallelav...,911921007,httpswwwredpisoesoficinainmobiliarialavapiesem...,REDPISO LAVAPIESEMBAJADORES,embajadores,12.0,2.0,centro,1.0,28012.0
4,Piso en venta en CALLE VIZCONDE DE ARLESSON En...,81500,RP172024123241,713,05022024,30,2.0,1.0,NaN,NaN,NaN,NaN,Estado A reformar,Ano de construccion 1960,1.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Gres,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,1.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,entrevias,131.0,1.0,puente de vallecas,13.0,28053.0


In [294]:
redpiso_unido2.isna().sum()

titulo                            0
precio                            0
referencia_web                  680
cantidad_visitas                680
fecha_publicacion               680
espacio_metros_cuadrados          0
cantidad_dormitorios_x          680
cantidad_banos                  687
cantidad_armarios              1566
consumo_energia                1894
emisiones_CO2                  1893
cal_energetica                 1158
condicino-estado                899
antiguedad                     1823
ascensor                        680
aire_acondicionado              680
orientacion                    1395
sistema_agua                   1460
sistema_calefaccion            1573
Tipo de suelo_fachada          1640
cocina_equipada                 680
garajes                        1848
m_cuadrados_utiles             1062
vista zona                     1022
piscina                         680
trastero                        680
Comunidad/personas edificio    2123
Zonas verdes                

In [295]:
redpiso_unido2

,titulo,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,cantidad_dormitorios_x,cantidad_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,aire_acondicionado,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,piscina,trastero,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,alarma,amueblado,url_imagen,reservado/sin_reserva,url_piso/id,contacto,officelink_url,nombre_oficina,barrios,cod_barrio,cod_barrio_2,distrito,cod_distrito,codigo_Postal
0,Apartamento en venta en CALLE Picos de Europa ...,54000,RP022023115210,1492,10062023,25,1.0,1.0,NaN,Consumo 393 KW hm2 ano,Emisiones 76 Kg CO2m2 ano,Cal energetica,Estado A reformar,Antiguedad Mas de 50 anos,0.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Baldosa,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,httpswwwredpisoesuploadpropertiesthumbsimg_092...,0.0,httpswwwredpisoesinmuebleapartamentoenventaenc...,914786966,httpswwwredpisoesoficinainmobiliariavallecasal...,REDPISO VALLECAS ALBUFERA NUEVA NUMANCIA,numancia,136.0,6.0,puente de vallecas,13.0,28038.0
1,Piso en venta en CALLE QUIJADA DE PANDIELLOS S...,67000,RP152024125550,722,15042024,48,1.0,1.0,1.0,NaN,NaN,NaN,Estado Depende del precio,NaN,0.0,1.0,NaN,Agua caliente Gas natural,Calefaccion Gas Natural,NaN,1.0,NaN,38,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallequi...,914349403,httpswwwredpisoesoficinainmobiliariavallecaspu...,REDPISO VALLECASPUENTE VALLECAS,san diego,132.0,2.0,puente de vallecas,13.0,28053.0
2,Piso en venta en CALLE ENRIQUE VELASCO Nueva N...,69000,RP02202294300,2190,17012022,35,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Ano de construccion 1940,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencalleenr...,914786966,httpswwwredpisoesoficinainmobiliariavallecasal...,REDPISO VALLECAS ALBUFERA NUEVA NUMANCIA,numancia,136.0,6.0,puente de vallecas,13.0,28038.0
3,Piso en venta en CALLE LAVAPIES Embajadores Ce...,69000,RP1222023116499,2285,11072023,28,1.0,1.0,NaN,NaN,NaN,Cal energetica,Estado A reformar,NaN,1.0,0.0,Orientacion Oeste,NaN,NaN,Tipo de fachada Mono Capa,0.0,NaN,27,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallelav...,911921007,httpswwwredpisoesoficinainmobiliarialavapiesem...,REDPISO LAVAPIESEMBAJADORES,embajadores,12.0,2.0,centro,1.0,28012.0
4,Piso en venta en CALLE VIZCONDE DE ARLESSON En...,81500,RP172024123241,713,05022024,30,2.0,1.0,NaN,NaN,NaN,NaN,Estado A reformar,Ano de construccion 1960,1.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Gres,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,1.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,entrevias,131.0,1.0,puente de vallecas,13.0,28053.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2149,Piso en venta en PASEO EDUARDO DATO Almagro Ch...,3850000,NaN,NaN,NaN,378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,httpswwwredpisoesuploadpropertiesthumbsimg_522...,0.0,httpswwwredpisoesinmueblepisoenventaenpaseoedu...,910378491,httpswwwredpisoesoficinainmobiliariachamberial...,REDPISO CHAMBERÍ ALMAGRO,almagro,74.0,4.0,chamberi,7.0,28010.0
2150,Piso en venta en AVENIDA MENENDEZ PELAYO Ibiza...,4000000,RP962024130172,150,12092024,300,5.0,5.0,NaN,NaN,NaN,Cal energetica,Estado Perfecto Estado,NaN,1.0,0.0,Orientacion Sur,Agua caliente Individual,NaN,NaN,1.0,NaN,260,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,ibiza,34.0,4.0,retiro,3.0,28009.0
2151,Piso en venta en AVENIDA MENÉNDEZ PELA

------------------------------------------

### **_Aqui se termina la union de redpiso y empieza la union de redpiso y pisosmadrid_**

In [296]:
pisos_madrid_unido.shape

(1157, 28)

In [297]:
redpiso_unido2.shape

(2154, 48)

In [298]:
pisos_madrid_unido.head(5)

,url_website,descripcion_y,cant_dormitorios_y,cant_banos_y,metros_cuadrados_construidos_y,descripcion_extendida_y,tipo_lugar,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio_y,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,barrios,distrito,cod_distrito,cod_barrio,cod_barrio_2,codigo_Postal
0,https://www.pisosmadrid.com.es/propiedad/40866...,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso reformado de 60m2 enfrente del R...,Piso,0,0,0,PM4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0,Ibiza,Retiro,3,34,4,28009
1,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,0,1,1,PM4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0,Ensanche de Vallecas,Villa de Vallecas,18,183,3,28031
2,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,0,0,0,PM4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0,Delicias,Arganzuela,2,25,5,28045
3,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en Los Angeles Madrid,3,1,57,Vivienda situada en Calle Godella en Madrid ca...,Piso,0,0,0,PM4086495,2024-10-24,40,70256,0,0,0,0,0,0,0,0,los angeles,Villaverde,17,174,4,28021
5,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en San Diego Madrid,1,1,21,Vivienda situada en Calle Callejo en Madrid ca...,Piso,0,0,0,PM4086488,2024-10-24,20,69313,0,0,0,0,0,0,0,0,San Diego,Puente de Vallecas,13,132,2,28053


In [299]:
# renombramiento de columnas 

pisos_madrid_unido=pisos_madrid_unido.rename(columns={"descripcion_y":"titulo"})
pisos_madrid_unido=pisos_madrid_unido.rename(columns={"tipo_lugar":"tipologia"})
pisos_madrid_unido=pisos_madrid_unido.rename(columns={"metros_cuadrados_construidos_y":"espacio_metros_cuadrados"})


In [300]:
# renombramiento de columnas 
redpiso_nombres_cambiar= {
    "cantidad_banos"      :"cant_banos",
    "cantidad_dormitorios_x":"cant_dormitorios",
    
}

redpiso_unido2=redpiso_unido2.rename(columns=redpiso_nombres_cambiar)

In [301]:
redpiso_unido2.head(5)

,titulo,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,cant_dormitorios,cant_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,aire_acondicionado,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,piscina,trastero,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,alarma,amueblado,url_imagen,reservado/sin_reserva,url_piso/id,contacto,officelink_url,nombre_oficina,barrios,cod_barrio,cod_barrio_2,distrito,cod_distrito,codigo_Postal
0,Apartamento en venta en CALLE Picos de Europa ...,54000,RP022023115210,1492,10062023,25,1.0,1.0,NaN,Consumo 393 KW hm2 ano,Emisiones 76 Kg CO2m2 ano,Cal energetica,Estado A reformar,Antiguedad Mas de 50 anos,0.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Baldosa,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,httpswwwredpisoesuploadpropertiesthumbsimg_092...,0.0,httpswwwredpisoesinmuebleapartamentoenventaenc...,914786966,httpswwwredpisoesoficinainmobiliariavallecasal...,REDPISO VALLECAS ALBUFERA NUEVA NUMANCIA,numancia,136.0,6.0,puente de vallecas,13.0,28038.0
1,Piso en venta en CALLE QUIJADA DE PANDIELLOS S...,67000,RP152024125550,722,15042024,48,1.0,1.0,1.0,NaN,NaN,NaN,Estado Depende del precio,NaN,0.0,1.0,NaN,Agua caliente Gas natural,Calefaccion Gas Natural,NaN,1.0,NaN,38,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallequi...,914349403,httpswwwredpisoesoficinainmobiliariavallecaspu...,REDPISO VALLECASPUENTE VALLECAS,san diego,132.0,2.0,puente de vallecas,13.0,28053.0
2,Piso en venta en CALLE ENRIQUE VELASCO Nueva N...,69000,RP02202294300,2190,17012022,35,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Ano de construccion 1940,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencalleenr...,914786966,httpswwwredpisoesoficinainmobiliariavallecasal...,REDPISO VALLECAS ALBUFERA NUEVA NUMANCIA,numancia,136.0,6.0,puente de vallecas,13.0,28038.0
3,Piso en venta en CALLE LAVAPIES Embajadores Ce...,69000,RP1222023116499,2285,11072023,28,1.0,1.0,NaN,NaN,NaN,Cal energetica,Estado A reformar,NaN,1.0,0.0,Orientacion Oeste,NaN,NaN,Tipo de fachada Mono Capa,0.0,NaN,27,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallelav...,911921007,httpswwwredpisoesoficinainmobiliarialavapiesem...,REDPISO LAVAPIESEMBAJADORES,embajadores,12.0,2.0,centro,1.0,28012.0
4,Piso en venta en CALLE VIZCONDE DE ARLESSON En...,81500,RP172024123241,713,05022024,30,2.0,1.0,NaN,NaN,NaN,NaN,Estado A reformar,Ano de construccion 1960,1.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Gres,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,1.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,entrevias,131.0,1.0,puente de vallecas,13.0,28053.0


In [302]:
pisos_madrid_unido.head(5)

,url_website,titulo,cant_dormitorios_y,cant_banos_y,espacio_metros_cuadrados,descripcion_extendida_y,tipologia,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio_y,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,barrios,distrito,cod_distrito,cod_barrio,cod_barrio_2,codigo_Postal
0,https://www.pisosmadrid.com.es/propiedad/40866...,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso reformado de 60m2 enfrente del R...,Piso,0,0,0,PM4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0,Ibiza,Retiro,3,34,4,28009
1,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,0,1,1,PM4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0,Ensanche de Vallecas,Villa de Vallecas,18,183,3,28031
2,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,0,0,0,PM4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0,Delicias,Arganzuela,2,25,5,28045
3,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en Los Angeles Madrid,3,1,57,Vivienda situada en Calle Godella en Madrid ca...,Piso,0,0,0,PM4086495,2024-10-24,40,70256,0,0,0,0,0,0,0,0,los angeles,Villaverde,17,174,4,28021
5,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en San Diego Madrid,1,1,21,Vivienda situada en Calle Callejo en Madrid ca...,Piso,0,0,0,PM4086488,2024-10-24,20,69313,0,0,0,0,0,0,0,0,San Diego,Puente de Vallecas,13,132,2,28053


In [303]:
# eliminacion de columnas 

redpiso_columnas_eliminar=["url_imagen","nombre_oficina","contacto"]

redpiso_unido2 = redpiso_unido2.drop(columns=redpiso_columnas_eliminar)

In [304]:
redpiso_unido2 = redpiso_unido2.drop(columns=["officelink_url"])

In [305]:
# Aqui creare una columna con la tipologia del lugar y filtrare los datos por la columna titulo

redpiso_unido2["tipologia"]= np.nan
# Función para normalizar la cadena (eliminar acentos y convertir a minúsculas)
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

# Función para obtener la primera palabra de un string normalizado
def obtener_primera_palabra(s):
    # Verificamos si la cadena no es nula o vacía
    if pd.notna(s) and len(s) > 0:
        # Normalizamos la cadena antes de dividir
        s_normalizado = normalize_string(s)
        # Dividir el string por espacios y tomar la primera palabra
        return s_normalizado.split()[0]
    return ''

# Función que recorre la columna "titulo" y asigna la primera palabra a "tipologia"
def asignar_tipologia(df):
    # Recorrer el DataFrame y aplicar la función a cada fila de la columna "titulo"
    df['tipologia'] = df['titulo'].apply(obtener_primera_palabra)
    return df

# Aplicar la función a tu DataFrame redpiso_unido2
redpiso_unido2 = asignar_tipologia(redpiso_unido2)

In [306]:
pisos_madrid_unido.head(5)

,url_website,titulo,cant_dormitorios_y,cant_banos_y,espacio_metros_cuadrados,descripcion_extendida_y,tipologia,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio_y,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,barrios,distrito,cod_distrito,cod_barrio,cod_barrio_2,codigo_Postal
0,https://www.pisosmadrid.com.es/propiedad/40866...,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso reformado de 60m2 enfrente del R...,Piso,0,0,0,PM4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0,Ibiza,Retiro,3,34,4,28009
1,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,0,1,1,PM4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0,Ensanche de Vallecas,Villa de Vallecas,18,183,3,28031
2,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,0,0,0,PM4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0,Delicias,Arganzuela,2,25,5,28045
3,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en Los Angeles Madrid,3,1,57,Vivienda situada en Calle Godella en Madrid ca...,Piso,0,0,0,PM4086495,2024-10-24,40,70256,0,0,0,0,0,0,0,0,los angeles,Villaverde,17,174,4,28021
5,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en San Diego Madrid,1,1,21,Vivienda situada en Calle Callejo en Madrid ca...,Piso,0,0,0,PM4086488,2024-10-24,20,69313,0,0,0,0,0,0,0,0,San Diego,Puente de Vallecas,13,132,2,28053


In [307]:
redpiso_unido2.head(5)

,titulo,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,cant_dormitorios,cant_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,aire_acondicionado,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,piscina,trastero,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,alarma,amueblado,reservado/sin_reserva,url_piso/id,barrios,cod_barrio,cod_barrio_2,distrito,cod_distrito,codigo_Postal,tipologia
0,Apartamento en venta en CALLE Picos de Europa ...,54000,RP022023115210,1492,10062023,25,1.0,1.0,NaN,Consumo 393 KW hm2 ano,Emisiones 76 Kg CO2m2 ano,Cal energetica,Estado A reformar,Antiguedad Mas de 50 anos,0.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Baldosa,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,httpswwwredpisoesinmuebleapartamentoenventaenc...,numancia,136.0,6.0,puente de vallecas,13.0,28038.0,apartamento
1,Piso en venta en CALLE QUIJADA DE PANDIELLOS S...,67000,RP152024125550,722,15042024,48,1.0,1.0,1.0,NaN,NaN,NaN,Estado Depende del precio,NaN,0.0,1.0,NaN,Agua caliente Gas natural,Calefaccion Gas Natural,NaN,1.0,NaN,38,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,httpswwwredpisoesinmueblepisoenventaencallequi...,san diego,132.0,2.0,puente de vallecas,13.0,28053.0,piso
2,Piso en venta en CALLE ENRIQUE VELASCO Nueva N...,69000,RP02202294300,2190,17012022,35,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Ano de construccion 1940,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,httpswwwredpisoesinmueblepisoenventaencalleenr...,numancia,136.0,6.0,puente de vallecas,13.0,28038.0,piso
3,Piso en venta en CALLE LAVAPIES Embajadores Ce...,69000,RP1222023116499,2285,11072023,28,1.0,1.0,NaN,NaN,NaN,Cal energetica,Estado A reformar,NaN,1.0,0.0,Orientacion Oeste,NaN,NaN,Tipo de fachada Mono Capa,0.0,NaN,27,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,httpswwwredpisoesinmueblepisoenventaencallelav...,embajadores,12.0,2.0,centro,1.0,28012.0,piso
4,Piso en venta en CALLE VIZCONDE DE ARLESSON En...,81500,RP172024123241,713,05022024,30,2.0,1.0,NaN,NaN,NaN,NaN,Estado A reformar,Ano de construccion 1960,1.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Gres,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,1.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,entrevias,131.0,1.0,puente de vallecas,13.0,28053.0,piso


In [308]:
redpiso_unido2.dtypes

titulo                          object
precio                           int32
referencia_web                  object
cantidad_visitas                object
fecha_publicacion               object
espacio_metros_cuadrados         int32
cant_dormitorios               float64
cant_banos                     float64
cantidad_armarios              float64
consumo_energia                 object
emisiones_CO2                   object
cal_energetica                  object
condicino-estado                object
antiguedad                      object
ascensor                       float64
aire_acondicionado             float64
orientacion                     object
sistema_agua                    object
sistema_calefaccion             object
Tipo de suelo_fachada           object
cocina_equipada                float64
garajes                         object
m_cuadrados_utiles              object
vista zona                      object
piscina                        float64
trastero                 

In [309]:
pisos_madrid_unido.dtypes

url_website                         object
titulo                              object
cant_dormitorios_y                   int64
cant_banos_y                         int64
espacio_metros_cuadrados             int64
descripcion_extendida_y             object
tipologia                           object
piscina                              int64
garaje                               int64
jardin                               int64
referencia                          object
fecha_publicacion           datetime64[ns]
n.visitas                            int64
precio_y                             int64
calefaccion                          int64
trastero                             int64
amueblado                            int64
chimenea                             int64
lavadero                             int64
videoportero                         int64
alarma                               int64
aire_acondicionado                   int64
barrios                             object
distrito   

In [310]:
redpiso_unido2["tipologia"].value_counts()

tipologia
piso           1932
atico            76
apartamento      36
estudio          32
casa             31
duplex           22
loft             11
chalet            9
venta             5
Name: count, dtype: int64

In [311]:
redpiso_unido2["tipologia"].isna().sum()

0

In [312]:
redpiso_unido2.shape

(2154, 45)

In [313]:
pisos_madrid_unido.shape

(1157, 28)

In [314]:
redpiso_unido2.head(5)

,titulo,precio,referencia_web,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,cant_dormitorios,cant_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,aire_acondicionado,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,piscina,trastero,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,alarma,amueblado,reservado/sin_reserva,url_piso/id,barrios,cod_barrio,cod_barrio_2,distrito,cod_distrito,codigo_Postal,tipologia
0,Apartamento en venta en CALLE Picos de Europa ...,54000,RP022023115210,1492,10062023,25,1.0,1.0,NaN,Consumo 393 KW hm2 ano,Emisiones 76 Kg CO2m2 ano,Cal energetica,Estado A reformar,Antiguedad Mas de 50 anos,0.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Baldosa,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,httpswwwredpisoesinmuebleapartamentoenventaenc...,numancia,136.0,6.0,puente de vallecas,13.0,28038.0,apartamento
1,Piso en venta en CALLE QUIJADA DE PANDIELLOS S...,67000,RP152024125550,722,15042024,48,1.0,1.0,1.0,NaN,NaN,NaN,Estado Depende del precio,NaN,0.0,1.0,NaN,Agua caliente Gas natural,Calefaccion Gas Natural,NaN,1.0,NaN,38,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,httpswwwredpisoesinmueblepisoenventaencallequi...,san diego,132.0,2.0,puente de vallecas,13.0,28053.0,piso
2,Piso en venta en CALLE ENRIQUE VELASCO Nueva N...,69000,RP02202294300,2190,17012022,35,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Ano de construccion 1940,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,httpswwwredpisoesinmueblepisoenventaencalleenr...,numancia,136.0,6.0,puente de vallecas,13.0,28038.0,piso
3,Piso en venta en CALLE LAVAPIES Embajadores Ce...,69000,RP1222023116499,2285,11072023,28,1.0,1.0,NaN,NaN,NaN,Cal energetica,Estado A reformar,NaN,1.0,0.0,Orientacion Oeste,NaN,NaN,Tipo de fachada Mono Capa,0.0,NaN,27,Exterior,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,httpswwwredpisoesinmueblepisoenventaencallelav...,embajadores,12.0,2.0,centro,1.0,28012.0,piso
4,Piso en venta en CALLE VIZCONDE DE ARLESSON En...,81500,RP172024123241,713,05022024,30,2.0,1.0,NaN,NaN,NaN,NaN,Estado A reformar,Ano de construccion 1960,1.0,0.0,NaN,Agua caliente Gas Butano,NaN,Tipo de suelo Gres,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,1.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,entrevias,131.0,1.0,puente de vallecas,13.0,28053.0,piso


In [315]:
# renombramiento de columnas 
pisos_madrid_unido=pisos_madrid_unido.rename(columns={"cant_dormitorios_y":"cant_dormitorios"})
pisos_madrid_unido=pisos_madrid_unido.rename(columns={"cant_banos_y":"cant_banos"})
pisos_madrid_unido=pisos_madrid_unido.rename(columns={"descripcion_extendida_y":"descripcion_extendida"})
pisos_madrid_unido=pisos_madrid_unido.rename(columns={"precio_y":"precio"})

In [316]:
pisos_madrid_unido.head(5)

,url_website,titulo,cant_dormitorios,cant_banos,espacio_metros_cuadrados,descripcion_extendida,tipologia,piscina,garaje,jardin,referencia,fecha_publicacion,n.visitas,precio,calefaccion,trastero,amueblado,chimenea,lavadero,videoportero,alarma,aire_acondicionado,barrios,distrito,cod_distrito,cod_barrio,cod_barrio_2,codigo_Postal
0,https://www.pisosmadrid.com.es/propiedad/40866...,Piso en venta en Ibiza Madrid,3,1,70,Precioso piso reformado de 60m2 enfrente del R...,Piso,0,0,0,PM4086665,2024-10-24,40,520000,1,0,0,0,0,0,0,0,Ibiza,Retiro,3,34,4,28009
1,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Madrid,3,2,95,SERVICHECK INMOBILIARIA comercializa un precio...,Piso,0,1,1,PM4086544,2024-10-24,20,329000,1,1,1,0,0,1,0,0,Ensanche de Vallecas,Villa de Vallecas,18,183,3,28031
2,https://www.pisosmadrid.com.es/propiedad/40865...,Piso en venta en Arganzuela Madrid,4,3,158,Se ofrece una magnifica oportunidad para adqui...,Piso,0,0,0,PM4086503,2024-10-24,30,740000,1,0,0,0,0,1,1,0,Delicias,Arganzuela,2,25,5,28045
3,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en Los Angeles Madrid,3,1,57,Vivienda situada en Calle Godella en Madrid ca...,Piso,0,0,0,PM4086495,2024-10-24,40,70256,0,0,0,0,0,0,0,0,los angeles,Villaverde,17,174,4,28021
5,https://www.pisosmadrid.com.es/propiedad/40864...,Piso en venta en San Diego Madrid,1,1,21,Vivienda situada en Calle Callejo en Madrid ca...,Piso,0,0,0,PM4086488,2024-10-24,20,69313,0,0,0,0,0,0,0,0,San Diego,Puente de Vallecas,13,132,2,28053


In [317]:
# normalizacion de columnas 

lista=["codigo_Postal","cant_dormitorios","espacio_metros_cuadrados","tipologia",
       "barrios","distrito","cod_distrito","cod_barrio","cant_banos"]

for col in lista:
    pisos_madrid_unido[col] = pisos_madrid_unido[col].astype(str).str.lower()


for col in lista:
    redpiso_unido2[col] = redpiso_unido2[col].astype(str).str.lower()

In [318]:
# merge de pisosmadrid y redpiso 
redpiso_pisosmadrid= pd.merge(redpiso_unido2,pisos_madrid_unido,on=["codigo_Postal","cant_dormitorios","espacio_metros_cuadrados","tipologia",
                                                                    "barrios","distrito","cod_distrito","cod_barrio","cant_banos"],how="outer")

In [319]:
redpiso_pisosmadrid.shape

(3311, 64)

In [320]:
# eliminacion de columnas duplicadas y otra cosa que nose como explicarla que ayuda a mejorar el orden de los datos . es como combinar los datos 
# pero sin mezclar los pisos 

for col in redpiso_pisosmadrid.columns:
    if '_x' in col:
        col_base = col.replace('_x', '')
        redpiso_pisosmadrid[col_base] = redpiso_pisosmadrid[col].fillna(redpiso_pisosmadrid.get(col_base + '_y'))
        redpiso_pisosmadrid = redpiso_pisosmadrid.drop(columns=[col, col_base + '_y'])

In [321]:
redpiso_pisosmadrid.shape

(3311, 55)

In [322]:
redpiso_pisosmadrid.isna().sum()

referencia_web                 1837
cantidad_visitas               1837
espacio_metros_cuadrados          0
cant_dormitorios                  0
cant_banos                        0
cantidad_armarios              2723
consumo_energia                3051
emisiones_CO2                  3050
cal_energetica                 2315
condicino-estado               2056
antiguedad                     2980
ascensor                       1837
orientacion                    2552
sistema_agua                   2617
sistema_calefaccion            2730
Tipo de suelo_fachada          2797
cocina_equipada                1837
garajes                        3005
m_cuadrados_utiles             2219
vista zona                     2179
Comunidad/personas edificio    3280
Zonas verdes                   1837
terrazas                       3267
acceso_minusvalido             1837
puerta_seguridad               1837
conserje                       1837
plantas                        3291
N_piso                      

In [323]:
redpiso_pisosmadrid.head(5)

,referencia_web,cantidad_visitas,espacio_metros_cuadrados,cant_dormitorios,cant_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,reservado/sin_reserva,url_piso/id,barrios,cod_barrio,distrito,cod_distrito,codigo_Postal,tipologia,url_website,descripcion_extendida,garaje,jardin,referencia,n.visitas,calefaccion,chimenea,lavadero,videoportero,titulo,precio,fecha_publicacion,aire_acondicionado,piscina,trastero,alarma,amueblado,cod_barrio_2
0,NaN,NaN,12,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39778...,Servicheck presenta las mejores plazas de gara...,0.0,0.0,PM3974960,70.0,0.0,0.0,0.0,0.0,Garaje en venta en Tetuan Madrid,25000.0,2024-08-01 00:00:00,0.0,0.0,0.0,0.0,0.0,2
1,NaN,NaN,24,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39473...,Garaje 0 dormitorios 0 banos 0 garajes Cerrado...,0.0,0.0,PM3945003,60.0,0.0,0.0,0.0,0.0,Garaje en venta en Pau Carabanchel Madrid,15500.0,2024-07-11 00:00:00,0.0,0.0,0.0,0.0,0.0,2
2,NaN,NaN,340,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recoletos,41,salamanca,4,28001,negocio,https://www.pisosmadrid.com.es/propiedad/40017...,TGG INMOBILIARIA vende en exclusiva NEGOCIO DE...,1.0,0.0,PM4000060,50.0,1.0,0.0,0.0,0.0,Negocio en venta en Recoletos Madrid,3000000.0,2024-08-29 00:00:00,0.0,0.0,1.0,1.0,1.0,1
3,NaN,NaN,399,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,local,https://www.pisosmadrid.com.es/propiedad/40407...,Amplio local o nave de 399 m2 distribuido en d...,0.0,0.0,PM4040343,30.0,0.0,0.0,0.0,0.0,Local en venta en Fuencarral Madrid,409100.0,2024-09-26 00:00:00,0.0,0.0,0.0,0.0,0.0,2
4,NaN,NaN,442,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recoletos,41,salamanca,4,28001,negocio,https://www.pisosmadrid.com.es/propiedad/39862...,TGG INMOBILIARIA vende en exclusiva NEGOCIO DE...,0.0,0.0,PM3984633,50.0,1.0,0.0,0.0,0.0,Negocio en venta en Recoletos Madrid,8000000.0,2024-08-09 00:00:00,0.0,0.0,0.0,1.0,1.0,1


In [324]:
redpiso_pisosmadrid

,referencia_web,cantidad_visitas,espacio_metros_cuadrados,cant_dormitorios,cant_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,orientacion,sistema_agua,sistema_calefaccion,Tipo de suelo_fachada,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,reservado/sin_reserva,url_piso/id,barrios,cod_barrio,distrito,cod_distrito,codigo_Postal,tipologia,url_website,descripcion_extendida,garaje,jardin,referencia,n.visitas,calefaccion,chimenea,lavadero,videoportero,titulo,precio,fecha_publicacion,aire_acondicionado,piscina,trastero,alarma,amueblado,cod_barrio_2
0,NaN,NaN,12,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39778...,Servicheck presenta las mejores plazas de gara...,0.0,0.0,PM3974960,70.0,0.0,0.0,0.0,0.0,Garaje en venta en Tetuan Madrid,25000.0,2024-08-01 00:00:00,0.0,0.0,0.0,0.0,0.0,2
1,NaN,NaN,24,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39473...,Garaje 0 dormitorios 0 banos 0 garajes Cerrado...,0.0,0.0,PM3945003,60.0,0.0,0.0,0.0,0.0,Garaje en venta en Pau Carabanchel Madrid,15500.0,2024-07-11 00:00:00,0.0,0.0,0.0,0.0,0.0,2
2,NaN,NaN,340,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recoletos,41,salamanca,4,28001,negocio,https://www.pisosmadrid.com.es/propiedad/40017...,TGG INMOBILIARIA vende en exclusiva NEGOCIO DE...,1.0,0.0,PM4000060,50.0,1.0,0.0,0.0,0.0,Negocio en venta en Recoletos Madrid,3000000.0,2024-08-29 00:00:00,0.0,0.0,1.0,1.0,1.0,1
3,NaN,NaN,399,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,local,https://www.pisosmadrid.com.es/propiedad/40407...,Amplio local o nave de 399 m2 distribuido en d...,0.0,0.0,PM4040343,30.0,0.0,0.0,0.0,0.0,Local en venta en Fuencarral Madrid,409100.0,2024-09-26 00:00:00,0.0,0.0,0.0,0.0,0.0,2
4,NaN,NaN,442,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recoletos,41,salamanca,4,28001,negocio,https://www.pisosmadrid.com.es/propiedad/39862...,TGG INMOBILIARIA vende en exclusiva NEGOCIO DE...,0.0,0.0,PM3984633,50.0,1.0,0.0,0.0,0.0,Negocio en venta en Recoletos Madrid,8000000.0,2024-08-09 00:00:00,0.0,0.0,0.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3306,NaN,NaN,36,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,httpswwwredpisoesinmuebleduplexenventaencallej...,nan,nan,barajas,21.0,nan,duplex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dúplex en venta en CALLE JUPITER Casco Históri...,190000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3307,NaN,NaN,57,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallenue...,nan,nan,barajas,21.0,nan,piso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Piso en venta en CALLE NUESTRA SEÑORA DE ARACE...,205000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3308,NaN,NaN,63,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencalleemp...,nan,nan,barajas,21.0,nan,piso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Piso en venta en CALLE EMPEDRADA Casco Históri...,232000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3309,NaN,NaN,68,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,httpswwwredpisoesinmueblepisoenventaencalleemp..

In [325]:
redpiso_pisosmadrid["Tipo de suelo_fachada"].value_counts()

Tipo de suelo_fachada
Tipo de suelo Gres                  129
Tipo de suelo Parquet               112
Tipo de fachada Ladrillo             70
Tipo de suelo Tarima normal          61
Tipo de suelo Tarima Flotante        53
Tipo de suelo Baldosa                16
Tipo de suelo Sintasol               15
Tipo de suelo Terrazo                14
Tipo de suelo Otros                  12
Tipo de fachada Mono Capa            11
Tipo de fachada Cemento               8
Tipo de suelo Porcelanico             6
Tipo de suelo Mosaico Hidraulico      3
Tipo de fachada Piedra                2
Tipo de suelo Marmol                  1
Tipo de fachada Otros materiales      1
Name: count, dtype: int64

In [326]:
# Aqui vamos a filtrar la informacion de tipo_fachada y la dividire en dos columnas donde se veran los valores de fachada y suelo separados
redpiso_pisosmadrid["tipo_fachada"]= np.nan

def asignar_fachada(df):
    # Recorrer cada fila de la columna Tipo de suelo/fachada
    for idx, fila in df.iterrows():
        tipo_suelo_fachada = str(fila['Tipo de suelo_fachada'])  # Asegurarse de que sea un string
        # Convertir el texto a minúsculas para la búsqueda sin importar mayúsculas
        if 'fachada' in tipo_suelo_fachada.lower():
            # Si encuentra la palabra 'fachada', asignar todo el texto a la columna tipo_fachada
            df.at[idx, 'tipo_fachada'] = tipo_suelo_fachada
            # Reemplazar el valor original en Tipo de suelo/fachada por NaN
            df.at[idx, 'Tipo de suelo_fachada'] = np.nan
    return df

# Llamar a la función pasándole tu dataframe
redpiso_pisosmadrid = asignar_fachada(redpiso_pisosmadrid)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20088\1571698920.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Tipo de fachada Mono Capa' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'tipo_fachada'] = tipo_suelo_fachada


In [327]:
redpiso_pisosmadrid["Tipo de suelo_fachada"].value_counts()

Tipo de suelo_fachada
Tipo de suelo Gres                  129
Tipo de suelo Parquet               112
Tipo de suelo Tarima normal          61
Tipo de suelo Tarima Flotante        53
Tipo de suelo Baldosa                16
Tipo de suelo Sintasol               15
Tipo de suelo Terrazo                14
Tipo de suelo Otros                  12
Tipo de suelo Porcelanico             6
Tipo de suelo Mosaico Hidraulico      3
Tipo de suelo Marmol                  1
Name: count, dtype: int64

In [328]:
redpiso_pisosmadrid["tipo_fachada"].value_counts()

tipo_fachada
Tipo de fachada Ladrillo            70
Tipo de fachada Mono Capa           11
Tipo de fachada Cemento              8
Tipo de fachada Piedra               2
Tipo de fachada Otros materiales     1
Name: count, dtype: int64

In [329]:
# renombramiento de columna 
redpiso_pisosmadrid= redpiso_pisosmadrid.rename(columns={"Tipo de suelo_fachada":"tipo_suelo"})

### **_Desde aqui empiezan los ajustes de compañera Silvia_**

In [330]:
# Nueva variable para cualquier cosa me regreso y cargo solo esta
redpiso_pisosmadrid2 =redpiso_pisosmadrid.copy()

In [331]:
redpiso_pisosmadrid2.head()

,referencia_web,cantidad_visitas,espacio_metros_cuadrados,cant_dormitorios,cant_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,orientacion,sistema_agua,sistema_calefaccion,tipo_suelo,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,reservado/sin_reserva,url_piso/id,barrios,cod_barrio,distrito,cod_distrito,codigo_Postal,tipologia,url_website,descripcion_extendida,garaje,jardin,referencia,n.visitas,calefaccion,chimenea,lavadero,videoportero,titulo,precio,fecha_publicacion,aire_acondicionado,piscina,trastero,alarma,amueblado,cod_barrio_2,tipo_fachada
0,NaN,NaN,12,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39778...,Servicheck presenta las mejores plazas de gara...,0.0,0.0,PM3974960,70.0,0.0,0.0,0.0,0.0,Garaje en venta en Tetuan Madrid,25000.0,2024-08-01 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN
1,NaN,NaN,24,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39473...,Garaje 0 dormitorios 0 banos 0 garajes Cerrado...,0.0,0.0,PM3945003,60.0,0.0,0.0,0.0,0.0,Garaje en venta en Pau Carabanchel Madrid,15500.0,2024-07-11 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN
2,NaN,NaN,340,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recoletos,41,salamanca,4,28001,negocio,https://www.pisosmadrid.com.es/propiedad/40017...,TGG INMOBILIARIA vende en exclusiva NEGOCIO DE...,1.0,0.0,PM4000060,50.0,1.0,0.0,0.0,0.0,Negocio en venta en Recoletos Madrid,3000000.0,2024-08-29 00:00:00,0.0,0.0,1.0,1.0,1.0,1,NaN
3,NaN,NaN,399,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,local,https://www.pisosmadrid.com.es/propiedad/40407...,Amplio local o nave de 399 m2 distribuido en d...,0.0,0.0,PM4040343,30.0,0.0,0.0,0.0,0.0,Local en venta en Fuencarral Madrid,409100.0,2024-09-26 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN
4,NaN,NaN,442,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recoletos,41,salamanca,4,28001,negocio,https://www.pisosmadrid.com.es/propiedad/39862...,TGG INMOBILIARIA vende en exclusiva NEGOCIO DE...,0.0,0.0,PM3984633,50.0,1.0,0.0,0.0,0.0,Negocio en venta en Recoletos Madrid,8000000.0,2024-08-09 00:00:00,0.0,0.0,0.0,1.0,1.0,1,NaN


In [332]:
redpiso_pisosmadrid2["antiguedad"].value_counts()

antiguedad
Antiguedad Mas de 50 anos         131
Antiguedad Entre 35 y 50 Anos      29
Ano de construccion 1960           15
Ano de construccion 1965           15
Ano de construccion 1930           13
Ano de construccion 1940            9
Ano de construccion 1964            7
Ano de construccion 1972            7
Antiguedad Mas de 25 Anos           6
Ano de construccion 1970            6
Ano de construccion 1925            5
Ano de construccion 1962            5
Ano de construccion 1966            5
Ano de construccion 1950            4
Ano de construccion 1900            4
Ano de construccion 2000            4
Ano de construccion 1959            4
Ano de construccion 1920            3
Antiguedad Menos de 5 Anos          3
Ano de construccion 1968            3
Ano de construccion 1961            3
Ano de construccion 1969            3
Ano de construccion 1955            3
Ano de construccion 1973            2
Ano de construccion 1967            2
Ano de construccion 1923            2
A

In [333]:
# dfi= pd.read_feather('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/df_Indomio22122024.ftr')
# dfr_pm= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/redpiso_pisosmadrid_unidos.csv')

In [334]:
# dfi.columns

In [335]:

# # Eliminar el símbolo € y la cadena /mes
# dfi['€_comunidad_mes'] = dfi['gastos_comunidad'].replace({'€ ': '', '/mes': ''}, regex=True)
# dfi['acceso_minusvalido']=dfi['acceso_minusv'].str.replace('No','0').str.replace('Sí','1')
# #elimino la columna
# dfi=dfi.drop("acceso_minusv",axis=1)
# dfi=dfi.drop("gastos_comunidad",axis=1)

#en la columna antiguedad de redpiso_pisosmadrid2 quitar el texto año Ano de construccion del valor
redpiso_pisosmadrid2['antiguedad']=redpiso_pisosmadrid2['antiguedad'].str.replace('Ano de construccion','')
redpiso_pisosmadrid2['antiguedad']=redpiso_pisosmadrid2['antiguedad'].str.replace('Antiguedad','')
# redpiso_pisosmadrid2["plantas"]=redpiso_pisosmadrid2['plantas'].str.replace('Plantas','')
# Crear la nueva columna 'exterior' y asignar el valor 1 cuando 'vista zona' sea 'Exterior'
# redpiso_pisosmadrid2=redpiso_pisosmadrid2.drop("plantas",axis=1)

redpiso_pisosmadrid2['exterior'] = redpiso_pisosmadrid2['vista zona'].apply(lambda x: 1 if x == 'Exterior' else np.nan)
# redpiso_pisosmadrid2=redpiso_pisosmadrid2.drop("vista zona",axis=1)


In [336]:
redpiso_pisosmadrid2["antiguedad"].value_counts()

antiguedad
Mas de 50 anos         131
Entre 35 y 50 Anos      29
1960                    15
1965                    15
1930                    13
1940                     9
1964                     7
1972                     7
Mas de 25 Anos           6
1970                     6
1925                     5
1962                     5
1966                     5
1950                     4
1900                     4
2000                     4
1959                     4
1920                     3
Menos de 5 Anos          3
1968                     3
1961                     3
1969                     3
1955                     3
1973                     2
1967                     2
1923                     2
1994                     2
 Entre 20 y 25 Anos      2
1979                     2
1910                     2
1929                     2
1957                     2
1986                     2
1958                     1
45                       1
1928                     1
1987             

In [337]:
redpiso_pisosmadrid2.head(1)

,referencia_web,cantidad_visitas,espacio_metros_cuadrados,cant_dormitorios,cant_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,orientacion,sistema_agua,sistema_calefaccion,tipo_suelo,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,reservado/sin_reserva,url_piso/id,barrios,cod_barrio,distrito,cod_distrito,codigo_Postal,tipologia,url_website,descripcion_extendida,garaje,jardin,referencia,n.visitas,calefaccion,chimenea,lavadero,videoportero,titulo,precio,fecha_publicacion,aire_acondicionado,piscina,trastero,alarma,amueblado,cod_barrio_2,tipo_fachada,exterior
0,NaN,NaN,12,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39778...,Servicheck presenta las mejores plazas de gara...,0.0,0.0,PM3974960,70.0,0.0,0.0,0.0,0.0,Garaje en venta en Tetuan Madrid,25000.0,2024-08-01 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN


In [338]:
# # Reemplazar "Sí" por 1 y "No" por 0 
# dfi['ascensor'] = dfi['ascensor'].replace({'Sí': 1, 'No': 0})
# dfi['balcon'] = dfi['balcon'].replace({'Sí': 1, 'No': 0})

In [339]:
# Diccionario con los nuevos nombres de las columnas
#voy a suponer que la superficie es superficie construida
nuevos_nombres ={'titulo': 'title',
'espacio_metros_cuadrados': 'm2_constr',
'm_cuadrados_utiles':'m2_utiles',
'cant_dormitorios': 'dormitorios',
'cant_banos': 'banos' ,
'condicino-estado':'estado',
'sistema_calefaccion':'calefaccion',
'antiguedad' :'ano_construccion'} 
redpiso_pisosmadrid2 = redpiso_pisosmadrid2.rename(columns=nuevos_nombres)

In [340]:
redpiso_pisosmadrid2.head(5)

,referencia_web,cantidad_visitas,m2_constr,dormitorios,banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,estado,ano_construccion,ascensor,orientacion,sistema_agua,calefaccion,tipo_suelo,cocina_equipada,garajes,m2_utiles,vista zona,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,reservado/sin_reserva,url_piso/id,barrios,cod_barrio,distrito,cod_distrito,codigo_Postal,tipologia,url_website,descripcion_extendida,garaje,jardin,referencia,n.visitas,calefaccion,chimenea,lavadero,videoportero,title,precio,fecha_publicacion,aire_acondicionado,piscina,trastero,alarma,amueblado,cod_barrio_2,tipo_fachada,exterior
0,NaN,NaN,12,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39778...,Servicheck presenta las mejores plazas de gara...,0.0,0.0,PM3974960,70.0,0.0,0.0,0.0,0.0,Garaje en venta en Tetuan Madrid,25000.0,2024-08-01 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN
1,NaN,NaN,24,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39473...,Garaje 0 dormitorios 0 banos 0 garajes Cerrado...,0.0,0.0,PM3945003,60.0,0.0,0.0,0.0,0.0,Garaje en venta en Pau Carabanchel Madrid,15500.0,2024-07-11 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN
2,NaN,NaN,340,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recoletos,41,salamanca,4,28001,negocio,https://www.pisosmadrid.com.es/propiedad/40017...,TGG INMOBILIARIA vende en exclusiva NEGOCIO DE...,1.0,0.0,PM4000060,50.0,1.0,0.0,0.0,0.0,Negocio en venta en Recoletos Madrid,3000000.0,2024-08-29 00:00:00,0.0,0.0,1.0,1.0,1.0,1,NaN,NaN
3,NaN,NaN,399,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,local,https://www.pisosmadrid.com.es/propiedad/40407...,Amplio local o nave de 399 m2 distribuido en d...,0.0,0.0,PM4040343,30.0,0.0,0.0,0.0,0.0,Local en venta en Fuencarral Madrid,409100.0,2024-09-26 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN
4,NaN,NaN,442,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recoletos,41,salamanca,4,28001,negocio,https://www.pisosmadrid.com.es/propiedad/39862...,TGG INMOBILIARIA vende en exclusiva NEGOCIO DE...,0.0,0.0,PM3984633,50.0,1.0,0.0,0.0,0.0,Negocio en venta en Recoletos Madrid,8000000.0,2024-08-09 00:00:00,0.0,0.0,0.0,1.0,1.0,1,NaN,NaN


In [341]:
# redpiso_pisosmadrid2= redpiso_pisosmadrid2.rename(columns={"antiguedad ":"ano_antiguedad"})

In [342]:
# redpiso_pisosmadrid2["antieguedad"].value_counts()

In [343]:
# nuevos_nombres ={'sistema_alarma': 'alarma',
# 'cocina_1': 'cocina',
# 'orientacion_sur,_este':'orientacion_sur_este',
# 'orientacion_norte,_sur':'orientacion_norte_sur'
#  } 
# dfi = dfi.rename(columns=nuevos_nombres)

In [344]:
#de alguna manera calefaccion estaba duplicada, quito las columnas duplicadas
redpiso_pisosmadrid2 = redpiso_pisosmadrid2.loc[:, ~redpiso_pisosmadrid2.columns.duplicated()]

In [345]:
#quito las filas con precio nulo
redpiso_pisosmadrid2=redpiso_pisosmadrid2.query("precio.notna()")

In [346]:
redpiso_pisosmadrid2.shape

(3311, 56)

In [347]:
# dfi.columns

In [348]:
# dfi=dfi.drop("orientacion", axis=1)

In [349]:
# dfi.shape

In [350]:
# # Usar melt para transformar el DataFrame
# df_melted = pd.melt(dfi, id_vars=['url'], value_vars=['orientacion_este', 'orientacion_norte',
#        'orientacion_norte_sur', 'orientacion_oeste', 'orientacion_sur',
#        'orientacion_sur_este'],var_name='orientacion', value_name='valor')

# # Filtrar las filas donde el valor es 1
# df_melted = df_melted[df_melted['valor'] == 1]

# # Eliminar la columna 'valor'
# df_melted = df_melted.drop(columns=['valor'])

# # Extraer la orientación real
# df_melted['orientacion'] = df_melted['orientacion'].str.replace('orientacion_', '')

In [351]:
# dfi.shape

In [352]:
# #uno al dataframe dfi
# dfi=dfi.merge(df_melted,on="url",how="left")

In [353]:
# dfi.loc[dfi['orientacion'].notnull(), ['orientacion', 'orientacion_este', 'orientacion_norte', 'orientacion_norte_sur', 'orientacion_oeste', 'orientacion_sur', 'orientacion_sur_este']]


In [354]:
# # Eliminar las columnas originales
# dfi = dfi.drop(columns=['orientacion_este', 'orientacion_norte', 'orientacion_norte_sur', 'orientacion_oeste', 'orientacion_sur', 'orientacion_sur_este'])



In [355]:
# Asegurarse de que todos los nombres de las columnas sean cadenas de texto 
redpiso_pisosmadrid2.columns = redpiso_pisosmadrid2.columns.astype(str) 
# Eliminar espacios en blanco al principio o al final de los nombres de las columnas
redpiso_pisosmadrid2.columns = redpiso_pisosmadrid2.columns.str.strip()

In [356]:
# dfi.to_csv("dUnionSilvia23122024.csv")

In [357]:
# dfr_pm.to_csv("dUnionGilberto23122024.csv")

In [358]:
# def asignar_zona_distrito_barrio(row):
#     if row['zona'] == 'Sanchinarro':
#         row['distrito'] = 'Hortaleza'
#         row['barrio'] = 'Valdefuentes'
#     if row['zona'] == 'San Andrés':
#         row['distrito'] = 'Villaverde'
#         row['barrio'] = 'VillaverdeAlto,CascoHistóricodeVillaverde'
#     if row['zona'] == 'Virgen del Cortijo':
#         row['distrito'] = 'Hortaleza'
#         row['barrio'] = 'Valdefuentes'
#     if row['zona'] == 'Montecarmelo':
#         row['distrito'] = 'Fuencarral-El Pardo'
#         row['barrio'] = 'Mirasierra'

#     return row


In [359]:
redpiso_pisosmadrid2

,referencia_web,cantidad_visitas,m2_constr,dormitorios,banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,estado,ano_construccion,ascensor,orientacion,sistema_agua,calefaccion,tipo_suelo,cocina_equipada,garajes,m2_utiles,vista zona,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,reservado/sin_reserva,url_piso/id,barrios,cod_barrio,distrito,cod_distrito,codigo_Postal,tipologia,url_website,descripcion_extendida,garaje,jardin,referencia,n.visitas,chimenea,lavadero,videoportero,title,precio,fecha_publicacion,aire_acondicionado,piscina,trastero,alarma,amueblado,cod_barrio_2,tipo_fachada,exterior
0,NaN,NaN,12,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39778...,Servicheck presenta las mejores plazas de gara...,0.0,0.0,PM3974960,70.0,0.0,0.0,0.0,Garaje en venta en Tetuan Madrid,25000.0,2024-08-01 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN
1,NaN,NaN,24,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39473...,Garaje 0 dormitorios 0 banos 0 garajes Cerrado...,0.0,0.0,PM3945003,60.0,0.0,0.0,0.0,Garaje en venta en Pau Carabanchel Madrid,15500.0,2024-07-11 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN
2,NaN,NaN,340,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recoletos,41,salamanca,4,28001,negocio,https://www.pisosmadrid.com.es/propiedad/40017...,TGG INMOBILIARIA vende en exclusiva NEGOCIO DE...,1.0,0.0,PM4000060,50.0,0.0,0.0,0.0,Negocio en venta en Recoletos Madrid,3000000.0,2024-08-29 00:00:00,0.0,0.0,1.0,1.0,1.0,1,NaN,NaN
3,NaN,NaN,399,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,local,https://www.pisosmadrid.com.es/propiedad/40407...,Amplio local o nave de 399 m2 distribuido en d...,0.0,0.0,PM4040343,30.0,0.0,0.0,0.0,Local en venta en Fuencarral Madrid,409100.0,2024-09-26 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN
4,NaN,NaN,442,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recoletos,41,salamanca,4,28001,negocio,https://www.pisosmadrid.com.es/propiedad/39862...,TGG INMOBILIARIA vende en exclusiva NEGOCIO DE...,0.0,0.0,PM3984633,50.0,0.0,0.0,0.0,Negocio en venta en Recoletos Madrid,8000000.0,2024-08-09 00:00:00,0.0,0.0,0.0,1.0,1.0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3306,NaN,NaN,36,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,httpswwwredpisoesinmuebleduplexenventaencallej...,nan,nan,barajas,21.0,nan,duplex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dúplex en venta en CALLE JUPITER Casco Históri...,190000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3307,NaN,NaN,57,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallenue...,nan,nan,barajas,21.0,nan,piso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Piso en venta en CALLE NUESTRA SEÑORA DE ARACE...,205000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3308,NaN,NaN,63,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencalleemp...,nan,nan,barajas,21.0,nan,piso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Piso en venta en CALLE EMPEDRADA Casco Históri...,232000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3309,NaN,NaN,68,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,httpswwwredpisoesinmueblepisoenventaencalleemp...,nan,nan,ba

In [362]:
redpiso_pisosmadrid2.to_csv("redpiso_pisosmadrid.csv",index= False)